# Worldle Solver
First stab at this. Strategy: Decision tree with score = "average depth (number of turns) needed to get to correct solution". Will need to make some choices for how to pare that tree down ...

Second Stab: Construct it as a game, with "Hero" (the human player) and "Villain" (the wordle), acting as something like:
```
Game:
Villain.choose_word()
while turn < max_turns:
    turn += 1
    guess = Hero.play()
    score = Villain.evaluate(guess)
    if score == 'win':
        return f"victory in {turn} turns!"
    Hero.update(score)
if == max_turns: 
    return f"Hero loses, failed to guess {Villain.solution}"
```
Second framework is better and more versatile because it allows me implement several different strategies as separate "agents".

Third version: similar to second -- see `WordleGameUpdate`. Not sure yet if that's an improvement ... kinda just going in circles.

In [16]:
import random
import copy

In [17]:
from itertools import product
from collections import defaultdict
from math import isclose

In [18]:
# word lists
soln_list = sorted(["cigar", "rebut", "sissy", "humph", "awake", "blush", "focal", "evade", "naval", "serve", "heath", "dwarf", "model", "karma", "stink", "grade", "quiet", "bench", "abate", "feign", "major", "death", "fresh", "crust", "stool", "colon", "abase", "marry", "react", "batty", "pride", "floss", "helix", "croak", "staff", "paper", "unfed", "whelp", "trawl", "outdo", "adobe", "crazy", "sower", "repay", "digit", "crate", "cluck", "spike", "mimic", "pound", "maxim", "linen", "unmet", "flesh", "booby", "forth", "first", "stand", "belly", "ivory", "seedy", "print", "yearn", "drain", "bribe", "stout", "panel", "crass", "flume", "offal", "agree", "error", "swirl", "argue", "bleed", "delta", "flick", "totem", "wooer", "front", "shrub", "parry", "biome", "lapel", "start", "greet", "goner", "golem", "lusty", "loopy", "round", "audit", "lying", "gamma", "labor", "islet", "civic", "forge", "corny", "moult", "basic", "salad", "agate", "spicy", "spray", "essay", "fjord", "spend", "kebab", "guild", "aback", "motor", "alone", "hatch", "hyper", "thumb", "dowry", "ought", "belch", "dutch", "pilot", "tweed", "comet", "jaunt", "enema", "steed", "abyss", "growl", "fling", "dozen", "boozy", "erode", "world", "gouge", "click", "briar", "great", "altar", "pulpy", "blurt", "coast", "duchy", "groin", "fixer", "group", "rogue", "badly", "smart", "pithy", "gaudy", "chill", "heron", "vodka", "finer", "surer", "radio", "rouge", "perch", "retch", "wrote", "clock", "tilde", "store", "prove", "bring", "solve", "cheat", "grime", "exult", "usher", "epoch", "triad", "break", "rhino", "viral", "conic", "masse", "sonic", "vital", "trace", "using", "peach", "champ", "baton", "brake", "pluck", "craze", "gripe", "weary", "picky", "acute", "ferry", "aside", "tapir", "troll", "unify", "rebus", "boost", "truss", "siege", "tiger", "banal", "slump", "crank", "gorge", "query", "drink", "favor", "abbey", "tangy", "panic", "solar", "shire", "proxy", "point", "robot", "prick", "wince", "crimp", "knoll", "sugar", "whack", "mount", "perky", "could", "wrung", "light", "those", "moist", "shard", "pleat", "aloft", "skill", "elder", "frame", "humor", "pause", "ulcer", "ultra", "robin", "cynic", "agora", "aroma", "caulk", "shake", "pupal", "dodge", "swill", "tacit", "other", "thorn", "trove", "bloke", "vivid", "spill", "chant", "choke", "rupee", "nasty", "mourn", "ahead", "brine", "cloth", "hoard", "sweet", "month", "lapse", "watch", "today", "focus", "smelt", "tease", "cater", "movie", "lynch", "saute", "allow", "renew", "their", "slosh", "purge", "chest", "depot", "epoxy", "nymph", "found", "shall", "harry", "stove", "lowly", "snout", "trope", "fewer", "shawl", "natal", "fibre", "comma", "foray", "scare", "stair", "black", "squad", "royal", "chunk", "mince", "slave", "shame", "cheek", "ample", "flair", "foyer", "cargo", "oxide", "plant", "olive", "inert", "askew", "heist", "shown", "zesty", "hasty", "trash", "fella", "larva", "forgo", "story", "hairy", "train", "homer", "badge", "midst", "canny", "fetus", "butch", "farce", "slung", "tipsy", "metal", "yield", "delve", "being", "scour", "glass", "gamer", "scrap", "money", "hinge", "album", "vouch", "asset", "tiara", "crept", "bayou", "atoll", "manor", "creak", "showy", "phase", "froth", "depth", "gloom", "flood", "trait", "girth", "piety", "payer", "goose", "float", "donor", "atone", "primo", "apron", "blown", "cacao", "loser", "input", "gloat", "awful", "brink", "smite", "beady", "rusty", "retro", "droll", "gawky", "hutch", "pinto", "gaily", "egret", "lilac", "sever", "field", "fluff", "hydro", "flack", "agape", "wench", "voice", "stead", "stalk", "berth", "madam", "night", "bland", "liver", "wedge", "augur", "roomy", "wacky", "flock", "angry", "bobby", "trite", "aphid", "tryst", "midge", "power", "elope", "cinch", "motto", "stomp", "upset", "bluff", "cramp", "quart", "coyly", "youth", "rhyme", "buggy", "alien", "smear", "unfit", "patty", "cling", "glean", "label", "hunky", "khaki", "poker", "gruel", "twice", "twang", "shrug", "treat", "unlit", "waste", "merit", "woven", "octal", "needy", "clown", "widow", "irony", "ruder", "gauze", "chief", "onset", "prize", "fungi", "charm", "gully", "inter", "whoop", "taunt", "leery", "class", "theme", "lofty", "tibia", "booze", "alpha", "thyme", "eclat", "doubt", "parer", "chute", "stick", "trice", "alike", "sooth", "recap", "saint", "liege", "glory", "grate", "admit", "brisk", "soggy", "usurp", "scald", "scorn", "leave", "twine", "sting", "bough", "marsh", "sloth", "dandy", "vigor", "howdy", "enjoy", "valid", "ionic", "equal", "unset", "floor", "catch", "spade", "stein", "exist", "quirk", "denim", "grove", "spiel", "mummy", "fault", "foggy", "flout", "carry", "sneak", "libel", "waltz", "aptly", "piney", "inept", "aloud", "photo", "dream", "stale", "vomit", "ombre", "fanny", "unite", "snarl", "baker", "there", "glyph", "pooch", "hippy", "spell", "folly", "louse", "gulch", "vault", "godly", "threw", "fleet", "grave", "inane", "shock", "crave", "spite", "valve", "skimp", "claim", "rainy", "musty", "pique", "daddy", "quasi", "arise", "aging", "valet", "opium", "avert", "stuck", "recut", "mulch", "genre", "plume", "rifle", "count", "incur", "total", "wrest", "mocha", "deter", "study", "lover", "safer", "rivet", "funny", "smoke", "mound", "undue", "sedan", "pagan", "swine", "guile", "gusty", "equip", "tough", "canoe", "chaos", "covet", "human", "udder", "lunch", "blast", "stray", "manga", "melee", "lefty", "quick", "paste", "given", "octet", "risen", "groan", "leaky", "grind", "carve", "loose", "sadly", "spilt", "apple", "slack", "honey", "final", "sheen", "eerie", "minty", "slick", "derby", "wharf", "spelt", "coach", "erupt", "singe", "price", "spawn", "fairy", "jiffy", "filmy", "stack", "chose", "sleep", "ardor", "nanny", "niece", "woozy", "handy", "grace", "ditto", "stank", "cream", "usual", "diode", "valor", "angle", "ninja", "muddy", "chase", "reply", "prone", "spoil", "heart", "shade", "diner", "arson", "onion", "sleet", "dowel", "couch", "palsy", "bowel", "smile", "evoke", "creek", "lance", "eagle", "idiot", "siren", "built", "embed", "award", "dross", "annul", "goody", "frown", "patio", "laden", "humid", "elite", "lymph", "edify", "might", "reset", "visit", "gusto", "purse", "vapor", "crock", "write", "sunny", "loath", "chaff", "slide", "queer", "venom", "stamp", "sorry", "still", "acorn", "aping", "pushy", "tamer", "hater", "mania", "awoke", "brawn", "swift", "exile", "birch", "lucky", "freer", "risky", "ghost", "plier", "lunar", "winch", "snare", "nurse", "house", "borax", "nicer", "lurch", "exalt", "about", "savvy", "toxin", "tunic", "pried", "inlay", "chump", "lanky", "cress", "eater", "elude", "cycle", "kitty", "boule", "moron", "tenet", "place", "lobby", "plush", "vigil", "index", "blink", "clung", "qualm", "croup", "clink", "juicy", "stage", "decay", "nerve", "flier", "shaft", "crook", "clean", "china", "ridge", "vowel", "gnome", "snuck", "icing", "spiny", "rigor", "snail", "flown", "rabid", "prose", "thank", "poppy", "budge", "fiber", "moldy", "dowdy", "kneel", "track", "caddy", "quell", "dumpy", "paler", "swore", "rebar", "scuba", "splat", "flyer", "horny", "mason", "doing", "ozone", "amply", "molar", "ovary", "beset", "queue", "cliff", "magic", "truce", "sport", "fritz", "edict", "twirl", "verse", "llama", "eaten", "range", "whisk", "hovel", "rehab", "macaw", "sigma", "spout", "verve", "sushi", "dying", "fetid", "brain", "buddy", "thump", "scion", "candy", "chord", "basin", "march", "crowd", "arbor", "gayly", "musky", "stain", "dally", "bless", "bravo", "stung", "title", "ruler", "kiosk", "blond", "ennui", "layer", "fluid", "tatty", "score", "cutie", "zebra", "barge", "matey", "bluer", "aider", "shook", "river", "privy", "betel", "frisk", "bongo", "begun", "azure", "weave", "genie", "sound", "glove", "braid", "scope", "wryly", "rover", "assay", "ocean", "bloom", "irate", "later", "woken", "silky", "wreck", "dwelt", "slate", "smack", "solid", "amaze", "hazel", "wrist", "jolly", "globe", "flint", "rouse", "civil", "vista", "relax", "cover", "alive", "beech", "jetty", "bliss", "vocal", "often", "dolly", "eight", "joker", "since", "event", "ensue", "shunt", "diver", "poser", "worst", "sweep", "alley", "creed", "anime", "leafy", "bosom", "dunce", "stare", "pudgy", "waive", "choir", "stood", "spoke", "outgo", "delay", "bilge", "ideal", "clasp", "seize", "hotly", "laugh", "sieve", "block", "meant", "grape", "noose", "hardy", "shied", "drawl", "daisy", "putty", "strut", "burnt", "tulip", "crick", "idyll", "vixen", "furor", "geeky", "cough", "naive", "shoal", "stork", "bathe", "aunty", "check", "prime", "brass", "outer", "furry", "razor", "elect", "evict", "imply", "demur", "quota", "haven", "cavil", "swear", "crump", "dough", "gavel", "wagon", "salon", "nudge", "harem", "pitch", "sworn", "pupil", "excel", "stony", "cabin", "unzip", "queen", "trout", "polyp", "earth", "storm", "until", "taper", "enter", "child", "adopt", "minor", "fatty", "husky", "brave", "filet", "slime", "glint", "tread", "steal", "regal", "guest", "every", "murky", "share", "spore", "hoist", "buxom", "inner", "otter", "dimly", "level", "sumac", "donut", "stilt", "arena", "sheet", "scrub", "fancy", "slimy", "pearl", "silly", "porch", "dingo", "sepia", "amble", "shady", "bread", "friar", "reign", "dairy", "quill", "cross", "brood", "tuber", "shear", "posit", "blank", "villa", "shank", "piggy", "freak", "which", "among", "fecal", "shell", "would", "algae", "large", "rabbi", "agony", "amuse", "bushy", "copse", "swoon", "knife", "pouch", "ascot", "plane", "crown", "urban", "snide", "relay", "abide", "viola", "rajah", "straw", "dilly", "crash", "amass", "third", "trick", "tutor", "woody", "blurb", "grief", "disco", "where", "sassy", "beach", "sauna", "comic", "clued", "creep", "caste", "graze", "snuff", "frock", "gonad", "drunk", "prong", "lurid", "steel", "halve", "buyer", "vinyl", "utile", "smell", "adage", "worry", "tasty", "local", "trade", "finch", "ashen", "modal", "gaunt", "clove", "enact", "adorn", "roast", "speck", "sheik", "missy", "grunt", "snoop", "party", "touch", "mafia", "emcee", "array", "south", "vapid", "jelly", "skulk", "angst", "tubal", "lower", "crest", "sweat", "cyber", "adore", "tardy", "swami", "notch", "groom", "roach", "hitch", "young", "align", "ready", "frond", "strap", "puree", "realm", "venue", "swarm", "offer", "seven", "dryer", "diary", "dryly", "drank", "acrid", "heady", "theta", "junto", "pixie", "quoth", "bonus", "shalt", "penne", "amend", "datum", "build", "piano", "shelf", "lodge", "suing", "rearm", "coral", "ramen", "worth", "psalm", "infer", "overt", "mayor", "ovoid", "glide", "usage", "poise", "randy", "chuck", "prank", "fishy", "tooth", "ether", "drove", "idler", "swath", "stint", "while", "begat", "apply", "slang", "tarot", "radar", "credo", "aware", "canon", "shift", "timer", "bylaw", "serum", "three", "steak", "iliac", "shirk", "blunt", "puppy", "penal", "joist", "bunny", "shape", "beget", "wheel", "adept", "stunt", "stole", "topaz", "chore", "fluke", "afoot", "bloat", "bully", "dense", "caper", "sneer", "boxer", "jumbo", "lunge", "space", "avail", "short", "slurp", "loyal", "flirt", "pizza", "conch", "tempo", "droop", "plate", "bible", "plunk", "afoul", "savoy", "steep", "agile", "stake", "dwell", "knave", "beard", "arose", "motif", "smash", "broil", "glare", "shove", "baggy", "mammy", "swamp", "along", "rugby", "wager", "quack", "squat", "snaky", "debit", "mange", "skate", "ninth", "joust", "tramp", "spurn", "medal", "micro", "rebel", "flank", "learn", "nadir", "maple", "comfy", "remit", "gruff", "ester", "least", "mogul", "fetch", "cause", "oaken", "aglow", "meaty", "gaffe", "shyly", "racer", "prowl", "thief", "stern", "poesy", "rocky", "tweet", "waist", "spire", "grope", "havoc", "patsy", "truly", "forty", "deity", "uncle", "swish", "giver", "preen", "bevel", "lemur", "draft", "slope", "annoy", "lingo", "bleak", "ditty", "curly", "cedar", "dirge", "grown", "horde", "drool", "shuck", "crypt", "cumin", "stock", "gravy", "locus", "wider", "breed", "quite", "chafe", "cache", "blimp", "deign", "fiend", "logic", "cheap", "elide", "rigid", "false", "renal", "pence", "rowdy", "shoot", "blaze", "envoy", "posse", "brief", "never", "abort", "mouse", "mucky", "sulky", "fiery", "media", "trunk", "yeast", "clear", "skunk", "scalp", "bitty", "cider", "koala", "duvet", "segue", "creme", "super", "grill", "after", "owner", "ember", "reach", "nobly", "empty", "speed", "gipsy", "recur", "smock", "dread", "merge", "burst", "kappa", "amity", "shaky", "hover", "carol", "snort", "synod", "faint", "haunt", "flour", "chair", "detox", "shrew", "tense", "plied", "quark", "burly", "novel", "waxen", "stoic", "jerky", "blitz", "beefy", "lyric", "hussy", "towel", "quilt", "below", "bingo", "wispy", "brash", "scone", "toast", "easel", "saucy", "value", "spice", "honor", "route", "sharp", "bawdy", "radii", "skull", "phony", "issue", "lager", "swell", "urine", "gassy", "trial", "flora", "upper", "latch", "wight", "brick", "retry", "holly", "decal", "grass", "shack", "dogma", "mover", "defer", "sober", "optic", "crier", "vying", "nomad", "flute", "hippo", "shark", "drier", "obese", "bugle", "tawny", "chalk", "feast", "ruddy", "pedal", "scarf", "cruel", "bleat", "tidal", "slush", "semen", "windy", "dusty", "sally", "igloo", "nerdy", "jewel", "shone", "whale", "hymen", "abuse", "fugue", "elbow", "crumb", "pansy", "welsh", "syrup", "terse", "suave", "gamut", "swung", "drake", "freed", "afire", "shirt", "grout", "oddly", "tithe", "plaid", "dummy", "broom", "blind", "torch", "enemy", "again", "tying", "pesky", "alter", "gazer", "noble", "ethos", "bride", "extol", "decor", "hobby", "beast", "idiom", "utter", "these", "sixth", "alarm", "erase", "elegy", "spunk", "piper", "scaly", "scold", "hefty", "chick", "sooty", "canal", "whiny", "slash", "quake", "joint", "swept", "prude", "heavy", "wield", "femme", "lasso", "maize", "shale", "screw", "spree", "smoky", "whiff", "scent", "glade", "spent", "prism", "stoke", "riper", "orbit", "cocoa", "guilt", "humus", "shush", "table", "smirk", "wrong", "noisy", "alert", "shiny", "elate", "resin", "whole", "hunch", "pixel", "polar", "hotel", "sword", "cleat", "mango", "rumba", "puffy", "filly", "billy", "leash", "clout", "dance", "ovate", "facet", "chili", "paint", "liner", "curio", "salty", "audio", "snake", "fable", "cloak", "navel", "spurt", "pesto", "balmy", "flash", "unwed", "early", "churn", "weedy", "stump", "lease", "witty", "wimpy", "spoof", "saner", "blend", "salsa", "thick", "warty", "manic", "blare", "squib", "spoon", "probe", "crepe", "knack", "force", "debut", "order", "haste", "teeth", "agent", "widen", "icily", "slice", "ingot", "clash", "juror", "blood", "abode", "throw", "unity", "pivot", "slept", "troop", "spare", "sewer", "parse", "morph", "cacti", "tacky", "spool", "demon", "moody", "annex", "begin", "fuzzy", "patch", "water", "lumpy", "admin", "omega", "limit", "tabby", "macho", "aisle", "skiff", "basis", "plank", "verge", "botch", "crawl", "lousy", "slain", "cubic", "raise", "wrack", "guide", "foist", "cameo", "under", "actor", "revue", "fraud", "harpy", "scoop", "climb", "refer", "olden", "clerk", "debar", "tally", "ethic", "cairn", "tulle", "ghoul", "hilly", "crude", "apart", "scale", "older", "plain", "sperm", "briny", "abbot", "rerun", "quest", "crisp", "bound", "befit", "drawn", "suite", "itchy", "cheer", "bagel", "guess", "broad", "axiom", "chard", "caput", "leant", "harsh", "curse", "proud", "swing", "opine", "taste", "lupus", "gumbo", "miner", "green", "chasm", "lipid", "topic", "armor", "brush", "crane", "mural", "abled", "habit", "bossy", "maker", "dusky", "dizzy", "lithe", "brook", "jazzy", "fifty", "sense", "giant", "surly", "legal", "fatal", "flunk", "began", "prune", "small", "slant", "scoff", "torus", "ninny", "covey", "viper", "taken", "moral", "vogue", "owing", "token", "entry", "booth", "voter", "chide", "elfin", "ebony", "neigh", "minim", "melon", "kneed", "decoy", "voila", "ankle", "arrow", "mushy", "tribe", "cease", "eager", "birth", "graph", "odder", "terra", "weird", "tried", "clack", "color", "rough", "weigh", "uncut", "ladle", "strip", "craft", "minus", "dicey", "titan", "lucid", "vicar", "dress", "ditch", "gypsy", "pasta", "taffy", "flame", "swoop", "aloof", "sight", "broke", "teary", "chart", "sixty", "wordy", "sheer", "leper", "nosey", "bulge", "savor", "clamp", "funky", "foamy", "toxic", "brand", "plumb", "dingy", "butte", "drill", "tripe", "bicep", "tenor", "krill", "worse", "drama", "hyena", "think", "ratio", "cobra", "basil", "scrum", "bused", "phone", "court", "camel", "proof", "heard", "angel", "petal", "pouty", "throb", "maybe", "fetal", "sprig", "spine", "shout", "cadet", "macro", "dodgy", "satyr", "rarer", "binge", "trend", "nutty", "leapt", "amiss", "split", "myrrh", "width", "sonar", "tower", "baron", "fever", "waver", "spark", "belie", "sloop", "expel", "smote", "baler", "above", "north", "wafer", "scant", "frill", "awash", "snack", "scowl", "frail", "drift", "limbo", "fence", "motel", "ounce", "wreak", "revel", "talon", "prior", "knelt", "cello", "flake", "debug", "anode", "crime", "salve", "scout", "imbue", "pinky", "stave", "vague", "chock", "fight", "video", "stone", "teach", "cleft", "frost", "prawn", "booty", "twist", "apnea", "stiff", "plaza", "ledge", "tweak", "board", "grant", "medic", "bacon", "cable", "brawl", "slunk", "raspy", "forum", "drone", "women", "mucus", "boast", "toddy", "coven", "tumor", "truer", "wrath", "stall", "steam", "axial", "purer", "daily", "trail", "niche", "mealy", "juice", "nylon", "plump", "merry", "flail", "papal", "wheat", "berry", "cower", "erect", "brute", "leggy", "snipe", "sinew", "skier", "penny", "jumpy", "rally", "umbra", "scary", "modem", "gross", "avian", "greed", "satin", "tonic", "parka", "sniff", "livid", "stark", "trump", "giddy", "reuse", "taboo", "avoid", "quote", "devil", "liken", "gloss", "gayer", "beret", "noise", "gland", "dealt", "sling", "rumor", "opera", "thigh", "tonga", "flare", "wound", "white", "bulky", "etude", "horse", "circa", "paddy", "inbox", "fizzy", "grain", "exert", "surge", "gleam", "belle", "salvo", "crush", "fruit", "sappy", "taker", "tract", "ovine", "spiky", "frank", "reedy", "filth", "spasm", "heave", "mambo", "right", "clank", "trust", "lumen", "borne", "spook", "sauce", "amber", "lathe", "carat", "corer", "dirty", "slyly", "affix", "alloy", "taint", "sheep", "kinky", "wooly", "mauve", "flung", "yacht", "fried", "quail", "brunt", "grimy", "curvy", "cagey", "rinse", "deuce", "state", "grasp", "milky", "bison", "graft", "sandy", "baste", "flask", "hedge", "girly", "swash", "boney", "coupe", "endow", "abhor", "welch", "blade", "tight", "geese", "miser", "mirth", "cloud", "cabal", "leech", "close", "tenth", "pecan", "droit", "grail", "clone", "guise", "ralph", "tango", "biddy", "smith", "mower", "payee", "serif", "drape", "fifth", "spank", "glaze", "allot", "truck", "kayak", "virus", "testy", "tepee", "fully", "zonal", "metro", "curry", "grand", "banjo", "axion", "bezel", "occur", "chain", "nasal", "gooey", "filer", "brace", "allay", "pubic", "raven", "plead", "gnash", "flaky", "munch", "dully", "eking", "thing", "slink", "hurry", "theft", "shorn", "pygmy", "ranch", "wring", "lemon", "shore", "mamma", "froze", "newer", "style", "moose", "antic", "drown", "vegan", "chess", "guppy", "union", "lever", "lorry", "image", "cabby", "druid", "exact", "truth", "dopey", "spear", "cried", "chime", "crony", "stunk", "timid", "batch", "gauge", "rotor", "crack", "curve", "latte", "witch", "bunch", "repel", "anvil", "soapy", "meter", "broth", "madly", "dried", "scene", "known", "magma", "roost", "woman", "thong", "punch", "pasty", "downy", "knead", "whirl", "rapid", "clang", "anger", "drive", "goofy", "email", "music", "stuff", "bleep", "rider", "mecca", "folio", "setup", "verso", "quash", "fauna", "gummy", "happy", "newly", "fussy", "relic", "guava", "ratty", "fudge", "femur", "chirp", "forte", "alibi", "whine", "petty", "golly", "plait", "fleck", "felon", "gourd", "brown", "thrum", "ficus", "stash", "decry", "wiser", "junta", "visor", "daunt", "scree", "impel", "await", "press", "whose", "turbo", "stoop", "speak", "mangy", "eying", "inlet", "crone", "pulse", "mossy", "staid", "hence", "pinch", "teddy", "sully", "snore", "ripen", "snowy", "attic", "going", "leach", "mouth", "hound", "clump", "tonal", "bigot", "peril", "piece", "blame", "haute", "spied", "undid", "intro", "basal", "shine", "gecko", "rodeo", "guard", "steer", "loamy", "scamp", "scram", "manly", "hello", "vaunt", "organ", "feral", "knock", "extra", "condo", "adapt", "willy", "polka", "rayon", "skirt", "faith", "torso", "match", "mercy", "tepid", "sleek", "riser", "twixt", "peace", "flush", "catty", "login", "eject", "roger", "rival", "untie", "refit", "aorta", "adult", "judge", "rower", "artsy", "rural", "shave"])
allowed_list = ["aahed", "aalii", "aargh", "aarti", "abaca", "abaci", "abacs", "abaft", "abaka", "abamp", "aband", "abash", "abask", "abaya", "abbas", "abbed", "abbes", "abcee", "abeam", "abear", "abele", "abers", "abets", "abies", "abler", "ables", "ablet", "ablow", "abmho", "abohm", "aboil", "aboma", "aboon", "abord", "abore", "abram", "abray", "abrim", "abrin", "abris", "absey", "absit", "abuna", "abune", "abuts", "abuzz", "abyes", "abysm", "acais", "acari", "accas", "accoy", "acerb", "acers", "aceta", "achar", "ached", "aches", "achoo", "acids", "acidy", "acing", "acini", "ackee", "acker", "acmes", "acmic", "acned", "acnes", "acock", "acold", "acred", "acres", "acros", "acted", "actin", "acton", "acyls", "adaws", "adays", "adbot", "addax", "added", "adder", "addio", "addle", "adeem", "adhan", "adieu", "adios", "adits", "adman", "admen", "admix", "adobo", "adown", "adoze", "adrad", "adred", "adsum", "aduki", "adunc", "adust", "advew", "adyta", "adzed", "adzes", "aecia", "aedes", "aegis", "aeons", "aerie", "aeros", "aesir", "afald", "afara", "afars", "afear", "aflaj", "afore", "afrit", "afros", "agama", "agami", "agars", "agast", "agave", "agaze", "agene", "agers", "agger", "aggie", "aggri", "aggro", "aggry", "aghas", "agila", "agios", "agism", "agist", "agita", "aglee", "aglet", "agley", "agloo", "aglus", "agmas", "agoge", "agone", "agons", "agood", "agria", "agrin", "agros", "agued", "agues", "aguna", "aguti", "aheap", "ahent", "ahigh", "ahind", "ahing", "ahint", "ahold", "ahull", "ahuru", "aidas", "aided", "aides", "aidoi", "aidos", "aiery", "aigas", "aight", "ailed", "aimed", "aimer", "ainee", "ainga", "aioli", "aired", "airer", "airns", "airth", "airts", "aitch", "aitus", "aiver", "aiyee", "aizle", "ajies", "ajiva", "ajuga", "ajwan", "akees", "akela", "akene", "aking", "akita", "akkas", "alaap", "alack", "alamo", "aland", "alane", "alang", "alans", "alant", "alapa", "alaps", "alary", "alate", "alays", "albas", "albee", "alcid", "alcos", "aldea", "alder", "aldol", "aleck", "alecs", "alefs", "aleft", "aleph", "alews", "aleye", "alfas", "algal", "algas", "algid", "algin", "algor", "algum", "alias", "alifs", "aline", "alist", "aliya", "alkie", "alkos", "alkyd", "alkyl", "allee", "allel", "allis", "allod", "allyl", "almah", "almas", "almeh", "almes", "almud", "almug", "alods", "aloed", "aloes", "aloha", "aloin", "aloos", "alowe", "altho", "altos", "alula", "alums", "alure", "alvar", "alway", "amahs", "amain", "amate", "amaut", "amban", "ambit", "ambos", "ambry", "ameba", "ameer", "amene", "amens", "ament", "amias", "amice", "amici", "amide", "amido", "amids", "amies", "amiga", "amigo", "amine", "amino", "amins", "amirs", "amlas", "amman", "ammon", "ammos", "amnia", "amnic", "amnio", "amoks", "amole", "amort", "amour", "amove", "amowt", "amped", "ampul", "amrit", "amuck", "amyls", "anana", "anata", "ancho", "ancle", "ancon", "andro", "anear", "anele", "anent", "angas", "anglo", "anigh", "anile", "anils", "anima", "animi", "anion", "anise", "anker", "ankhs", "ankus", "anlas", "annal", "annas", "annat", "anoas", "anole", "anomy", "ansae", "antae", "antar", "antas", "anted", "antes", "antis", "antra", "antre", "antsy", "anura", "anyon", "apace", "apage", "apaid", "apayd", "apays", "apeak", "apeek", "apers", "apert", "apery", "apgar", "aphis", "apian", "apiol", "apish", "apism", "apode", "apods", "apoop", "aport", "appal", "appay", "appel", "appro", "appui", "appuy", "apres", "apses", "apsis", "apsos", "apted", "apter", "aquae", "aquas", "araba", "araks", "arame", "arars", "arbas", "arced", "archi", "arcos", "arcus", "ardeb", "ardri", "aread", "areae", "areal", "arear", "areas", "areca", "aredd", "arede", "arefy", "areic", "arene", "arepa", "arere", "arete", "arets", "arett", "argal", "argan", "argil", "argle", "argol", "argon", "argot", "argus", "arhat", "arias", "ariel", "ariki", "arils", "ariot", "arish", "arked", "arled", "arles", "armed", "armer", "armet", "armil", "arnas", "arnut", "aroba", "aroha", "aroid", "arpas", "arpen", "arrah", "arras", "arret", "arris", "arroz", "arsed", "arses", "arsey", "arsis", "artal", "artel", "artic", "artis", "aruhe", "arums", "arval", "arvee", "arvos", "aryls", "asana", "ascon", "ascus", "asdic", "ashed", "ashes", "ashet", "asked", "asker", "askoi", "askos", "aspen", "asper", "aspic", "aspie", "aspis", "aspro", "assai", "assam", "asses", "assez", "assot", "aster", "astir", "astun", "asura", "asway", "aswim", "asyla", "ataps", "ataxy", "atigi", "atilt", "atimy", "atlas", "atman", "atmas", "atmos", "atocs", "atoke", "atoks", "atoms", "atomy", "atony", "atopy", "atria", "atrip", "attap", "attar", "atuas", "audad", "auger", "aught", "aulas", "aulic", "auloi", "aulos", "aumil", "aunes", "aunts", "aurae", "aural", "aurar", "auras", "aurei", "aures", "auric", "auris", "aurum", "autos", "auxin", "avale", "avant", "avast", "avels", "avens", "avers", "avgas", "avine", "avion", "avise", "aviso", "avize", "avows", "avyze", "awarn", "awato", "awave", "aways", "awdls", "aweel", "aweto", "awing", "awmry", "awned", "awner", "awols", "awork", "axels", "axile", "axils", "axing", "axite", "axled", "axles", "axman", "axmen", "axoid", "axone", "axons", "ayahs", "ayaya", "ayelp", "aygre", "ayins", "ayont", "ayres", "ayrie", "azans", "azide", "azido", "azine", "azlon", "azoic", "azole", "azons", "azote", "azoth", "azuki", "azurn", "azury", "azygy", "azyme", "azyms", "baaed", "baals", "babas", "babel", "babes", "babka", "baboo", "babul", "babus", "bacca", "bacco", "baccy", "bacha", "bachs", "backs", "baddy", "baels", "baffs", "baffy", "bafts", "baghs", "bagie", "bahts", "bahus", "bahut", "bails", "bairn", "baisa", "baith", "baits", "baiza", "baize", "bajan", "bajra", "bajri", "bajus", "baked", "baken", "bakes", "bakra", "balas", "balds", "baldy", "baled", "bales", "balks", "balky", "balls", "bally", "balms", "baloo", "balsa", "balti", "balun", "balus", "bambi", "banak", "banco", "bancs", "banda", "bandh", "bands", "bandy", "baned", "banes", "bangs", "bania", "banks", "banns", "bants", "bantu", "banty", "banya", "bapus", "barbe", "barbs", "barby", "barca", "barde", "bardo", "bards", "bardy", "bared", "barer", "bares", "barfi", "barfs", "baric", "barks", "barky", "barms", "barmy", "barns", "barny", "barps", "barra", "barre", "barro", "barry", "barye", "basan", "based", "basen", "baser", "bases", "basho", "basij", "basks", "bason", "basse", "bassi", "basso", "bassy", "basta", "basti", "basto", "basts", "bated", "bates", "baths", "batik", "batta", "batts", "battu", "bauds", "bauks", "baulk", "baurs", "bavin", "bawds", "bawks", "bawls", "bawns", "bawrs", "bawty", "bayed", "bayer", "bayes", "bayle", "bayts", "bazar", "bazoo", "beads", "beaks", "beaky", "beals", "beams", "beamy", "beano", "beans", "beany", "beare", "bears", "beath", "beats", "beaty", "beaus", "beaut", "beaux", "bebop", "becap", "becke", "becks", "bedad", "bedel", "bedes", "bedew", "bedim", "bedye", "beedi", "beefs", "beeps", "beers", "beery", "beets", "befog", "begad", "begar", "begem", "begot", "begum", "beige", "beigy", "beins", "bekah", "belah", "belar", "belay", "belee", "belga", "bells", "belon", "belts", "bemad", "bemas", "bemix", "bemud", "bends", "bendy", "benes", "benet", "benga", "benis", "benne", "benni", "benny", "bento", "bents", "benty", "bepat", "beray", "beres", "bergs", "berko", "berks", "berme", "berms", "berob", "beryl", "besat", "besaw", "besee", "beses", "besit", "besom", "besot", "besti", "bests", "betas", "beted", "betes", "beths", "betid", "beton", "betta", "betty", "bever", "bevor", "bevue", "bevvy", "bewet", "bewig", "bezes", "bezil", "bezzy", "bhais", "bhaji", "bhang", "bhats", "bhels", "bhoot", "bhuna", "bhuts", "biach", "biali", "bialy", "bibbs", "bibes", "biccy", "bices", "bided", "bider", "bides", "bidet", "bidis", "bidon", "bield", "biers", "biffo", "biffs", "biffy", "bifid", "bigae", "biggs", "biggy", "bigha", "bight", "bigly", "bigos", "bijou", "biked", "biker", "bikes", "bikie", "bilbo", "bilby", "biled", "biles", "bilgy", "bilks", "bills", "bimah", "bimas", "bimbo", "binal", "bindi", "binds", "biner", "bines", "bings", "bingy", "binit", "binks", "bints", "biogs", "biont", "biota", "biped", "bipod", "birds", "birks", "birle", "birls", "biros", "birrs", "birse", "birsy", "bises", "bisks", "bisom", "bitch", "biter", "bites", "bitos", "bitou", "bitsy", "bitte", "bitts", "bivia", "bivvy", "bizes", "bizzo", "bizzy", "blabs", "blads", "blady", "blaer", "blaes", "blaff", "blags", "blahs", "blain", "blams", "blart", "blase", "blash", "blate", "blats", "blatt", "blaud", "blawn", "blaws", "blays", "blear", "blebs", "blech", "blees", "blent", "blert", "blest", "blets", "bleys", "blimy", "bling", "blini", "blins", "bliny", "blips", "blist", "blite", "blits", "blive", "blobs", "blocs", "blogs", "blook", "bloop", "blore", "blots", "blows", "blowy", "blubs", "blude", "bluds", "bludy", "blued", "blues", "bluet", "bluey", "bluid", "blume", "blunk", "blurs", "blype", "boabs", "boaks", "boars", "boart", "boats", "bobac", "bobak", "bobas", "bobol", "bobos", "bocca", "bocce", "bocci", "boche", "bocks", "boded", "bodes", "bodge", "bodhi", "bodle", "boeps", "boets", "boeuf", "boffo", "boffs", "bogan", "bogey", "boggy", "bogie", "bogle", "bogue", "bogus", "bohea", "bohos", "boils", "boing", "boink", "boite", "boked", "bokeh", "bokes", "bokos", "bolar", "bolas", "bolds", "boles", "bolix", "bolls", "bolos", "bolts", "bolus", "bomas", "bombe", "bombo", "bombs", "bonce", "bonds", "boned", "boner", "bones", "bongs", "bonie", "bonks", "bonne", "bonny", "bonza", "bonze", "booai", "booay", "boobs", "boody", "booed", "boofy", "boogy", "boohs", "books", "booky", "bools", "booms", "boomy", "boong", "boons", "boord", "boors", "boose", "boots", "boppy", "borak", "boral", "boras", "borde", "bords", "bored", "boree", "borel", "borer", "bores", "borgo", "boric", "borks", "borms", "borna", "boron", "borts", "borty", "bortz", "bosie", "bosks", "bosky", "boson", "bosun", "botas", "botel", "botes", "bothy", "botte", "botts", "botty", "bouge", "bouks", "boult", "bouns", "bourd", "bourg", "bourn", "bouse", "bousy", "bouts", "bovid", "bowat", "bowed", "bower", "bowes", "bowet", "bowie", "bowls", "bowne", "bowrs", "bowse", "boxed", "boxen", "boxes", "boxla", "boxty", "boyar", "boyau", "boyed", "boyfs", "boygs", "boyla", "boyos", "boysy", "bozos", "braai", "brach", "brack", "bract", "brads", "braes", "brags", "brail", "braks", "braky", "brame", "brane", "brank", "brans", "brant", "brast", "brats", "brava", "bravi", "braws", "braxy", "brays", "braza", "braze", "bream", "brede", "breds", "breem", "breer", "brees", "breid", "breis", "breme", "brens", "brent", "brere", "brers", "breve", "brews", "breys", "brier", "bries", "brigs", "briki", "briks", "brill", "brims", "brins", "brios", "brise", "briss", "brith", "brits", "britt", "brize", "broch", "brock", "brods", "brogh", "brogs", "brome", "bromo", "bronc", "brond", "brool", "broos", "brose", "brosy", "brows", "brugh", "bruin", "bruit", "brule", "brume", "brung", "brusk", "brust", "bruts", "buats", "buaze", "bubal", "bubas", "bubba", "bubbe", "bubby", "bubus", "buchu", "bucko", "bucks", "bucku", "budas", "budis", "budos", "buffa", "buffe", "buffi", "buffo", "buffs", "buffy", "bufos", "bufty", "buhls", "buhrs", "buiks", "buist", "bukes", "bulbs", "bulgy", "bulks", "bulla", "bulls", "bulse", "bumbo", "bumfs", "bumph", "bumps", "bumpy", "bunas", "bunce", "bunco", "bunde", "bundh", "bunds", "bundt", "bundu", "bundy", "bungs", "bungy", "bunia", "bunje", "bunjy", "bunko", "bunks", "bunns", "bunts", "bunty", "bunya", "buoys", "buppy", "buran", "buras", "burbs", "burds", "buret", "burfi", "burgh", "burgs", "burin", "burka", "burke", "burks", "burls", "burns", "buroo", "burps", "burqa", "burro", "burrs", "burry", "bursa", "burse", "busby", "buses", "busks", "busky", "bussu", "busti", "busts", "busty", "buteo", "butes", "butle", "butoh", "butts", "butty", "butut", "butyl", "buzzy", "bwana", "bwazi", "byded", "bydes", "byked", "bykes", "byres", "byrls", "byssi", "bytes", "byway", "caaed", "cabas", "caber", "cabob", "caboc", "cabre", "cacas", "cacks", "cacky", "cadee", "cades", "cadge", "cadgy", "cadie", "cadis", "cadre", "caeca", "caese", "cafes", "caffs", "caged", "cager", "cages", "cagot", "cahow", "caids", "cains", "caird", "cajon", "cajun", "caked", "cakes", "cakey", "calfs", "calid", "calif", "calix", "calks", "calla", "calls", "calms", "calmy", "calos", "calpa", "calps", "calve", "calyx", "caman", "camas", "cames", "camis", "camos", "campi", "campo", "camps", "campy", "camus", "caned", "caneh", "caner", "canes", "cangs", "canid", "canna", "canns", "canso", "canst", "canto", "cants", "canty", "capas", "caped", "capes", "capex", "caphs", "capiz", "caple", "capon", "capos", "capot", "capri", "capul", "carap", "carbo", "carbs", "carby", "cardi", "cards", "cardy", "cared", "carer", "cares", "caret", "carex", "carks", "carle", "carls", "carns", "carny", "carob", "carom", "caron", "carpi", "carps", "carrs", "carse", "carta", "carte", "carts", "carvy", "casas", "casco", "cased", "cases", "casks", "casky", "casts", "casus", "cates", "cauda", "cauks", "cauld", "cauls", "caums", "caups", "cauri", "causa", "cavas", "caved", "cavel", "caver", "caves", "cavie", "cawed", "cawks", "caxon", "ceaze", "cebid", "cecal", "cecum", "ceded", "ceder", "cedes", "cedis", "ceiba", "ceili", "ceils", "celeb", "cella", "celli", "cells", "celom", "celts", "cense", "cento", "cents", "centu", "ceorl", "cepes", "cerci", "cered", "ceres", "cerge", "ceria", "ceric", "cerne", "ceroc", "ceros", "certs", "certy", "cesse", "cesta", "cesti", "cetes", "cetyl", "cezve", "chace", "chack", "chaco", "chado", "chads", "chaft", "chais", "chals", "chams", "chana", "chang", "chank", "chape", "chaps", "chapt", "chara", "chare", "chark", "charr", "chars", "chary", "chats", "chave", "chavs", "chawk", "chaws", "chaya", "chays", "cheep", "chefs", "cheka", "chela", "chelp", "chemo", "chems", "chere", "chert", "cheth", "chevy", "chews", "chewy", "chiao", "chias", "chibs", "chica", "chich", "chico", "chics", "chiel", "chiks", "chile", "chimb", "chimo", "chimp", "chine", "ching", "chink", "chino", "chins", "chips", "chirk", "chirl", "chirm", "chiro", "chirr", "chirt", "chiru", "chits", "chive", "chivs", "chivy", "chizz", "choco", "chocs", "chode", "chogs", "choil", "choko", "choky", "chola", "choli", "cholo", "chomp", "chons", "choof", "chook", "choom", "choon", "chops", "chota", "chott", "chout", "choux", "chowk", "chows", "chubs", "chufa", "chuff", "chugs", "chums", "churl", "churr", "chuse", "chuts", "chyle", "chyme", "chynd", "cibol", "cided", "cides", "ciels", "ciggy", "cilia", "cills", "cimar", "cimex", "cinct", "cines", "cinqs", "cions", "cippi", "circs", "cires", "cirls", "cirri", "cisco", "cissy", "cists", "cital", "cited", "citer", "cites", "cives", "civet", "civie", "civvy", "clach", "clade", "clads", "claes", "clags", "clame", "clams", "clans", "claps", "clapt", "claro", "clart", "clary", "clast", "clats", "claut", "clave", "clavi", "claws", "clays", "cleck", "cleek", "cleep", "clefs", "clegs", "cleik", "clems", "clepe", "clept", "cleve", "clews", "clied", "clies", "clift", "clime", "cline", "clint", "clipe", "clips", "clipt", "clits", "cloam", "clods", "cloff", "clogs", "cloke", "clomb", "clomp", "clonk", "clons", "cloop", "cloot", "clops", "clote", "clots", "clour", "clous", "clows", "cloye", "cloys", "cloze", "clubs", "clues", "cluey", "clunk", "clype", "cnida", "coact", "coady", "coala", "coals", "coaly", "coapt", "coarb", "coate", "coati", "coats", "cobbs", "cobby", "cobia", "coble", "cobza", "cocas", "cocci", "cocco", "cocks", "cocky", "cocos", "codas", "codec", "coded", "coden", "coder", "codes", "codex", "codon", "coeds", "coffs", "cogie", "cogon", "cogue", "cohab", "cohen", "cohoe", "cohog", "cohos", "coifs", "coign", "coils", "coins", "coirs", "coits", "coked", "cokes", "colas", "colby", "colds", "coled", "coles", "coley", "colic", "colin", "colls", "colly", "colog", "colts", "colza", "comae", "comal", "comas", "combe", "combi", "combo", "combs", "comby", "comer", "comes", "comix", "commo", "comms", "commy", "compo", "comps", "compt", "comte", "comus", "coned", "cones", "coney", "confs", "conga", "conge", "congo", "conia", "conin", "conks", "conky", "conne", "conns", "conte", "conto", "conus", "convo", "cooch", "cooed", "cooee", "cooer", "cooey", "coofs", "cooks", "cooky", "cools", "cooly", "coomb", "cooms", "coomy", "coons", "coops", "coopt", "coost", "coots", "cooze", "copal", "copay", "coped", "copen", "coper", "copes", "coppy", "copra", "copsy", "coqui", "coram", "corbe", "corby", "cords", "cored", "cores", "corey", "corgi", "coria", "corks", "corky", "corms", "corni", "corno", "corns", "cornu", "corps", "corse", "corso", "cosec", "cosed", "coses", "coset", "cosey", "cosie", "costa", "coste", "costs", "cotan", "coted", "cotes", "coths", "cotta", "cotts", "coude", "coups", "courb", "courd", "coure", "cours", "couta", "couth", "coved", "coves", "covin", "cowal", "cowan", "cowed", "cowks", "cowls", "cowps", "cowry", "coxae", "coxal", "coxed", "coxes", "coxib", "coyau", "coyed", "coyer", "coypu", "cozed", "cozen", "cozes", "cozey", "cozie", "craal", "crabs", "crags", "craic", "craig", "crake", "crame", "crams", "crans", "crape", "craps", "crapy", "crare", "craws", "crays", "creds", "creel", "crees", "crems", "crena", "creps", "crepy", "crewe", "crews", "crias", "cribs", "cries", "crims", "crine", "crios", "cripe", "crips", "crise", "crith", "crits", "croci", "crocs", "croft", "crogs", "cromb", "crome", "cronk", "crons", "crool", "croon", "crops", "crore", "crost", "crout", "crows", "croze", "cruck", "crudo", "cruds", "crudy", "crues", "cruet", "cruft", "crunk", "cruor", "crura", "cruse", "crusy", "cruve", "crwth", "cryer", "ctene", "cubby", "cubeb", "cubed", "cuber", "cubes", "cubit", "cuddy", "cuffo", "cuffs", "cuifs", "cuing", "cuish", "cuits", "cukes", "culch", "culet", "culex", "culls", "cully", "culms", "culpa", "culti", "cults", "culty", "cumec", "cundy", "cunei", "cunit", "cunts", "cupel", "cupid", "cuppa", "cuppy", "curat", "curbs", "curch", "curds", "curdy", "cured", "curer", "cures", "curet", "curfs", "curia", "curie", "curli", "curls", "curns", "curny", "currs", "cursi", "curst", "cusec", "cushy", "cusks", "cusps", "cuspy", "cusso", "cusum", "cutch", "cuter", "cutes", "cutey", "cutin", "cutis", "cutto", "cutty", "cutup", "cuvee", "cuzes", "cwtch", "cyano", "cyans", "cycad", "cycas", "cyclo", "cyder", "cylix", "cymae", "cymar", "cymas", "cymes", "cymol", "cysts", "cytes", "cyton", "czars", "daals", "dabba", "daces", "dacha", "dacks", "dadah", "dadas", "dados", "daffs", "daffy", "dagga", "daggy", "dagos", "dahls", "daiko", "daine", "daint", "daker", "daled", "dales", "dalis", "dalle", "dalts", "daman", "damar", "dames", "damme", "damns", "damps", "dampy", "dancy", "dangs", "danio", "danks", "danny", "dants", "daraf", "darbs", "darcy", "dared", "darer", "dares", "darga", "dargs", "daric", "daris", "darks", "darky", "darns", "darre", "darts", "darzi", "dashi", "dashy", "datal", "dated", "dater", "dates", "datos", "datto", "daube", "daubs", "dauby", "dauds", "dault", "daurs", "dauts", "daven", "davit", "dawah", "dawds", "dawed", "dawen", "dawks", "dawns", "dawts", "dayan", "daych", "daynt", "dazed", "dazer", "dazes", "deads", "deair", "deals", "deans", "deare", "dearn", "dears", "deary", "deash", "deave", "deaws", "deawy", "debag", "debby", "debel", "debes", "debts", "debud", "debur", "debus", "debye", "decad", "decaf", "decan", "decko", "decks", "decos", "dedal", "deeds", "deedy", "deely", "deems", "deens", "deeps", "deere", "deers", "deets", "deeve", "deevs", "defat", "deffo", "defis", "defog", "degas", "degum", "degus", "deice", "deids", "deify", "deils", "deism", "deist", "deked", "dekes", "dekko", "deled", "deles", "delfs", "delft", "delis", "dells", "delly", "delos", "delph", "delts", "deman", "demes", "demic", "demit", "demob", "demoi", "demos", "dempt", "denar", "denay", "dench", "denes", "denet", "denis", "dents", "deoxy", "derat", "deray", "dered", "deres", "derig", "derma", "derms", "derns", "derny", "deros", "derro", "derry", "derth", "dervs", "desex", "deshi", "desis", "desks", "desse", "devas", "devel", "devis", "devon", "devos", "devot", "dewan", "dewar", "dewax", "dewed", "dexes", "dexie", "dhaba", "dhaks", "dhals", "dhikr", "dhobi", "dhole", "dholl", "dhols", "dhoti", "dhows", "dhuti", "diact", "dials", "diane", "diazo", "dibbs", "diced", "dicer", "dices", "dicht", "dicks", "dicky", "dicot", "dicta", "dicts", "dicty", "diddy", "didie", "didos", "didst", "diebs", "diels", "diene", "diets", "diffs", "dight", "dikas", "diked", "diker", "dikes", "dikey", "dildo", "dilli", "dills", "dimbo", "dimer", "dimes", "dimps", "dinar", "dined", "dines", "dinge", "dings", "dinic", "dinks", "dinky", "dinna", "dinos", "dints", "diols", "diota", "dippy", "dipso", "diram", "direr", "dirke", "dirks", "dirls", "dirts", "disas", "disci", "discs", "dishy", "disks", "disme", "dital", "ditas", "dited", "dites", "ditsy", "ditts", "ditzy", "divan", "divas", "dived", "dives", "divis", "divna", "divos", "divot", "divvy", "diwan", "dixie", "dixit", "diyas", "dizen", "djinn", "djins", "doabs", "doats", "dobby", "dobes", "dobie", "dobla", "dobra", "dobro", "docht", "docks", "docos", "docus", "doddy", "dodos", "doeks", "doers", "doest", "doeth", "doffs", "dogan", "doges", "dogey", "doggo", "doggy", "dogie", "dohyo", "doilt", "doily", "doits", "dojos", "dolce", "dolci", "doled", "doles", "dolia", "dolls", "dolma", "dolor", "dolos", "dolts", "domal", "domed", "domes", "domic", "donah", "donas", "donee", "doner", "donga", "dongs", "donko", "donna", "donne", "donny", "donsy", "doobs", "dooce", "doody", "dooks", "doole", "dools", "dooly", "dooms", "doomy", "doona", "doorn", "doors", "doozy", "dopas", "doped", "doper", "dopes", "dorad", "dorba", "dorbs", "doree", "dores", "doric", "doris", "dorks", "dorky", "dorms", "dormy", "dorps", "dorrs", "dorsa", "dorse", "dorts", "dorty", "dosai", "dosas", "dosed", "doseh", "doser", "doses", "dosha", "dotal", "doted", "doter", "dotes", "dotty", "douar", "douce", "doucs", "douks", "doula", "douma", "doums", "doups", "doura", "douse", "douts", "doved", "doven", "dover", "doves", "dovie", "dowar", "dowds", "dowed", "dower", "dowie", "dowle", "dowls", "dowly", "downa", "downs", "dowps", "dowse", "dowts", "doxed", "doxes", "doxie", "doyen", "doyly", "dozed", "dozer", "dozes", "drabs", "drack", "draco", "draff", "drags", "drail", "drams", "drant", "draps", "drats", "drave", "draws", "drays", "drear", "dreck", "dreed", "dreer", "drees", "dregs", "dreks", "drent", "drere", "drest", "dreys", "dribs", "drice", "dries", "drily", "drips", "dript", "droid", "droil", "droke", "drole", "drome", "drony", "droob", "droog", "drook", "drops", "dropt", "drouk", "drows", "drubs", "drugs", "drums", "drupe", "druse", "drusy", "druxy", "dryad", "dryas", "dsobo", "dsomo", "duads", "duals", "duans", "duars", "dubbo", "ducal", "ducat", "duces", "ducks", "ducky", "ducts", "duddy", "duded", "dudes", "duels", "duets", "duett", "duffs", "dufus", "duing", "duits", "dukas", "duked", "dukes", "dukka", "dulce", "dules", "dulia", "dulls", "dulse", "dumas", "dumbo", "dumbs", "dumka", "dumky", "dumps", "dunam", "dunch", "dunes", "dungs", "dungy", "dunks", "dunno", "dunny", "dunsh", "dunts", "duomi", "duomo", "duped", "duper", "dupes", "duple", "duply", "duppy", "dural", "duras", "dured", "dures", "durgy", "durns", "duroc", "duros", "duroy", "durra", "durrs", "durry", "durst", "durum", "durzi", "dusks", "dusts", "duxes", "dwaal", "dwale", "dwalm", "dwams", "dwang", "dwaum", "dweeb", "dwile", "dwine", "dyads", "dyers", "dyked", "dykes", "dykey", "dykon", "dynel", "dynes", "dzhos", "eagre", "ealed", "eales", "eaned", "eards", "eared", "earls", "earns", "earnt", "earst", "eased", "easer", "eases", "easle", "easts", "eathe", "eaved", "eaves", "ebbed", "ebbet", "ebons", "ebook", "ecads", "eched", "eches", "echos", "ecrus", "edema", "edged", "edger", "edges", "edile", "edits", "educe", "educt", "eejit", "eensy", "eeven", "eevns", "effed", "egads", "egers", "egest", "eggar", "egged", "egger", "egmas", "ehing", "eider", "eidos", "eigne", "eiked", "eikon", "eilds", "eisel", "ejido", "ekkas", "elain", "eland", "elans", "elchi", "eldin", "elemi", "elfed", "eliad", "elint", "elmen", "eloge", "elogy", "eloin", "elops", "elpee", "elsin", "elute", "elvan", "elven", "elver", "elves", "emacs", "embar", "embay", "embog", "embow", "embox", "embus", "emeer", "emend", "emerg", "emery", "emeus", "emics", "emirs", "emits", "emmas", "emmer", "emmet", "emmew", "emmys", "emoji", "emong", "emote", "emove", "empts", "emule", "emure", "emyde", "emyds", "enarm", "enate", "ended", "ender", "endew", "endue", "enews", "enfix", "eniac", "enlit", "enmew", "ennog", "enoki", "enols", "enorm", "enows", "enrol", "ensew", "ensky", "entia", "enure", "enurn", "envoi", "enzym", "eorls", "eosin", "epact", "epees", "ephah", "ephas", "ephod", "ephor", "epics", "epode", "epopt", "epris", "eques", "equid", "erbia", "erevs", "ergon", "ergos", "ergot", "erhus", "erica", "erick", "erics", "ering", "erned", "ernes", "erose", "erred", "erses", "eruct", "erugo", "eruvs", "erven", "ervil", "escar", "escot", "esile", "eskar", "esker", "esnes", "esses", "estoc", "estop", "estro", "etage", "etape", "etats", "etens", "ethal", "ethne", "ethyl", "etics", "etnas", "ettin", "ettle", "etuis", "etwee", "etyma", "eughs", "euked", "eupad", "euros", "eusol", "evens", "evert", "evets", "evhoe", "evils", "evite", "evohe", "ewers", "ewest", "ewhow", "ewked", "exams", "exeat", "execs", "exeem", "exeme", "exfil", "exies", "exine", "exing", "exits", "exode", "exome", "exons", "expat", "expos", "exude", "exuls", "exurb", "eyass", "eyers", "eyots", "eyras", "eyres", "eyrie", "eyrir", "ezine", "fabby", "faced", "facer", "faces", "facia", "facta", "facts", "faddy", "faded", "fader", "fades", "fadge", "fados", "faena", "faery", "faffs", "faffy", "faggy", "fagin", "fagot", "faiks", "fails", "faine", "fains", "fairs", "faked", "faker", "fakes", "fakey", "fakie", "fakir", "falaj", "falls", "famed", "fames", "fanal", "fands", "fanes", "fanga", "fango", "fangs", "fanks", "fanon", "fanos", "fanum", "faqir", "farad", "farci", "farcy", "fards", "fared", "farer", "fares", "farle", "farls", "farms", "faros", "farro", "farse", "farts", "fasci", "fasti", "fasts", "fated", "fates", "fatly", "fatso", "fatwa", "faugh", "fauld", "fauns", "faurd", "fauts", "fauve", "favas", "favel", "faver", "faves", "favus", "fawns", "fawny", "faxed", "faxes", "fayed", "fayer", "fayne", "fayre", "fazed", "fazes", "feals", "feare", "fears", "feart", "fease", "feats", "feaze", "feces", "fecht", "fecit", "fecks", "fedex", "feebs", "feeds", "feels", "feens", "feers", "feese", "feeze", "fehme", "feint", "feist", "felch", "felid", "fells", "felly", "felts", "felty", "femal", "femes", "femmy", "fends", "fendy", "fenis", "fenks", "fenny", "fents", "feods", "feoff", "ferer", "feres", "feria", "ferly", "fermi", "ferms", "ferns", "ferny", "fesse", "festa", "fests", "festy", "fetas", "feted", "fetes", "fetor", "fetta", "fetts", "fetwa", "feuar", "feuds", "feued", "feyed", "feyer", "feyly", "fezes", "fezzy", "fiars", "fiats", "fibro", "fices", "fiche", "fichu", "ficin", "ficos", "fides", "fidge", "fidos", "fiefs", "fient", "fiere", "fiers", "fiest", "fifed", "fifer", "fifes", "fifis", "figgy", "figos", "fiked", "fikes", "filar", "filch", "filed", "files", "filii", "filks", "fille", "fillo", "fills", "filmi", "films", "filos", "filum", "finca", "finds", "fined", "fines", "finis", "finks", "finny", "finos", "fiord", "fiqhs", "fique", "fired", "firer", "fires", "firie", "firks", "firms", "firns", "firry", "firth", "fiscs", "fisks", "fists", "fisty", "fitch", "fitly", "fitna", "fitte", "fitts", "fiver", "fives", "fixed", "fixes", "fixit", "fjeld", "flabs", "flaff", "flags", "flaks", "flamm", "flams", "flamy", "flane", "flans", "flaps", "flary", "flats", "flava", "flawn", "flaws", "flawy", "flaxy", "flays", "fleam", "fleas", "fleek", "fleer", "flees", "flegs", "fleme", "fleur", "flews", "flexi", "flexo", "fleys", "flics", "flied", "flies", "flimp", "flims", "flips", "flirs", "flisk", "flite", "flits", "flitt", "flobs", "flocs", "floes", "flogs", "flong", "flops", "flors", "flory", "flosh", "flota", "flote", "flows", "flubs", "flued", "flues", "fluey", "fluky", "flump", "fluor", "flurr", "fluty", "fluyt", "flyby", "flype", "flyte", "foals", "foams", "foehn", "fogey", "fogie", "fogle", "fogou", "fohns", "foids", "foils", "foins", "folds", "foley", "folia", "folic", "folie", "folks", "folky", "fomes", "fonda", "fonds", "fondu", "fones", "fonly", "fonts", "foods", "foody", "fools", "foots", "footy", "foram", "forbs", "forby", "fordo", "fords", "forel", "fores", "forex", "forks", "forky", "forme", "forms", "forts", "forza", "forze", "fossa", "fosse", "fouat", "fouds", "fouer", "fouet", "foule", "fouls", "fount", "fours", "fouth", "fovea", "fowls", "fowth", "foxed", "foxes", "foxie", "foyle", "foyne", "frabs", "frack", "fract", "frags", "fraim", "franc", "frape", "fraps", "frass", "frate", "frati", "frats", "fraus", "frays", "frees", "freet", "freit", "fremd", "frena", "freon", "frere", "frets", "fribs", "frier", "fries", "frigs", "frise", "frist", "frith", "frits", "fritt", "frize", "frizz", "froes", "frogs", "frons", "frore", "frorn", "frory", "frosh", "frows", "frowy", "frugs", "frump", "frush", "frust", "fryer", "fubar", "fubby", "fubsy", "fucks", "fucus", "fuddy", "fudgy", "fuels", "fuero", "fuffs", "fuffy", "fugal", "fuggy", "fugie", "fugio", "fugle", "fugly", "fugus", "fujis", "fulls", "fumed", "fumer", "fumes", "fumet", "fundi", "funds", "fundy", "fungo", "fungs", "funks", "fural", "furan", "furca", "furls", "furol", "furrs", "furth", "furze", "furzy", "fused", "fusee", "fusel", "fuses", "fusil", "fusks", "fusts", "fusty", "futon", "fuzed", "fuzee", "fuzes", "fuzil", "fyces", "fyked", "fykes", "fyles", "fyrds", "fytte", "gabba", "gabby", "gable", "gaddi", "gades", "gadge", "gadid", "gadis", "gadje", "gadjo", "gadso", "gaffs", "gaged", "gager", "gages", "gaids", "gains", "gairs", "gaita", "gaits", "gaitt", "gajos", "galah", "galas", "galax", "galea", "galed", "gales", "galls", "gally", "galop", "galut", "galvo", "gamas", "gamay", "gamba", "gambe", "gambo", "gambs", "gamed", "games", "gamey", "gamic", "gamin", "gamme", "gammy", "gamps", "ganch", "gandy", "ganef", "ganev", "gangs", "ganja", "ganof", "gants", "gaols", "gaped", "gaper", "gapes", "gapos", "gappy", "garbe", "garbo", "garbs", "garda", "gares", "garis", "garms", "garni", "garre", "garth", "garum", "gases", "gasps", "gaspy", "gasts", "gatch", "gated", "gater", "gates", "gaths", "gator", "gauch", "gaucy", "gauds", "gauje", "gault", "gaums", "gaumy", "gaups", "gaurs", "gauss", "gauzy", "gavot", "gawcy", "gawds", "gawks", "gawps", "gawsy", "gayal", "gazal", "gazar", "gazed", "gazes", "gazon", "gazoo", "geals", "geans", "geare", "gears", "geats", "gebur", "gecks", "geeks", "geeps", "geest", "geist", "geits", "gelds", "gelee", "gelid", "gelly", "gelts", "gemel", "gemma", "gemmy", "gemot", "genal", "genas", "genes", "genet", "genic", "genii", "genip", "genny", "genoa", "genom", "genro", "gents", "genty", "genua", "genus", "geode", "geoid", "gerah", "gerbe", "geres", "gerle", "germs", "germy", "gerne", "gesse", "gesso", "geste", "gests", "getas", "getup", "geums", "geyan", "geyer", "ghast", "ghats", "ghaut", "ghazi", "ghees", "ghest", "ghyll", "gibed", "gibel", "giber", "gibes", "gibli", "gibus", "gifts", "gigas", "gighe", "gigot", "gigue", "gilas", "gilds", "gilet", "gills", "gilly", "gilpy", "gilts", "gimel", "gimme", "gimps", "gimpy", "ginch", "ginge", "gings", "ginks", "ginny", "ginzo", "gipon", "gippo", "gippy", "girds", "girls", "girns", "giron", "giros", "girrs", "girsh", "girts", "gismo", "gisms", "gists", "gitch", "gites", "giust", "gived", "gives", "gizmo", "glace", "glads", "glady", "glaik", "glair", "glams", "glans", "glary", "glaum", "glaur", "glazy", "gleba", "glebe", "gleby", "glede", "gleds", "gleed", "gleek", "glees", "gleet", "gleis", "glens", "glent", "gleys", "glial", "glias", "glibs", "gliff", "glift", "glike", "glime", "glims", "glisk", "glits", "glitz", "gloam", "globi", "globs", "globy", "glode", "glogg", "gloms", "gloop", "glops", "glost", "glout", "glows", "gloze", "glued", "gluer", "glues", "gluey", "glugs", "glume", "glums", "gluon", "glute", "gluts", "gnarl", "gnarr", "gnars", "gnats", "gnawn", "gnaws", "gnows", "goads", "goafs", "goals", "goary", "goats", "goaty", "goban", "gobar", "gobbi", "gobbo", "gobby", "gobis", "gobos", "godet", "godso", "goels", "goers", "goest", "goeth", "goety", "gofer", "goffs", "gogga", "gogos", "goier", "gojis", "golds", "goldy", "goles", "golfs", "golpe", "golps", "gombo", "gomer", "gompa", "gonch", "gonef", "gongs", "gonia", "gonif", "gonks", "gonna", "gonof", "gonys", "gonzo", "gooby", "goods", "goofs", "googs", "gooks", "gooky", "goold", "gools", "gooly", "goons", "goony", "goops", "goopy", "goors", "goory", "goosy", "gopak", "gopik", "goral", "goras", "gored", "gores", "goris", "gorms", "gormy", "gorps", "gorse", "gorsy", "gosht", "gosse", "gotch", "goths", "gothy", "gotta", "gouch", "gouks", "goura", "gouts", "gouty", "gowan", "gowds", "gowfs", "gowks", "gowls", "gowns", "goxes", "goyim", "goyle", "graal", "grabs", "grads", "graff", "graip", "grama", "grame", "gramp", "grams", "grana", "grans", "grapy", "gravs", "grays", "grebe", "grebo", "grece", "greek", "grees", "grege", "grego", "grein", "grens", "grese", "greve", "grews", "greys", "grice", "gride", "grids", "griff", "grift", "grigs", "grike", "grins", "griot", "grips", "gript", "gripy", "grise", "grist", "grisy", "grith", "grits", "grize", "groat", "grody", "grogs", "groks", "groma", "grone", "groof", "grosz", "grots", "grouf", "grovy", "grows", "grrls", "grrrl", "grubs", "grued", "grues", "grufe", "grume", "grump", "grund", "gryce", "gryde", "gryke", "grype", "grypt", "guaco", "guana", "guano", "guans", "guars", "gucks", "gucky", "gudes", "guffs", "gugas", "guids", "guimp", "guiro", "gulag", "gular", "gulas", "gules", "gulet", "gulfs", "gulfy", "gulls", "gulph", "gulps", "gulpy", "gumma", "gummi", "gumps", "gundy", "gunge", "gungy", "gunks", "gunky", "gunny", "guqin", "gurdy", "gurge", "gurls", "gurly", "gurns", "gurry", "gursh", "gurus", "gushy", "gusla", "gusle", "gusli", "gussy", "gusts", "gutsy", "gutta", "gutty", "guyed", "guyle", "guyot", "guyse", "gwine", "gyals", "gyans", "gybed", "gybes", "gyeld", "gymps", "gynae", "gynie", "gynny", "gynos", "gyoza", "gypos", "gyppo", "gyppy", "gyral", "gyred", "gyres", "gyron", "gyros", "gyrus", "gytes", "gyved", "gyves", "haafs", "haars", "hable", "habus", "hacek", "hacks", "hadal", "haded", "hades", "hadji", "hadst", "haems", "haets", "haffs", "hafiz", "hafts", "haggs", "hahas", "haick", "haika", "haiks", "haiku", "hails", "haily", "hains", "haint", "hairs", "haith", "hajes", "hajis", "hajji", "hakam", "hakas", "hakea", "hakes", "hakim", "hakus", "halal", "haled", "haler", "hales", "halfa", "halfs", "halid", "hallo", "halls", "halma", "halms", "halon", "halos", "halse", "halts", "halva", "halwa", "hamal", "hamba", "hamed", "hames", "hammy", "hamza", "hanap", "hance", "hanch", "hands", "hangi", "hangs", "hanks", "hanky", "hansa", "hanse", "hants", "haole", "haoma", "hapax", "haply", "happi", "hapus", "haram", "hards", "hared", "hares", "harim", "harks", "harls", "harms", "harns", "haros", "harps", "harts", "hashy", "hasks", "hasps", "hasta", "hated", "hates", "hatha", "hauds", "haufs", "haugh", "hauld", "haulm", "hauls", "hault", "hauns", "hause", "haver", "haves", "hawed", "hawks", "hawms", "hawse", "hayed", "hayer", "hayey", "hayle", "hazan", "hazed", "hazer", "hazes", "heads", "heald", "heals", "heame", "heaps", "heapy", "heare", "hears", "heast", "heats", "heben", "hebes", "hecht", "hecks", "heder", "hedgy", "heeds", "heedy", "heels", "heeze", "hefte", "hefts", "heids", "heigh", "heils", "heirs", "hejab", "hejra", "heled", "heles", "helio", "hells", "helms", "helos", "helot", "helps", "helve", "hemal", "hemes", "hemic", "hemin", "hemps", "hempy", "hench", "hends", "henge", "henna", "henny", "henry", "hents", "hepar", "herbs", "herby", "herds", "heres", "herls", "herma", "herms", "herns", "heros", "herry", "herse", "hertz", "herye", "hesps", "hests", "hetes", "heths", "heuch", "heugh", "hevea", "hewed", "hewer", "hewgh", "hexad", "hexed", "hexer", "hexes", "hexyl", "heyed", "hiant", "hicks", "hided", "hider", "hides", "hiems", "highs", "hight", "hijab", "hijra", "hiked", "hiker", "hikes", "hikoi", "hilar", "hilch", "hillo", "hills", "hilts", "hilum", "hilus", "himbo", "hinau", "hinds", "hings", "hinky", "hinny", "hints", "hiois", "hiply", "hired", "hiree", "hirer", "hires", "hissy", "hists", "hithe", "hived", "hiver", "hives", "hizen", "hoaed", "hoagy", "hoars", "hoary", "hoast", "hobos", "hocks", "hocus", "hodad", "hodja", "hoers", "hogan", "hogen", "hoggs", "hoghs", "hohed", "hoick", "hoied", "hoiks", "hoing", "hoise", "hokas", "hoked", "hokes", "hokey", "hokis", "hokku", "hokum", "holds", "holed", "holes", "holey", "holks", "holla", "hollo", "holme", "holms", "holon", "holos", "holts", "homas", "homed", "homes", "homey", "homie", "homme", "homos", "honan", "honda", "honds", "honed", "honer", "hones", "hongi", "hongs", "honks", "honky", "hooch", "hoods", "hoody", "hooey", "hoofs", "hooka", "hooks", "hooky", "hooly", "hoons", "hoops", "hoord", "hoors", "hoosh", "hoots", "hooty", "hoove", "hopak", "hoped", "hoper", "hopes", "hoppy", "horah", "horal", "horas", "horis", "horks", "horme", "horns", "horst", "horsy", "hosed", "hosel", "hosen", "hoser", "hoses", "hosey", "hosta", "hosts", "hotch", "hoten", "hotty", "houff", "houfs", "hough", "houri", "hours", "houts", "hovea", "hoved", "hoven", "hoves", "howbe", "howes", "howff", "howfs", "howks", "howls", "howre", "howso", "hoxed", "hoxes", "hoyas", "hoyed", "hoyle", "hubby", "hucks", "hudna", "hudud", "huers", "huffs", "huffy", "huger", "huggy", "huhus", "huias", "hulas", "hules", "hulks", "hulky", "hullo", "hulls", "hully", "humas", "humfs", "humic", "humps", "humpy", "hunks", "hunts", "hurds", "hurls", "hurly", "hurra", "hurst", "hurts", "hushy", "husks", "husos", "hutia", "huzza", "huzzy", "hwyls", "hydra", "hyens", "hygge", "hying", "hykes", "hylas", "hyleg", "hyles", "hylic", "hymns", "hynde", "hyoid", "hyped", "hypes", "hypha", "hyphy", "hypos", "hyrax", "hyson", "hythe", "iambi", "iambs", "ibrik", "icers", "iched", "iches", "ichor", "icier", "icker", "ickle", "icons", "ictal", "ictic", "ictus", "idant", "ideas", "idees", "ident", "idled", "idles", "idola", "idols", "idyls", "iftar", "igapo", "igged", "iglus", "ihram", "ikans", "ikats", "ikons", "ileac", "ileal", "ileum", "ileus", "iliad", "ilial", "ilium", "iller", "illth", "imago", "imams", "imari", "imaum", "imbar", "imbed", "imide", "imido", "imids", "imine", "imino", "immew", "immit", "immix", "imped", "impis", "impot", "impro", "imshi", "imshy", "inapt", "inarm", "inbye", "incel", "incle", "incog", "incus", "incut", "indew", "india", "indie", "indol", "indow", "indri", "indue", "inerm", "infix", "infos", "infra", "ingan", "ingle", "inion", "inked", "inker", "inkle", "inned", "innit", "inorb", "inrun", "inset", "inspo", "intel", "intil", "intis", "intra", "inula", "inure", "inurn", "inust", "invar", "inwit", "iodic", "iodid", "iodin", "iotas", "ippon", "irade", "irids", "iring", "irked", "iroko", "irone", "irons", "isbas", "ishes", "isled", "isles", "isnae", "issei", "istle", "items", "ither", "ivied", "ivies", "ixias", "ixnay", "ixora", "ixtle", "izard", "izars", "izzat", "jaaps", "jabot", "jacal", "jacks", "jacky", "jaded", "jades", "jafas", "jaffa", "jagas", "jager", "jaggs", "jaggy", "jagir", "jagra", "jails", "jaker", "jakes", "jakey", "jalap", "jalop", "jambe", "jambo", "jambs", "jambu", "james", "jammy", "jamon", "janes", "janns", "janny", "janty", "japan", "japed", "japer", "japes", "jarks", "jarls", "jarps", "jarta", "jarul", "jasey", "jaspe", "jasps", "jatos", "jauks", "jaups", "javas", "javel", "jawan", "jawed", "jaxie", "jeans", "jeats", "jebel", "jedis", "jeels", "jeely", "jeeps", "jeers", "jeeze", "jefes", "jeffs", "jehad", "jehus", "jelab", "jello", "jells", "jembe", "jemmy", "jenny", "jeons", "jerid", "jerks", "jerry", "jesse", "jests", "jesus", "jetes", "jeton", "jeune", "jewed", "jewie", "jhala", "jiaos", "jibba", "jibbs", "jibed", "jiber", "jibes", "jiffs", "jiggy", "jigot", "jihad", "jills", "jilts", "jimmy", "jimpy", "jingo", "jinks", "jinne", "jinni", "jinns", "jirds", "jirga", "jirre", "jisms", "jived", "jiver", "jives", "jivey", "jnana", "jobed", "jobes", "jocko", "jocks", "jocky", "jocos", "jodel", "joeys", "johns", "joins", "joked", "jokes", "jokey", "jokol", "joled", "joles", "jolls", "jolts", "jolty", "jomon", "jomos", "jones", "jongs", "jonty", "jooks", "joram", "jorum", "jotas", "jotty", "jotun", "joual", "jougs", "jouks", "joule", "jours", "jowar", "jowed", "jowls", "jowly", "joyed", "jubas", "jubes", "jucos", "judas", "judgy", "judos", "jugal", "jugum", "jujus", "juked", "jukes", "jukus", "julep", "jumar", "jumby", "jumps", "junco", "junks", "junky", "jupes", "jupon", "jural", "jurat", "jurel", "jures", "justs", "jutes", "jutty", "juves", "juvie", "kaama", "kabab", "kabar", "kabob", "kacha", "kacks", "kadai", "kades", "kadis", "kafir", "kagos", "kagus", "kahal", "kaiak", "kaids", "kaies", "kaifs", "kaika", "kaiks", "kails", "kaims", "kaing", "kains", "kakas", "kakis", "kalam", "kales", "kalif", "kalis", "kalpa", "kamas", "kames", "kamik", "kamis", "kamme", "kanae", "kanas", "kandy", "kaneh", "kanes", "kanga", "kangs", "kanji", "kants", "kanzu", "kaons", "kapas", "kaphs", "kapok", "kapow", "kapus", "kaput", "karas", "karat", "karks", "karns", "karoo", "karos", "karri", "karst", "karsy", "karts", "karzy", "kasha", "kasme", "katal", "katas", "katis", "katti", "kaugh", "kauri", "kauru", "kaury", "kaval", "kavas", "kawas", "kawau", "kawed", "kayle", "kayos", "kazis", "kazoo", "kbars", "kebar", "kebob", "kecks", "kedge", "kedgy", "keech", "keefs", "keeks", "keels", "keema", "keeno", "keens", "keeps", "keets", "keeve", "kefir", "kehua", "keirs", "kelep", "kelim", "kells", "kelly", "kelps", "kelpy", "kelts", "kelty", "kembo", "kembs", "kemps", "kempt", "kempy", "kenaf", "kench", "kendo", "kenos", "kente", "kents", "kepis", "kerbs", "kerel", "kerfs", "kerky", "kerma", "kerne", "kerns", "keros", "kerry", "kerve", "kesar", "kests", "ketas", "ketch", "ketes", "ketol", "kevel", "kevil", "kexes", "keyed", "keyer", "khadi", "khafs", "khans", "khaph", "khats", "khaya", "khazi", "kheda", "kheth", "khets", "khoja", "khors", "khoum", "khuds", "kiaat", "kiack", "kiang", "kibbe", "kibbi", "kibei", "kibes", "kibla", "kicks", "kicky", "kiddo", "kiddy", "kidel", "kidge", "kiefs", "kiers", "kieve", "kievs", "kight", "kikes", "kikoi", "kiley", "kilim", "kills", "kilns", "kilos", "kilps", "kilts", "kilty", "kimbo", "kinas", "kinda", "kinds", "kindy", "kines", "kings", "kinin", "kinks", "kinos", "kiore", "kipes", "kippa", "kipps", "kirby", "kirks", "kirns", "kirri", "kisan", "kissy", "kists", "kited", "kiter", "kites", "kithe", "kiths", "kitul", "kivas", "kiwis", "klang", "klaps", "klett", "klick", "klieg", "kliks", "klong", "kloof", "kluge", "klutz", "knags", "knaps", "knarl", "knars", "knaur", "knawe", "knees", "knell", "knish", "knits", "knive", "knobs", "knops", "knosp", "knots", "knout", "knowe", "knows", "knubs", "knurl", "knurr", "knurs", "knuts", "koans", "koaps", "koban", "kobos", "koels", "koffs", "kofta", "kogal", "kohas", "kohen", "kohls", "koine", "kojis", "kokam", "kokas", "koker", "kokra", "kokum", "kolas", "kolos", "kombu", "konbu", "kondo", "konks", "kooks", "kooky", "koori", "kopek", "kophs", "kopje", "koppa", "korai", "koras", "korat", "kores", "korma", "koros", "korun", "korus", "koses", "kotch", "kotos", "kotow", "koura", "kraal", "krabs", "kraft", "krais", "krait", "krang", "krans", "kranz", "kraut", "krays", "kreep", "kreng", "krewe", "krona", "krone", "kroon", "krubi", "krunk", "ksars", "kubie", "kudos", "kudus", "kudzu", "kufis", "kugel", "kuias", "kukri", "kukus", "kulak", "kulan", "kulas", "kulfi", "kumis", "kumys", "kuris", "kurre", "kurta", "kurus", "kusso", "kutas", "kutch", "kutis", "kutus", "kuzus", "kvass", "kvell", "kwela", "kyack", "kyaks", "kyang", "kyars", "kyats", "kybos", "kydst", "kyles", "kylie", "kylin", "kylix", "kyloe", "kynde", "kynds", "kypes", "kyrie", "kytes", "kythe", "laari", "labda", "labia", "labis", "labra", "laced", "lacer", "laces", "lacet", "lacey", "lacks", "laddy", "laded", "lader", "lades", "laers", "laevo", "lagan", "lahal", "lahar", "laich", "laics", "laids", "laigh", "laika", "laiks", "laird", "lairs", "lairy", "laith", "laity", "laked", "laker", "lakes", "lakhs", "lakin", "laksa", "laldy", "lalls", "lamas", "lambs", "lamby", "lamed", "lamer", "lames", "lamia", "lammy", "lamps", "lanai", "lanas", "lanch", "lande", "lands", "lanes", "lanks", "lants", "lapin", "lapis", "lapje", "larch", "lards", "lardy", "laree", "lares", "largo", "laris", "larks", "larky", "larns", "larnt", "larum", "lased", "laser", "lases", "lassi", "lassu", "lassy", "lasts", "latah", "lated", "laten", "latex", "lathi", "laths", "lathy", "latke", "latus", "lauan", "lauch", "lauds", "laufs", "laund", "laura", "laval", "lavas", "laved", "laver", "laves", "lavra", "lavvy", "lawed", "lawer", "lawin", "lawks", "lawns", "lawny", "laxed", "laxer", "laxes", "laxly", "layed", "layin", "layup", "lazar", "lazed", "lazes", "lazos", "lazzi", "lazzo", "leads", "leady", "leafs", "leaks", "leams", "leans", "leany", "leaps", "leare", "lears", "leary", "leats", "leavy", "leaze", "leben", "leccy", "ledes", "ledgy", "ledum", "leear", "leeks", "leeps", "leers", "leese", "leets", "leeze", "lefte", "lefts", "leger", "leges", "legge", "leggo", "legit", "lehrs", "lehua", "leirs", "leish", "leman", "lemed", "lemel", "lemes", "lemma", "lemme", "lends", "lenes", "lengs", "lenis", "lenos", "lense", "lenti", "lento", "leone", "lepid", "lepra", "lepta", "lered", "leres", "lerps", "lesbo", "leses", "lests", "letch", "lethe", "letup", "leuch", "leuco", "leuds", "leugh", "levas", "levee", "leves", "levin", "levis", "lewis", "lexes", "lexis", "lezes", "lezza", "lezzy", "liana", "liane", "liang", "liard", "liars", "liart", "liber", "libra", "libri", "lichi", "licht", "licit", "licks", "lidar", "lidos", "liefs", "liens", "liers", "lieus", "lieve", "lifer", "lifes", "lifts", "ligan", "liger", "ligge", "ligne", "liked", "liker", "likes", "likin", "lills", "lilos", "lilts", "liman", "limas", "limax", "limba", "limbi", "limbs", "limby", "limed", "limen", "limes", "limey", "limma", "limns", "limos", "limpa", "limps", "linac", "linch", "linds", "lindy", "lined", "lines", "liney", "linga", "lings", "lingy", "linin", "links", "linky", "linns", "linny", "linos", "lints", "linty", "linum", "linux", "lions", "lipas", "lipes", "lipin", "lipos", "lippy", "liras", "lirks", "lirot", "lisks", "lisle", "lisps", "lists", "litai", "litas", "lited", "liter", "lites", "litho", "liths", "litre", "lived", "liven", "lives", "livor", "livre", "llano", "loach", "loads", "loafs", "loams", "loans", "loast", "loave", "lobar", "lobed", "lobes", "lobos", "lobus", "loche", "lochs", "locie", "locis", "locks", "locos", "locum", "loden", "lodes", "loess", "lofts", "logan", "loges", "loggy", "logia", "logie", "logoi", "logon", "logos", "lohan", "loids", "loins", "loipe", "loirs", "lokes", "lolls", "lolly", "lolog", "lomas", "lomed", "lomes", "loner", "longa", "longe", "longs", "looby", "looed", "looey", "loofa", "loofs", "looie", "looks", "looky", "looms", "loons", "loony", "loops", "loord", "loots", "loped", "loper", "lopes", "loppy", "loral", "loran", "lords", "lordy", "lorel", "lores", "loric", "loris", "losed", "losel", "losen", "loses", "lossy", "lotah", "lotas", "lotes", "lotic", "lotos", "lotsa", "lotta", "lotte", "lotto", "lotus", "loued", "lough", "louie", "louis", "louma", "lound", "louns", "loupe", "loups", "loure", "lours", "loury", "louts", "lovat", "loved", "loves", "lovey", "lovie", "lowan", "lowed", "lowes", "lownd", "lowne", "lowns", "lowps", "lowry", "lowse", "lowts", "loxed", "loxes", "lozen", "luach", "luaus", "lubed", "lubes", "lubra", "luces", "lucks", "lucre", "ludes", "ludic", "ludos", "luffa", "luffs", "luged", "luger", "luges", "lulls", "lulus", "lumas", "lumbi", "lumme", "lummy", "lumps", "lunas", "lunes", "lunet", "lungi", "lungs", "lunks", "lunts", "lupin", "lured", "lurer", "lures", "lurex", "lurgi", "lurgy", "lurks", "lurry", "lurve", "luser", "lushy", "lusks", "lusts", "lusus", "lutea", "luted", "luter", "lutes", "luvvy", "luxed", "luxer", "luxes", "lweis", "lyams", "lyard", "lyart", "lyase", "lycea", "lycee", "lycra", "lymes", "lynes", "lyres", "lysed", "lyses", "lysin", "lysis", "lysol", "lyssa", "lyted", "lytes", "lythe", "lytic", "lytta", "maaed", "maare", "maars", "mabes", "macas", "maced", "macer", "maces", "mache", "machi", "machs", "macks", "macle", "macon", "madge", "madid", "madre", "maerl", "mafic", "mages", "maggs", "magot", "magus", "mahoe", "mahua", "mahwa", "maids", "maiko", "maiks", "maile", "maill", "mails", "maims", "mains", "maire", "mairs", "maise", "maist", "makar", "makes", "makis", "makos", "malam", "malar", "malas", "malax", "males", "malic", "malik", "malis", "malls", "malms", "malmy", "malts", "malty", "malus", "malva", "malwa", "mamas", "mamba", "mamee", "mamey", "mamie", "manas", "manat", "mandi", "maneb", "maned", "maneh", "manes", "manet", "mangs", "manis", "manky", "manna", "manos", "manse", "manta", "manto", "manty", "manul", "manus", "mapau", "maqui", "marae", "marah", "maras", "marcs", "mardy", "mares", "marge", "margs", "maria", "marid", "marka", "marks", "marle", "marls", "marly", "marms", "maron", "maror", "marra", "marri", "marse", "marts", "marvy", "masas", "mased", "maser", "mases", "mashy", "masks", "massa", "massy", "masts", "masty", "masus", "matai", "mated", "mater", "mates", "maths", "matin", "matlo", "matte", "matts", "matza", "matzo", "mauby", "mauds", "mauls", "maund", "mauri", "mausy", "mauts", "mauzy", "maven", "mavie", "mavin", "mavis", "mawed", "mawks", "mawky", "mawns", "mawrs", "maxed", "maxes", "maxis", "mayan", "mayas", "mayed", "mayos", "mayst", "mazed", "mazer", "mazes", "mazey", "mazut", "mbira", "meads", "meals", "meane", "means", "meany", "meare", "mease", "meath", "meats", "mebos", "mechs", "mecks", "medii", "medle", "meeds", "meers", "meets", "meffs", "meins", "meint", "meiny", "meith", "mekka", "melas", "melba", "melds", "melic", "melik", "mells", "melts", "melty", "memes", "memos", "menad", "mends", "mened", "menes", "menge", "mengs", "mensa", "mense", "mensh", "menta", "mento", "menus", "meous", "meows", "merch", "mercs", "merde", "mered", "merel", "merer", "meres", "meril", "meris", "merks", "merle", "merls", "merse", "mesal", "mesas", "mesel", "meses", "meshy", "mesic", "mesne", "meson", "messy", "mesto", "meted", "metes", "metho", "meths", "metic", "metif", "metis", "metol", "metre", "meuse", "meved", "meves", "mewed", "mewls", "meynt", "mezes", "mezze", "mezzo", "mhorr", "miaou", "miaow", "miasm", "miaul", "micas", "miche", "micht", "micks", "micky", "micos", "micra", "middy", "midgy", "midis", "miens", "mieve", "miffs", "miffy", "mifty", "miggs", "mihas", "mihis", "miked", "mikes", "mikra", "mikva", "milch", "milds", "miler", "miles", "milfs", "milia", "milko", "milks", "mille", "mills", "milor", "milos", "milpa", "milts", "milty", "miltz", "mimed", "mimeo", "mimer", "mimes", "mimsy", "minae", "minar", "minas", "mincy", "minds", "mined", "mines", "minge", "mings", "mingy", "minis", "minke", "minks", "minny", "minos", "mints", "mired", "mires", "mirex", "mirid", "mirin", "mirks", "mirky", "mirly", "miros", "mirvs", "mirza", "misch", "misdo", "mises", "misgo", "misos", "missa", "mists", "misty", "mitch", "miter", "mites", "mitis", "mitre", "mitts", "mixed", "mixen", "mixer", "mixes", "mixte", "mixup", "mizen", "mizzy", "mneme", "moans", "moats", "mobby", "mobes", "mobey", "mobie", "moble", "mochi", "mochs", "mochy", "mocks", "moder", "modes", "modge", "modii", "modus", "moers", "mofos", "moggy", "mohel", "mohos", "mohrs", "mohua", "mohur", "moile", "moils", "moira", "moire", "moits", "mojos", "mokes", "mokis", "mokos", "molal", "molas", "molds", "moled", "moles", "molla", "molls", "molly", "molto", "molts", "molys", "momes", "momma", "mommy", "momus", "monad", "monal", "monas", "monde", "mondo", "moner", "mongo", "mongs", "monic", "monie", "monks", "monos", "monte", "monty", "moobs", "mooch", "moods", "mooed", "mooks", "moola", "mooli", "mools", "mooly", "moong", "moons", "moony", "moops", "moors", "moory", "moots", "moove", "moped", "moper", "mopes", "mopey", "moppy", "mopsy", "mopus", "morae", "moras", "morat", "moray", "morel", "mores", "moria", "morne", "morns", "morra", "morro", "morse", "morts", "mosed", "moses", "mosey", "mosks", "mosso", "moste", "mosts", "moted", "moten", "motes", "motet", "motey", "moths", "mothy", "motis", "motte", "motts", "motty", "motus", "motza", "mouch", "moues", "mould", "mouls", "moups", "moust", "mousy", "moved", "moves", "mowas", "mowed", "mowra", "moxas", "moxie", "moyas", "moyle", "moyls", "mozed", "mozes", "mozos", "mpret", "mucho", "mucic", "mucid", "mucin", "mucks", "mucor", "mucro", "mudge", "mudir", "mudra", "muffs", "mufti", "mugga", "muggs", "muggy", "muhly", "muids", "muils", "muirs", "muist", "mujik", "mulct", "muled", "mules", "muley", "mulga", "mulie", "mulla", "mulls", "mulse", "mulsh", "mumms", "mumps", "mumsy", "mumus", "munga", "munge", "mungo", "mungs", "munis", "munts", "muntu", "muons", "muras", "mured", "mures", "murex", "murid", "murks", "murls", "murly", "murra", "murre", "murri", "murrs", "murry", "murti", "murva", "musar", "musca", "mused", "muser", "muses", "muset", "musha", "musit", "musks", "musos", "musse", "mussy", "musth", "musts", "mutch", "muted", "muter", "mutes", "mutha", "mutis", "muton", "mutts", "muxed", "muxes", "muzak", "muzzy", "mvule", "myall", "mylar", "mynah", "mynas", "myoid", "myoma", "myope", "myops", "myopy", "mysid", "mythi", "myths", "mythy", "myxos", "mzees", "naams", "naans", "nabes", "nabis", "nabks", "nabla", "nabob", "nache", "nacho", "nacre", "nadas", "naeve", "naevi", "naffs", "nagas", "naggy", "nagor", "nahal", "naiad", "naifs", "naiks", "nails", "naira", "nairu", "naked", "naker", "nakfa", "nalas", "naled", "nalla", "named", "namer", "names", "namma", "namus", "nanas", "nance", "nancy", "nandu", "nanna", "nanos", "nanua", "napas", "naped", "napes", "napoo", "nappa", "nappe", "nappy", "naras", "narco", "narcs", "nards", "nares", "naric", "naris", "narks", "narky", "narre", "nashi", "natch", "nates", "natis", "natty", "nauch", "naunt", "navar", "naves", "navew", "navvy", "nawab", "nazes", "nazir", "nazis", "nduja", "neafe", "neals", "neaps", "nears", "neath", "neats", "nebek", "nebel", "necks", "neddy", "needs", "neeld", "neele", "neemb", "neems", "neeps", "neese", "neeze", "negro", "negus", "neifs", "neist", "neive", "nelis", "nelly", "nemas", "nemns", "nempt", "nenes", "neons", "neper", "nepit", "neral", "nerds", "nerka", "nerks", "nerol", "nerts", "nertz", "nervy", "nests", "netes", "netop", "netts", "netty", "neuks", "neume", "neums", "nevel", "neves", "nevus", "newbs", "newed", "newel", "newie", "newsy", "newts", "nexts", "nexus", "ngaio", "ngana", "ngati", "ngoma", "ngwee", "nicad", "nicht", "nicks", "nicol", "nidal", "nided", "nides", "nidor", "nidus", "niefs", "nieve", "nifes", "niffs", "niffy", "nifty", "niger", "nighs", "nihil", "nikab", "nikah", "nikau", "nills", "nimbi", "nimbs", "nimps", "niner", "nines", "ninon", "nipas", "nippy", "niqab", "nirls", "nirly", "nisei", "nisse", "nisus", "niter", "nites", "nitid", "niton", "nitre", "nitro", "nitry", "nitty", "nival", "nixed", "nixer", "nixes", "nixie", "nizam", "nkosi", "noahs", "nobby", "nocks", "nodal", "noddy", "nodes", "nodus", "noels", "noggs", "nohow", "noils", "noily", "noint", "noirs", "noles", "nolls", "nolos", "nomas", "nomen", "nomes", "nomic", "nomoi", "nomos", "nonas", "nonce", "nones", "nonet", "nongs", "nonis", "nonny", "nonyl", "noobs", "nooit", "nooks", "nooky", "noons", "noops", "nopal", "noria", "noris", "norks", "norma", "norms", "nosed", "noser", "noses", "notal", "noted", "noter", "notes", "notum", "nould", "noule", "nouls", "nouns", "nouny", "noups", "novae", "novas", "novum", "noway", "nowed", "nowls", "nowts", "nowty", "noxal", "noxes", "noyau", "noyed", "noyes", "nubby", "nubia", "nucha", "nuddy", "nuder", "nudes", "nudie", "nudzh", "nuffs", "nugae", "nuked", "nukes", "nulla", "nulls", "numbs", "numen", "nummy", "nunny", "nurds", "nurdy", "nurls", "nurrs", "nutso", "nutsy", "nyaff", "nyala", "nying", "nyssa", "oaked", "oaker", "oakum", "oared", "oases", "oasis", "oasts", "oaten", "oater", "oaths", "oaves", "obang", "obeah", "obeli", "obeys", "obias", "obied", "obiit", "obits", "objet", "oboes", "obole", "oboli", "obols", "occam", "ocher", "oches", "ochre", "ochry", "ocker", "ocrea", "octad", "octan", "octas", "octyl", "oculi", "odahs", "odals", "odeon", "odeum", "odism", "odist", "odium", "odors", "odour", "odyle", "odyls", "ofays", "offed", "offie", "oflag", "ofter", "ogams", "ogeed", "ogees", "oggin", "ogham", "ogive", "ogled", "ogler", "ogles", "ogmic", "ogres", "ohias", "ohing", "ohmic", "ohone", "oidia", "oiled", "oiler", "oinks", "oints", "ojime", "okapi", "okays", "okehs", "okras", "oktas", "oldie", "oleic", "olein", "olent", "oleos", "oleum", "olios", "ollas", "ollav", "oller", "ollie", "ology", "olpae", "olpes", "omasa", "omber", "ombus", "omens", "omers", "omits", "omlah", "omovs", "omrah", "oncer", "onces", "oncet", "oncus", "onely", "oners", "onery", "onium", "onkus", "onlay", "onned", "ontic", "oobit", "oohed", "oomph", "oonts", "ooped", "oorie", "ooses", "ootid", "oozed", "oozes", "opahs", "opals", "opens", "opepe", "oping", "oppos", "opsin", "opted", "opter", "orach", "oracy", "orals", "orang", "orant", "orate", "orbed", "orcas", "orcin", "ordos", "oread", "orfes", "orgia", "orgic", "orgue", "oribi", "oriel", "orixa", "orles", "orlon", "orlop", "ormer", "ornis", "orpin", "orris", "ortho", "orval", "orzos", "oscar", "oshac", "osier", "osmic", "osmol", "ossia", "ostia", "otaku", "otary", "ottar", "ottos", "oubit", "oucht", "ouens", "ouija", "oulks", "oumas", "oundy", "oupas", "ouped", "ouphe", "ouphs", "ourie", "ousel", "ousts", "outby", "outed", "outre", "outro", "outta", "ouzel", "ouzos", "ovals", "ovels", "ovens", "overs", "ovist", "ovoli", "ovolo", "ovule", "owche", "owies", "owled", "owler", "owlet", "owned", "owres", "owrie", "owsen", "oxbow", "oxers", "oxeye", "oxids", "oxies", "oxime", "oxims", "oxlip", "oxter", "oyers", "ozeki", "ozzie", "paals", "paans", "pacas", "paced", "pacer", "paces", "pacey", "pacha", "packs", "pacos", "pacta", "pacts", "padis", "padle", "padma", "padre", "padri", "paean", "paedo", "paeon", "paged", "pager", "pages", "pagle", "pagod", "pagri", "paiks", "pails", "pains", "paire", "pairs", "paisa", "paise", "pakka", "palas", "palay", "palea", "paled", "pales", "palet", "palis", "palki", "palla", "palls", "pally", "palms", "palmy", "palpi", "palps", "palsa", "pampa", "panax", "pance", "panda", "pands", "pandy", "paned", "panes", "panga", "pangs", "panim", "panko", "panne", "panni", "panto", "pants", "panty", "paoli", "paolo", "papas", "papaw", "papes", "pappi", "pappy", "parae", "paras", "parch", "pardi", "pards", "pardy", "pared", "paren", "pareo", "pares", "pareu", "parev", "parge", "pargo", "paris", "parki", "parks", "parky", "parle", "parly", "parma", "parol", "parps", "parra", "parrs", "parti", "parts", "parve", "parvo", "paseo", "pases", "pasha", "pashm", "paska", "paspy", "passe", "pasts", "pated", "paten", "pater", "pates", "paths", "patin", "patka", "patly", "patte", "patus", "pauas", "pauls", "pavan", "paved", "paven", "paver", "paves", "pavid", "pavin", "pavis", "pawas", "pawaw", "pawed", "pawer", "pawks", "pawky", "pawls", "pawns", "paxes", "payed", "payor", "paysd", "peage", "peags", "peaks", "peaky", "peals", "peans", "peare", "pears", "peart", "pease", "peats", "peaty", "peavy", "peaze", "pebas", "pechs", "pecke", "pecks", "pecky", "pedes", "pedis", "pedro", "peece", "peeks", "peels", "peens", "peeoy", "peepe", "peeps", "peers", "peery", "peeve", "peggy", "peghs", "peins", "peise", "peize", "pekan", "pekes", "pekin", "pekoe", "pelas", "pelau", "peles", "pelfs", "pells", "pelma", "pelon", "pelta", "pelts", "pends", "pendu", "pened", "penes", "pengo", "penie", "penis", "penks", "penna", "penni", "pents", "peons", "peony", "pepla", "pepos", "peppy", "pepsi", "perai", "perce", "percs", "perdu", "perdy", "perea", "peres", "peris", "perks", "perms", "perns", "perog", "perps", "perry", "perse", "perst", "perts", "perve", "pervo", "pervs", "pervy", "pesos", "pests", "pesty", "petar", "peter", "petit", "petre", "petri", "petti", "petto", "pewee", "pewit", "peyse", "phage", "phang", "phare", "pharm", "pheer", "phene", "pheon", "phese", "phial", "phish", "phizz", "phlox", "phoca", "phono", "phons", "phots", "phpht", "phuts", "phyla", "phyle", "piani", "pians", "pibal", "pical", "picas", "piccy", "picks", "picot", "picra", "picul", "piend", "piers", "piert", "pieta", "piets", "piezo", "pight", "pigmy", "piing", "pikas", "pikau", "piked", "piker", "pikes", "pikey", "pikis", "pikul", "pilae", "pilaf", "pilao", "pilar", "pilau", "pilaw", "pilch", "pilea", "piled", "pilei", "piler", "piles", "pilis", "pills", "pilow", "pilum", "pilus", "pimas", "pimps", "pinas", "pined", "pines", "pingo", "pings", "pinko", "pinks", "pinna", "pinny", "pinon", "pinot", "pinta", "pints", "pinup", "pions", "piony", "pious", "pioye", "pioys", "pipal", "pipas", "piped", "pipes", "pipet", "pipis", "pipit", "pippy", "pipul", "pirai", "pirls", "pirns", "pirog", "pisco", "pises", "pisky", "pisos", "pissy", "piste", "pitas", "piths", "piton", "pitot", "pitta", "piums", "pixes", "pized", "pizes", "plaas", "plack", "plage", "plans", "plaps", "plash", "plasm", "plast", "plats", "platt", "platy", "playa", "plays", "pleas", "plebe", "plebs", "plena", "pleon", "plesh", "plews", "plica", "plies", "plims", "pling", "plink", "ploat", "plods", "plong", "plonk", "plook", "plops", "plots", "plotz", "plouk", "plows", "ploye", "ploys", "plues", "pluff", "plugs", "plums", "plumy", "pluot", "pluto", "plyer", "poach", "poaka", "poake", "poboy", "pocks", "pocky", "podal", "poddy", "podex", "podge", "podgy", "podia", "poems", "poeps", "poets", "pogey", "pogge", "pogos", "pohed", "poilu", "poind", "pokal", "poked", "pokes", "pokey", "pokie", "poled", "poler", "poles", "poley", "polio", "polis", "polje", "polks", "polls", "polly", "polos", "polts", "polys", "pombe", "pomes", "pommy", "pomos", "pomps", "ponce", "poncy", "ponds", "pones", "poney", "ponga", "pongo", "pongs", "pongy", "ponks", "ponts", "ponty", "ponzu", "poods", "pooed", "poofs", "poofy", "poohs", "pooja", "pooka", "pooks", "pools", "poons", "poops", "poopy", "poori", "poort", "poots", "poove", "poovy", "popes", "poppa", "popsy", "porae", "poral", "pored", "porer", "pores", "porge", "porgy", "porin", "porks", "porky", "porno", "porns", "porny", "porta", "ports", "porty", "posed", "poses", "posey", "posho", "posts", "potae", "potch", "poted", "potes", "potin", "potoo", "potsy", "potto", "potts", "potty", "pouff", "poufs", "pouke", "pouks", "poule", "poulp", "poult", "poupe", "poupt", "pours", "pouts", "powan", "powin", "pownd", "powns", "powny", "powre", "poxed", "poxes", "poynt", "poyou", "poyse", "pozzy", "praam", "prads", "prahu", "prams", "prana", "prang", "praos", "prase", "prate", "prats", "pratt", "praty", "praus", "prays", "predy", "preed", "prees", "preif", "prems", "premy", "prent", "preon", "preop", "preps", "presa", "prese", "prest", "preve", "prexy", "preys", "prial", "pricy", "prief", "prier", "pries", "prigs", "prill", "prima", "primi", "primp", "prims", "primy", "prink", "prion", "prise", "priss", "proas", "probs", "prods", "proem", "profs", "progs", "proin", "proke", "prole", "proll", "promo", "proms", "pronk", "props", "prore", "proso", "pross", "prost", "prosy", "proto", "proul", "prows", "proyn", "prunt", "pruta", "pryer", "pryse", "pseud", "pshaw", "psion", "psoae", "psoai", "psoas", "psora", "psych", "psyop", "pubco", "pubes", "pubis", "pucan", "pucer", "puces", "pucka", "pucks", "puddy", "pudge", "pudic", "pudor", "pudsy", "pudus", "puers", "puffa", "puffs", "puggy", "pugil", "puhas", "pujah", "pujas", "pukas", "puked", "puker", "pukes", "pukey", "pukka", "pukus", "pulao", "pulas", "puled", "puler", "pules", "pulik", "pulis", "pulka", "pulks", "pulli", "pulls", "pully", "pulmo", "pulps", "pulus", "pumas", "pumie", "pumps", "punas", "punce", "punga", "pungs", "punji", "punka", "punks", "punky", "punny", "punto", "punts", "punty", "pupae", "pupas", "pupus", "purda", "pured", "pures", "purin", "puris", "purls", "purpy", "purrs", "pursy", "purty", "puses", "pusle", "pussy", "putid", "puton", "putti", "putto", "putts", "puzel", "pwned", "pyats", "pyets", "pygal", "pyins", "pylon", "pyned", "pynes", "pyoid", "pyots", "pyral", "pyran", "pyres", "pyrex", "pyric", "pyros", "pyxed", "pyxes", "pyxie", "pyxis", "pzazz", "qadis", "qaids", "qajaq", "qanat", "qapik", "qibla", "qophs", "qorma", "quads", "quaff", "quags", "quair", "quais", "quaky", "quale", "quant", "quare", "quass", "quate", "quats", "quayd", "quays", "qubit", "quean", "queme", "quena", "quern", "queyn", "queys", "quich", "quids", "quiff", "quims", "quina", "quine", "quino", "quins", "quint", "quipo", "quips", "quipu", "quire", "quirt", "quist", "quits", "quoad", "quods", "quoif", "quoin", "quoit", "quoll", "quonk", "quops", "qursh", "quyte", "rabat", "rabic", "rabis", "raced", "races", "rache", "racks", "racon", "radge", "radix", "radon", "raffs", "rafts", "ragas", "ragde", "raged", "ragee", "rager", "rages", "ragga", "raggs", "raggy", "ragis", "ragus", "rahed", "rahui", "raias", "raids", "raiks", "raile", "rails", "raine", "rains", "raird", "raita", "raits", "rajas", "rajes", "raked", "rakee", "raker", "rakes", "rakia", "rakis", "rakus", "rales", "ramal", "ramee", "ramet", "ramie", "ramin", "ramis", "rammy", "ramps", "ramus", "ranas", "rance", "rands", "ranee", "ranga", "rangi", "rangs", "rangy", "ranid", "ranis", "ranke", "ranks", "rants", "raped", "raper", "rapes", "raphe", "rappe", "rared", "raree", "rares", "rarks", "rased", "raser", "rases", "rasps", "rasse", "rasta", "ratal", "ratan", "ratas", "ratch", "rated", "ratel", "rater", "rates", "ratha", "rathe", "raths", "ratoo", "ratos", "ratus", "rauns", "raupo", "raved", "ravel", "raver", "raves", "ravey", "ravin", "rawer", "rawin", "rawly", "rawns", "raxed", "raxes", "rayah", "rayas", "rayed", "rayle", "rayne", "razed", "razee", "razer", "razes", "razoo", "readd", "reads", "reais", "reaks", "realo", "reals", "reame", "reams", "reamy", "reans", "reaps", "rears", "reast", "reata", "reate", "reave", "rebbe", "rebec", "rebid", "rebit", "rebop", "rebuy", "recal", "recce", "recco", "reccy", "recit", "recks", "recon", "recta", "recti", "recto", "redan", "redds", "reddy", "reded", "redes", "redia", "redid", "redip", "redly", "redon", "redos", "redox", "redry", "redub", "redux", "redye", "reech", "reede", "reeds", "reefs", "reefy", "reeks", "reeky", "reels", "reens", "reest", "reeve", "refed", "refel", "reffo", "refis", "refix", "refly", "refry", "regar", "reges", "reggo", "regie", "regma", "regna", "regos", "regur", "rehem", "reifs", "reify", "reiki", "reiks", "reink", "reins", "reird", "reist", "reive", "rejig", "rejon", "reked", "rekes", "rekey", "relet", "relie", "relit", "rello", "reman", "remap", "remen", "remet", "remex", "remix", "renay", "rends", "reney", "renga", "renig", "renin", "renne", "renos", "rente", "rents", "reoil", "reorg", "repeg", "repin", "repla", "repos", "repot", "repps", "repro", "reran", "rerig", "resat", "resaw", "resay", "resee", "reses", "resew", "resid", "resit", "resod", "resow", "resto", "rests", "resty", "resus", "retag", "retax", "retem", "retia", "retie", "retox", "revet", "revie", "rewan", "rewax", "rewed", "rewet", "rewin", "rewon", "rewth", "rexes", "rezes", "rheas", "rheme", "rheum", "rhies", "rhime", "rhine", "rhody", "rhomb", "rhone", "rhumb", "rhyne", "rhyta", "riads", "rials", "riant", "riata", "ribas", "ribby", "ribes", "riced", "ricer", "rices", "ricey", "richt", "ricin", "ricks", "rides", "ridgy", "ridic", "riels", "riems", "rieve", "rifer", "riffs", "rifte", "rifts", "rifty", "riggs", "rigol", "riled", "riles", "riley", "rille", "rills", "rimae", "rimed", "rimer", "rimes", "rimus", "rinds", "rindy", "rines", "rings", "rinks", "rioja", "riots", "riped", "ripes", "ripps", "rises", "rishi", "risks", "risps", "risus", "rites", "ritts", "ritzy", "rivas", "rived", "rivel", "riven", "rives", "riyal", "rizas", "roads", "roams", "roans", "roars", "roary", "roate", "robed", "robes", "roble", "rocks", "roded", "rodes", "roguy", "rohes", "roids", "roils", "roily", "roins", "roist", "rojak", "rojis", "roked", "roker", "rokes", "rolag", "roles", "rolfs", "rolls", "romal", "roman", "romeo", "romps", "ronde", "rondo", "roneo", "rones", "ronin", "ronne", "ronte", "ronts", "roods", "roofs", "roofy", "rooks", "rooky", "rooms", "roons", "roops", "roopy", "roosa", "roose", "roots", "rooty", "roped", "roper", "ropes", "ropey", "roque", "roral", "rores", "roric", "rorid", "rorie", "rorts", "rorty", "rosed", "roses", "roset", "roshi", "rosin", "rosit", "rosti", "rosts", "rotal", "rotan", "rotas", "rotch", "roted", "rotes", "rotis", "rotls", "roton", "rotos", "rotte", "rouen", "roues", "roule", "rouls", "roums", "roups", "roupy", "roust", "routh", "routs", "roved", "roven", "roves", "rowan", "rowed", "rowel", "rowen", "rowie", "rowme", "rownd", "rowth", "rowts", "royne", "royst", "rozet", "rozit", "ruana", "rubai", "rubby", "rubel", "rubes", "rubin", "ruble", "rubli", "rubus", "ruche", "rucks", "rudas", "rudds", "rudes", "rudie", "rudis", "rueda", "ruers", "ruffe", "ruffs", "rugae", "rugal", "ruggy", "ruing", "ruins", "rukhs", "ruled", "rules", "rumal", "rumbo", "rumen", "rumes", "rumly", "rummy", "rumpo", "rumps", "rumpy", "runch", "runds", "runed", "runes", "rungs", "runic", "runny", "runts", "runty", "rupia", "rurps", "rurus", "rusas", "ruses", "rushy", "rusks", "rusma", "russe", "rusts", "ruths", "rutin", "rutty", "ryals", "rybat", "ryked", "rykes", "rymme", "rynds", "ryots", "ryper", "saags", "sabal", "sabed", "saber", "sabes", "sabha", "sabin", "sabir", "sable", "sabot", "sabra", "sabre", "sacks", "sacra", "saddo", "sades", "sadhe", "sadhu", "sadis", "sados", "sadza", "safed", "safes", "sagas", "sager", "sages", "saggy", "sagos", "sagum", "saheb", "sahib", "saice", "saick", "saics", "saids", "saiga", "sails", "saims", "saine", "sains", "sairs", "saist", "saith", "sajou", "sakai", "saker", "sakes", "sakia", "sakis", "sakti", "salal", "salat", "salep", "sales", "salet", "salic", "salix", "salle", "salmi", "salol", "salop", "salpa", "salps", "salse", "salto", "salts", "salue", "salut", "saman", "samas", "samba", "sambo", "samek", "samel", "samen", "sames", "samey", "samfu", "sammy", "sampi", "samps", "sands", "saned", "sanes", "sanga", "sangh", "sango", "sangs", "sanko", "sansa", "santo", "sants", "saola", "sapan", "sapid", "sapor", "saran", "sards", "sared", "saree", "sarge", "sargo", "sarin", "saris", "sarks", "sarky", "sarod", "saros", "sarus", "saser", "sasin", "sasse", "satai", "satay", "sated", "satem", "sates", "satis", "sauba", "sauch", "saugh", "sauls", "sault", "saunt", "saury", "sauts", "saved", "saver", "saves", "savey", "savin", "sawah", "sawed", "sawer", "saxes", "sayed", "sayer", "sayid", "sayne", "sayon", "sayst", "sazes", "scabs", "scads", "scaff", "scags", "scail", "scala", "scall", "scams", "scand", "scans", "scapa", "scape", "scapi", "scarp", "scars", "scart", "scath", "scats", "scatt", "scaud", "scaup", "scaur", "scaws", "sceat", "scena", "scend", "schav", "schmo", "schul", "schwa", "sclim", "scody", "scogs", "scoog", "scoot", "scopa", "scops", "scots", "scoug", "scoup", "scowp", "scows", "scrab", "scrae", "scrag", "scran", "scrat", "scraw", "scray", "scrim", "scrip", "scrob", "scrod", "scrog", "scrow", "scudi", "scudo", "scuds", "scuff", "scuft", "scugs", "sculk", "scull", "sculp", "sculs", "scums", "scups", "scurf", "scurs", "scuse", "scuta", "scute", "scuts", "scuzz", "scyes", "sdayn", "sdein", "seals", "seame", "seams", "seamy", "seans", "seare", "sears", "sease", "seats", "seaze", "sebum", "secco", "sechs", "sects", "seder", "sedes", "sedge", "sedgy", "sedum", "seeds", "seeks", "seeld", "seels", "seely", "seems", "seeps", "seepy", "seers", "sefer", "segar", "segni", "segno", "segol", "segos", "sehri", "seifs", "seils", "seine", "seirs", "seise", "seism", "seity", "seiza", "sekos", "sekts", "selah", "seles", "selfs", "sella", "selle", "sells", "selva", "semee", "semes", "semie", "semis", "senas", "sends", "senes", "sengi", "senna", "senor", "sensa", "sensi", "sente", "senti", "sents", "senvy", "senza", "sepad", "sepal", "sepic", "sepoy", "septa", "septs", "serac", "serai", "seral", "sered", "serer", "seres", "serfs", "serge", "seric", "serin", "serks", "seron", "serow", "serra", "serre", "serrs", "serry", "servo", "sesey", "sessa", "setae", "setal", "seton", "setts", "sewan", "sewar", "sewed", "sewel", "sewen", "sewin", "sexed", "sexer", "sexes", "sexto", "sexts", "seyen", "shads", "shags", "shahs", "shako", "shakt", "shalm", "shaly", "shama", "shams", "shand", "shans", "shaps", "sharn", "shash", "shaul", "shawm", "shawn", "shaws", "shaya", "shays", "shchi", "sheaf", "sheal", "sheas", "sheds", "sheel", "shend", "shent", "sheol", "sherd", "shere", "shero", "shets", "sheva", "shewn", "shews", "shiai", "shiel", "shier", "shies", "shill", "shily", "shims", "shins", "ships", "shirr", "shirs", "shish", "shiso", "shist", "shite", "shits", "shiur", "shiva", "shive", "shivs", "shlep", "shlub", "shmek", "shmoe", "shoat", "shoed", "shoer", "shoes", "shogi", "shogs", "shoji", "shojo", "shola", "shool", "shoon", "shoos", "shope", "shops", "shorl", "shote", "shots", "shott", "showd", "shows", "shoyu", "shred", "shris", "shrow", "shtik", "shtum", "shtup", "shule", "shuln", "shuls", "shuns", "shura", "shute", "shuts", "shwas", "shyer", "sials", "sibbs", "sibyl", "sices", "sicht", "sicko", "sicks", "sicky", "sidas", "sided", "sider", "sides", "sidha", "sidhe", "sidle", "sield", "siens", "sient", "sieth", "sieur", "sifts", "sighs", "sigil", "sigla", "signa", "signs", "sijos", "sikas", "siker", "sikes", "silds", "siled", "silen", "siler", "siles", "silex", "silks", "sills", "silos", "silts", "silty", "silva", "simar", "simas", "simba", "simis", "simps", "simul", "sinds", "sined", "sines", "sings", "sinhs", "sinks", "sinky", "sinus", "siped", "sipes", "sippy", "sired", "siree", "sires", "sirih", "siris", "siroc", "sirra", "sirup", "sisal", "sises", "sista", "sists", "sitar", "sited", "sites", "sithe", "sitka", "situp", "situs", "siver", "sixer", "sixes", "sixmo", "sixte", "sizar", "sized", "sizel", "sizer", "sizes", "skags", "skail", "skald", "skank", "skart", "skats", "skatt", "skaws", "skean", "skear", "skeds", "skeed", "skeef", "skeen", "skeer", "skees", "skeet", "skegg", "skegs", "skein", "skelf", "skell", "skelm", "skelp", "skene", "skens", "skeos", "skeps", "skers", "skets", "skews", "skids", "skied", "skies", "skiey", "skimo", "skims", "skink", "skins", "skint", "skios", "skips", "skirl", "skirr", "skite", "skits", "skive", "skivy", "sklim", "skoal", "skody", "skoff", "skogs", "skols", "skool", "skort", "skosh", "skran", "skrik", "skuas", "skugs", "skyed", "skyer", "skyey", "skyfs", "skyre", "skyrs", "skyte", "slabs", "slade", "slaes", "slags", "slaid", "slake", "slams", "slane", "slank", "slaps", "slart", "slats", "slaty", "slaws", "slays", "slebs", "sleds", "sleer", "slews", "sleys", "slier", "slily", "slims", "slipe", "slips", "slipt", "slish", "slits", "slive", "sloan", "slobs", "sloes", "slogs", "sloid", "slojd", "slomo", "sloom", "sloot", "slops", "slopy", "slorm", "slots", "slove", "slows", "sloyd", "slubb", "slubs", "slued", "slues", "sluff", "slugs", "sluit", "slums", "slurb", "slurs", "sluse", "sluts", "slyer", "slype", "smaak", "smaik", "smalm", "smalt", "smarm", "smaze", "smeek", "smees", "smeik", "smeke", "smerk", "smews", "smirr", "smirs", "smits", "smogs", "smoko", "smolt", "smoor", "smoot", "smore", "smorg", "smout", "smowt", "smugs", "smurs", "smush", "smuts", "snabs", "snafu", "snags", "snaps", "snarf", "snark", "snars", "snary", "snash", "snath", "snaws", "snead", "sneap", "snebs", "sneck", "sneds", "sneed", "snees", "snell", "snibs", "snick", "snies", "snift", "snigs", "snips", "snipy", "snirt", "snits", "snobs", "snods", "snoek", "snoep", "snogs", "snoke", "snood", "snook", "snool", "snoot", "snots", "snowk", "snows", "snubs", "snugs", "snush", "snyes", "soaks", "soaps", "soare", "soars", "soave", "sobas", "socas", "soces", "socko", "socks", "socle", "sodas", "soddy", "sodic", "sodom", "sofar", "sofas", "softa", "softs", "softy", "soger", "sohur", "soils", "soily", "sojas", "sojus", "sokah", "soken", "sokes", "sokol", "solah", "solan", "solas", "solde", "soldi", "soldo", "solds", "soled", "solei", "soler", "soles", "solon", "solos", "solum", "solus", "soman", "somas", "sonce", "sonde", "sones", "songs", "sonly", "sonne", "sonny", "sonse", "sonsy", "sooey", "sooks", "sooky", "soole", "sools", "sooms", "soops", "soote", "soots", "sophs", "sophy", "sopor", "soppy", "sopra", "soral", "soras", "sorbo", "sorbs", "sorda", "sordo", "sords", "sored", "soree", "sorel", "sorer", "sores", "sorex", "sorgo", "sorns", "sorra", "sorta", "sorts", "sorus", "soths", "sotol", "souce", "souct", "sough", "souks", "souls", "soums", "soups", "soupy", "sours", "souse", "souts", "sowar", "sowce", "sowed", "sowff", "sowfs", "sowle", "sowls", "sowms", "sownd", "sowne", "sowps", "sowse", "sowth", "soyas", "soyle", "soyuz", "sozin", "spacy", "spado", "spaed", "spaer", "spaes", "spags", "spahi", "spail", "spain", "spait", "spake", "spald", "spale", "spall", "spalt", "spams", "spane", "spang", "spans", "spard", "spars", "spart", "spate", "spats", "spaul", "spawl", "spaws", "spayd", "spays", "spaza", "spazz", "speal", "spean", "speat", "specs", "spect", "speel", "speer", "speil", "speir", "speks", "speld", "spelk", "speos", "spets", "speug", "spews", "spewy", "spial", "spica", "spick", "spics", "spide", "spier", "spies", "spiff", "spifs", "spiks", "spile", "spims", "spina", "spink", "spins", "spirt", "spiry", "spits", "spitz", "spivs", "splay", "splog", "spode", "spods", "spoom", "spoor", "spoot", "spork", "sposh", "spots", "sprad", "sprag", "sprat", "spred", "sprew", "sprit", "sprod", "sprog", "sprue", "sprug", "spuds", "spued", "spuer", "spues", "spugs", "spule", "spume", "spumy", "spurs", "sputa", "spyal", "spyre", "squab", "squaw", "squeg", "squid", "squit", "squiz", "stabs", "stade", "stags", "stagy", "staig", "stane", "stang", "staph", "staps", "starn", "starr", "stars", "stats", "staun", "staws", "stays", "stean", "stear", "stedd", "stede", "steds", "steek", "steem", "steen", "steil", "stela", "stele", "stell", "steme", "stems", "stend", "steno", "stens", "stent", "steps", "stept", "stere", "stets", "stews", "stewy", "steys", "stich", "stied", "sties", "stilb", "stile", "stime", "stims", "stimy", "stipa", "stipe", "stire", "stirk", "stirp", "stirs", "stive", "stivy", "stoae", "stoai", "stoas", "stoat", "stobs", "stoep", "stogy", "stoit", "stoln", "stoma", "stond", "stong", "stonk", "stonn", "stook", "stoor", "stope", "stops", "stopt", "stoss", "stots", "stott", "stoun", "stoup", "stour", "stown", "stowp", "stows", "strad", "strae", "strag", "strak", "strep", "strew", "stria", "strig", "strim", "strop", "strow", "stroy", "strum", "stubs", "stude", "studs", "stull", "stulm", "stumm", "stums", "stuns", "stupa", "stupe", "sture", "sturt", "styed", "styes", "styli", "stylo", "styme", "stymy", "styre", "styte", "subah", "subas", "subby", "suber", "subha", "succi", "sucks", "sucky", "sucre", "sudds", "sudor", "sudsy", "suede", "suent", "suers", "suete", "suets", "suety", "sugan", "sughs", "sugos", "suhur", "suids", "suint", "suits", "sujee", "sukhs", "sukuk", "sulci", "sulfa", "sulfo", "sulks", "sulph", "sulus", "sumis", "summa", "sumos", "sumph", "sumps", "sunis", "sunks", "sunna", "sunns", "sunup", "supes", "supra", "surah", "sural", "suras", "surat", "surds", "sured", "sures", "surfs", "surfy", "surgy", "surra", "sused", "suses", "susus", "sutor", "sutra", "sutta", "swabs", "swack", "swads", "swage", "swags", "swail", "swain", "swale", "swaly", "swamy", "swang", "swank", "swans", "swaps", "swapt", "sward", "sware", "swarf", "swart", "swats", "swayl", "sways", "sweal", "swede", "sweed", "sweel", "sweer", "swees", "sweir", "swelt", "swerf", "sweys", "swies", "swigs", "swile", "swims", "swink", "swipe", "swire", "swiss", "swith", "swits", "swive", "swizz", "swobs", "swole", "swoln", "swops", "swopt", "swots", "swoun", "sybbe", "sybil", "syboe", "sybow", "sycee", "syces", "sycon", "syens", "syker", "sykes", "sylis", "sylph", "sylva", "symar", "synch", "syncs", "synds", "syned", "synes", "synth", "syped", "sypes", "syphs", "syrah", "syren", "sysop", "sythe", "syver", "taals", "taata", "taber", "tabes", "tabid", "tabis", "tabla", "tabor", "tabun", "tabus", "tacan", "taces", "tacet", "tache", "tacho", "tachs", "tacks", "tacos", "tacts", "taels", "tafia", "taggy", "tagma", "tahas", "tahrs", "taiga", "taigs", "taiko", "tails", "tains", "taira", "taish", "taits", "tajes", "takas", "takes", "takhi", "takin", "takis", "takky", "talak", "talaq", "talar", "talas", "talcs", "talcy", "talea", "taler", "tales", "talks", "talky", "talls", "talma", "talpa", "taluk", "talus", "tamal", "tamed", "tames", "tamin", "tamis", "tammy", "tamps", "tanas", "tanga", "tangi", "tangs", "tanhs", "tanka", "tanks", "tanky", "tanna", "tansy", "tanti", "tanto", "tanty", "tapas", "taped", "tapen", "tapes", "tapet", "tapis", "tappa", "tapus", "taras", "tardo", "tared", "tares", "targa", "targe", "tarns", "taroc", "tarok", "taros", "tarps", "tarre", "tarry", "tarsi", "tarts", "tarty", "tasar", "tased", "taser", "tases", "tasks", "tassa", "tasse", "tasso", "tatar", "tater", "tates", "taths", "tatie", "tatou", "tatts", "tatus", "taube", "tauld", "tauon", "taupe", "tauts", "tavah", "tavas", "taver", "tawai", "tawas", "tawed", "tawer", "tawie", "tawse", "tawts", "taxed", "taxer", "taxes", "taxis", "taxol", "taxon", "taxor", "taxus", "tayra", "tazza", "tazze", "teade", "teads", "teaed", "teaks", "teals", "teams", "tears", "teats", "teaze", "techs", "techy", "tecta", "teels", "teems", "teend", "teene", "teens", "teeny", "teers", "teffs", "teggs", "tegua", "tegus", "tehrs", "teiid", "teils", "teind", "teins", "telae", "telco", "teles", "telex", "telia", "telic", "tells", "telly", "teloi", "telos", "temed", "temes", "tempi", "temps", "tempt", "temse", "tench", "tends", "tendu", "tenes", "tenge", "tenia", "tenne", "tenno", "tenny", "tenon", "tents", "tenty", "tenue", "tepal", "tepas", "tepoy", "terai", "teras", "terce", "terek", "teres", "terfe", "terfs", "terga", "terms", "terne", "terns", "terry", "terts", "tesla", "testa", "teste", "tests", "tetes", "teths", "tetra", "tetri", "teuch", "teugh", "tewed", "tewel", "tewit", "texas", "texes", "texts", "thack", "thagi", "thaim", "thale", "thali", "thana", "thane", "thang", "thans", "thanx", "tharm", "thars", "thaws", "thawy", "thebe", "theca", "theed", "theek", "thees", "thegn", "theic", "thein", "thelf", "thema", "thens", "theow", "therm", "thesp", "thete", "thews", "thewy", "thigs", "thilk", "thill", "thine", "thins", "thiol", "thirl", "thoft", "thole", "tholi", "thoro", "thorp", "thous", "thowl", "thrae", "thraw", "thrid", "thrip", "throe", "thuds", "thugs", "thuja", "thunk", "thurl", "thuya", "thymi", "thymy", "tians", "tiars", "tical", "ticca", "ticed", "tices", "tichy", "ticks", "ticky", "tiddy", "tided", "tides", "tiers", "tiffs", "tifos", "tifts", "tiges", "tigon", "tikas", "tikes", "tikis", "tikka", "tilak", "tiled", "tiler", "tiles", "tills", "tilly", "tilth", "tilts", "timbo", "timed", "times", "timon", "timps", "tinas", "tinct", "tinds", "tinea", "tined", "tines", "tinge", "tings", "tinks", "tinny", "tints", "tinty", "tipis", "tippy", "tired", "tires", "tirls", "tiros", "tirrs", "titch", "titer", "titis", "titre", "titty", "titup", "tiyin", "tiyns", "tizes", "tizzy", "toads", "toady", "toaze", "tocks", "tocky", "tocos", "todde", "toeas", "toffs", "toffy", "tofts", "tofus", "togae", "togas", "toged", "toges", "togue", "tohos", "toile", "toils", "toing", "toise", "toits", "tokay", "toked", "toker", "tokes", "tokos", "tolan", "tolar", "tolas", "toled", "toles", "tolls", "tolly", "tolts", "tolus", "tolyl", "toman", "tombs", "tomes", "tomia", "tommy", "tomos", "tondi", "tondo", "toned", "toner", "tones", "toney", "tongs", "tonka", "tonks", "tonne", "tonus", "tools", "tooms", "toons", "toots", "toped", "topee", "topek", "toper", "topes", "tophe", "tophi", "tophs", "topis", "topoi", "topos", "toppy", "toque", "torah", "toran", "toras", "torcs", "tores", "toric", "torii", "toros", "torot", "torrs", "torse", "torsi", "torsk", "torta", "torte", "torts", "tosas", "tosed", "toses", "toshy", "tossy", "toted", "toter", "totes", "totty", "touks", "touns", "tours", "touse", "tousy", "touts", "touze", "touzy", "towed", "towie", "towns", "towny", "towse", "towsy", "towts", "towze", "towzy", "toyed", "toyer", "toyon", "toyos", "tozed", "tozes", "tozie", "trabs", "trads", "tragi", "traik", "trams", "trank", "tranq", "trans", "trant", "trape", "traps", "trapt", "trass", "trats", "tratt", "trave", "trayf", "trays", "treck", "treed", "treen", "trees", "trefa", "treif", "treks", "trema", "trems", "tress", "trest", "trets", "trews", "treyf", "treys", "triac", "tride", "trier", "tries", "triff", "trigo", "trigs", "trike", "trild", "trill", "trims", "trine", "trins", "triol", "trior", "trios", "trips", "tripy", "trist", "troad", "troak", "troat", "trock", "trode", "trods", "trogs", "trois", "troke", "tromp", "trona", "tronc", "trone", "tronk", "trons", "trooz", "troth", "trots", "trows", "troys", "trued", "trues", "trugo", "trugs", "trull", "tryer", "tryke", "tryma", "tryps", "tsade", "tsadi", "tsars", "tsked", "tsuba", "tsubo", "tuans", "tuart", "tuath", "tubae", "tubar", "tubas", "tubby", "tubed", "tubes", "tucks", "tufas", "tuffe", "tuffs", "tufts", "tufty", "tugra", "tuile", "tuina", "tuism", "tuktu", "tules", "tulpa", "tulsi", "tumid", "tummy", "tumps", "tumpy", "tunas", "tunds", "tuned", "tuner", "tunes", "tungs", "tunny", "tupek", "tupik", "tuple", "tuque", "turds", "turfs", "turfy", "turks", "turme", "turms", "turns", "turnt", "turps", "turrs", "tushy", "tusks", "tusky", "tutee", "tutti", "tutty", "tutus", "tuxes", "tuyer", "twaes", "twain", "twals", "twank", "twats", "tways", "tweel", "tween", "tweep", "tweer", "twerk", "twerp", "twier", "twigs", "twill", "twilt", "twink", "twins", "twiny", "twire", "twirp", "twite", "twits", "twoer", "twyer", "tyees", "tyers", "tyiyn", "tykes", "tyler", "tymps", "tynde", "tyned", "tynes", "typal", "typed", "types", "typey", "typic", "typos", "typps", "typto", "tyran", "tyred", "tyres", "tyros", "tythe", "tzars", "udals", "udons", "ugali", "ugged", "uhlan", "uhuru", "ukase", "ulama", "ulans", "ulema", "ulmin", "ulnad", "ulnae", "ulnar", "ulnas", "ulpan", "ulvas", "ulyie", "ulzie", "umami", "umbel", "umber", "umble", "umbos", "umbre", "umiac", "umiak", "umiaq", "ummah", "ummas", "ummed", "umped", "umphs", "umpie", "umpty", "umrah", "umras", "unais", "unapt", "unarm", "unary", "unaus", "unbag", "unban", "unbar", "unbed", "unbid", "unbox", "uncap", "unces", "uncia", "uncos", "uncoy", "uncus", "undam", "undee", "undos", "undug", "uneth", "unfix", "ungag", "unget", "ungod", "ungot", "ungum", "unhat", "unhip", "unica", "units", "unjam", "unked", "unket", "unkid", "unlaw", "unlay", "unled", "unlet", "unlid", "unman", "unmew", "unmix", "unpay", "unpeg", "unpen", "unpin", "unred", "unrid", "unrig", "unrip", "unsaw", "unsay", "unsee", "unsew", "unsex", "unsod", "untax", "untin", "unwet", "unwit", "unwon", "upbow", "upbye", "updos", "updry", "upend", "upjet", "uplay", "upled", "uplit", "upped", "upran", "uprun", "upsee", "upsey", "uptak", "upter", "uptie", "uraei", "urali", "uraos", "urare", "urari", "urase", "urate", "urbex", "urbia", "urdee", "ureal", "ureas", "uredo", "ureic", "urena", "urent", "urged", "urger", "urges", "urial", "urite", "urman", "urnal", "urned", "urped", "ursae", "ursid", "urson", "urubu", "urvas", "users", "usnea", "usque", "usure", "usury", "uteri", "uveal", "uveas", "uvula", "vacua", "vaded", "vades", "vagal", "vagus", "vails", "vaire", "vairs", "vairy", "vakas", "vakil", "vales", "valis", "valse", "vamps", "vampy", "vanda", "vaned", "vanes", "vangs", "vants", "vaped", "vaper", "vapes", "varan", "varas", "vardy", "varec", "vares", "varia", "varix", "varna", "varus", "varve", "vasal", "vases", "vasts", "vasty", "vatic", "vatus", "vauch", "vaute", "vauts", "vawte", "vaxes", "veale", "veals", "vealy", "veena", "veeps", "veers", "veery", "vegas", "veges", "vegie", "vegos", "vehme", "veils", "veily", "veins", "veiny", "velar", "velds", "veldt", "veles", "vells", "velum", "venae", "venal", "vends", "vendu", "veney", "venge", "venin", "vents", "venus", "verbs", "verra", "verry", "verst", "verts", "vertu", "vespa", "vesta", "vests", "vetch", "vexed", "vexer", "vexes", "vexil", "vezir", "vials", "viand", "vibes", "vibex", "vibey", "viced", "vices", "vichy", "viers", "views", "viewy", "vifda", "viffs", "vigas", "vigia", "vilde", "viler", "villi", "vills", "vimen", "vinal", "vinas", "vinca", "vined", "viner", "vines", "vinew", "vinic", "vinos", "vints", "viold", "viols", "vired", "vireo", "vires", "virga", "virge", "virid", "virls", "virtu", "visas", "vised", "vises", "visie", "visne", "vison", "visto", "vitae", "vitas", "vitex", "vitro", "vitta", "vivas", "vivat", "vivda", "viver", "vives", "vizir", "vizor", "vleis", "vlies", "vlogs", "voars", "vocab", "voces", "voddy", "vodou", "vodun", "voema", "vogie", "voids", "voile", "voips", "volae", "volar", "voled", "voles", "volet", "volks", "volta", "volte", "volti", "volts", "volva", "volve", "vomer", "voted", "votes", "vouge", "voulu", "vowed", "vower", "voxel", "vozhd", "vraic", "vrils", "vroom", "vrous", "vrouw", "vrows", "vuggs", "vuggy", "vughs", "vughy", "vulgo", "vulns", "vulva", "vutty", "waacs", "wacke", "wacko", "wacks", "wadds", "waddy", "waded", "wader", "wades", "wadge", "wadis", "wadts", "waffs", "wafts", "waged", "wages", "wagga", "wagyu", "wahoo", "waide", "waifs", "waift", "wails", "wains", "wairs", "waite", "waits", "wakas", "waked", "waken", "waker", "wakes", "wakfs", "waldo", "walds", "waled", "waler", "wales", "walie", "walis", "walks", "walla", "walls", "wally", "walty", "wamed", "wames", "wamus", "wands", "waned", "wanes", "waney", "wangs", "wanks", "wanky", "wanle", "wanly", "wanna", "wants", "wanty", "wanze", "waqfs", "warbs", "warby", "wards", "wared", "wares", "warez", "warks", "warms", "warns", "warps", "warre", "warst", "warts", "wases", "washy", "wasms", "wasps", "waspy", "wasts", "watap", "watts", "wauff", "waugh", "wauks", "waulk", "wauls", "waurs", "waved", "waves", "wavey", "wawas", "wawes", "wawls", "waxed", "waxer", "waxes", "wayed", "wazir", "wazoo", "weald", "weals", "weamb", "weans", "wears", "webby", "weber", "wecht", "wedel", "wedgy", "weeds", "weeke", "weeks", "weels", "weems", "weens", "weeny", "weeps", "weepy", "weest", "weete", "weets", "wefte", "wefts", "weids", "weils", "weirs", "weise", "weize", "wekas", "welds", "welke", "welks", "welkt", "wells", "welly", "welts", "wembs", "wends", "wenge", "wenny", "wents", "weros", "wersh", "wests", "wetas", "wetly", "wexed", "wexes", "whamo", "whams", "whang", "whaps", "whare", "whata", "whats", "whaup", "whaur", "wheal", "whear", "wheen", "wheep", "wheft", "whelk", "whelm", "whens", "whets", "whews", "wheys", "whids", "whift", "whigs", "whilk", "whims", "whins", "whios", "whips", "whipt", "whirr", "whirs", "whish", "whiss", "whist", "whits", "whity", "whizz", "whomp", "whoof", "whoot", "whops", "whore", "whorl", "whort", "whoso", "whows", "whump", "whups", "whyda", "wicca", "wicks", "wicky", "widdy", "wides", "wiels", "wifed", "wifes", "wifey", "wifie", "wifty", "wigan", "wigga", "wiggy", "wikis", "wilco", "wilds", "wiled", "wiles", "wilga", "wilis", "wilja", "wills", "wilts", "wimps", "winds", "wined", "wines", "winey", "winge", "wings", "wingy", "winks", "winna", "winns", "winos", "winze", "wiped", "wiper", "wipes", "wired", "wirer", "wires", "wirra", "wised", "wises", "wisha", "wisht", "wisps", "wists", "witan", "wited", "wites", "withe", "withs", "withy", "wived", "wiver", "wives", "wizen", "wizes", "woads", "woald", "wocks", "wodge", "woful", "wojus", "woker", "wokka", "wolds", "wolfs", "wolly", "wolve", "wombs", "womby", "womyn", "wonga", "wongi", "wonks", "wonky", "wonts", "woods", "wooed", "woofs", "woofy", "woold", "wools", "woons", "woops", "woopy", "woose", "woosh", "wootz", "words", "works", "worms", "wormy", "worts", "wowed", "wowee", "woxen", "wrang", "wraps", "wrapt", "wrast", "wrate", "wrawl", "wrens", "wrick", "wried", "wrier", "wries", "writs", "wroke", "wroot", "wroth", "wryer", "wuddy", "wudus", "wulls", "wurst", "wuses", "wushu", "wussy", "wuxia", "wyled", "wyles", "wynds", "wynns", "wyted", "wytes", "xebec", "xenia", "xenic", "xenon", "xeric", "xerox", "xerus", "xoana", "xrays", "xylan", "xylem", "xylic", "xylol", "xylyl", "xysti", "xysts", "yaars", "yabas", "yabba", "yabby", "yacca", "yacka", "yacks", "yaffs", "yager", "yages", "yagis", "yahoo", "yaird", "yakka", "yakow", "yales", "yamen", "yampy", "yamun", "yangs", "yanks", "yapok", "yapon", "yapps", "yappy", "yarak", "yarco", "yards", "yarer", "yarfa", "yarks", "yarns", "yarrs", "yarta", "yarto", "yates", "yauds", "yauld", "yaups", "yawed", "yawey", "yawls", "yawns", "yawny", "yawps", "ybore", "yclad", "ycled", "ycond", "ydrad", "ydred", "yeads", "yeahs", "yealm", "yeans", "yeard", "years", "yecch", "yechs", "yechy", "yedes", "yeeds", "yeesh", "yeggs", "yelks", "yells", "yelms", "yelps", "yelts", "yenta", "yente", "yerba", "yerds", "yerks", "yeses", "yesks", "yests", "yesty", "yetis", "yetts", "yeuks", "yeuky", "yeven", "yeves", "yewen", "yexed", "yexes", "yfere", "yiked", "yikes", "yills", "yince", "yipes", "yippy", "yirds", "yirks", "yirrs", "yirth", "yites", "yitie", "ylems", "ylike", "ylkes", "ymolt", "ympes", "yobbo", "yobby", "yocks", "yodel", "yodhs", "yodle", "yogas", "yogee", "yoghs", "yogic", "yogin", "yogis", "yoick", "yojan", "yoked", "yokel", "yoker", "yokes", "yokul", "yolks", "yolky", "yomim", "yomps", "yonic", "yonis", "yonks", "yoofs", "yoops", "yores", "yorks", "yorps", "youks", "yourn", "yours", "yourt", "youse", "yowed", "yowes", "yowie", "yowls", "yowza", "yrapt", "yrent", "yrivd", "yrneh", "ysame", "ytost", "yuans", "yucas", "yucca", "yucch", "yucko", "yucks", "yucky", "yufts", "yugas", "yuked", "yukes", "yukky", "yukos", "yulan", "yules", "yummo", "yummy", "yumps", "yupon", "yuppy", "yurta", "yurts", "yuzus", "zabra", "zacks", "zaida", "zaidy", "zaire", "zakat", "zaman", "zambo", "zamia", "zanja", "zante", "zanza", "zanze", "zappy", "zarfs", "zaris", "zatis", "zaxes", "zayin", "zazen", "zeals", "zebec", "zebub", "zebus", "zedas", "zeins", "zendo", "zerda", "zerks", "zeros", "zests", "zetas", "zexes", "zezes", "zhomo", "zibet", "ziffs", "zigan", "zilas", "zilch", "zilla", "zills", "zimbi", "zimbs", "zinco", "zincs", "zincy", "zineb", "zines", "zings", "zingy", "zinke", "zinky", "zippo", "zippy", "ziram", "zitis", "zizel", "zizit", "zlote", "zloty", "zoaea", "zobos", "zobus", "zocco", "zoeae", "zoeal", "zoeas", "zoism", "zoist", "zombi", "zonae", "zonda", "zoned", "zoner", "zones", "zonks", "zooea", "zooey", "zooid", "zooks", "zooms", "zoons", "zooty", "zoppa", "zoppo", "zoril", "zoris", "zorro", "zouks", "zowee", "zowie", "zulus", "zupan", "zupas", "zuppa", "zurfs", "zuzim", "zygal", "zygon", "zymes", "zymic"]
combined_list = soln_list + allowed_list

In [19]:
fonts = {0: u'\u2B1C', 1: u'\U0001f7e8', 2: u'\U0001f7e9'}

In [20]:
class GameState():
    non_solutions = allowed_list
    solutions = soln_list
    allowed = soln_list + allowed_list
    win = (2, 2, 2, 2, 2)
    miss = (0, 0, 0, 0, 0)
    max_turns = 6
    
    def __init__(self, previous=None, guesses=None, scores=None, turn=0, candidates=None, word=None):
        self.turn = turn
        self.previous = previous        
        self.guesses = guesses if guesses is not None else []
        self.scores = scores if scores is not None else []
        self.candidates = candidates if candidates is not None else GameState.solutions
        self.word = word if word is not None else random.choice(GameState.solutions)
        
        assert(len(self.guesses) == len(self.scores))

    def __repr__(self):
        """Format all pretty like"""
        if self.turn == 0:
            return GameState.format_result(GameState.miss)
        return "\n".join(f'{move}: {GameState.format_result(score)}' for move,score in zip(self.guesses, self.scores))
        
    def play_guess(self, guess):
        if guess not in GameState.allowed:
            print(f'Guess {guess} invalid, try again')
            return self
        guesses = self.guesses + [guess]
        score = GameState.evaluate_guess(guess, self.word)
        scores = self.scores + [score]
        next_state = GameState(previous=self, guesses=guesses, scores=scores,
                               turn=self.turn + 1, candidates=self.refine_candidates(guess, score), word=self.word)
        return next_state
        
    def refine_candidates(self, guess, score):
        return [x for x in self.candidates if self.evaluate_guess(guess, x) == score]
    
    def is_over(self):
        if self.turn == 0:
            return False
        return self.turn == GameState.max_turns or self.scores[-1] == GameState.win
    
    def game_result(self):
        player_win = self.scores[-1] == GameState.win
        if self.is_over():
            return player_win, self.turn
        else:
            return None
    
    @classmethod
    def new_game(cls, seed=None):
        word = cls.solutions[seed % len(cls.solutions)] if seed is not None else random.choice(cls.solutions)
        return GameState(word=word)
    
    @classmethod
    def evaluate_guess(cls, guess, truth):
        """{'miss': 0, 'contains': 1, 'correct': 2}"""
        correct = [2*(g==t) for g,t in zip(guess, truth)]  # cast as int
        remains = [t for t,c in zip(truth, correct) if not c]
        for i, (g,c) in enumerate(zip(guess, correct)):
            if not c and g in remains:
                correct[i] = 1
                remains.remove(g)
        return tuple(correct)

    @classmethod
    def format_result(cls, score):
        return " ".join(fonts[r] for r in score)

In [21]:
class WordleGameUpdate:
    
    def __init__(self, player, seed=None, verbose=True, summary=True):
        self.player = player
        self.game_state = GameState.new_game(seed)
        self.verbose = verbose
        self.summary = summary
        
    def play(self):
        while not self.game_state.is_over():
            if self.verbose:
                print(self.game_state)
            guess = self.player.play(self.game_state)
            self.game_state = self.game_state.play_guess(guess)
        result = self.game_state.game_result()
        if self.summary:
            print(self.game_state)
            print(f'Result: Player {"wins" if result[0] else "loses"} in {result[-1]} guesses')
            if not result[0]:
                print(f'Correct word was {self.game_state.word}')
        return self.game_state.game_result()

In [22]:
class Agent:
    
    def __init__(self):
        pass
    
    def play(self, game_state):
        pass
    
    def evaluate(self, score):
        pass
    
    def _score_guess(self, guess, truth):
        correct = [2*(g==t) for g,t in zip(guess, truth)]  # cast as int
        remains = [t for t,c in zip(truth, correct) if not c]
        for i, (g,c) in enumerate(zip(guess, correct)):
            if not c and g in remains:
                correct[i] = 1
                remains.remove(g)
        return tuple(correct)

In [23]:
class Basic(Agent):
    
    def __init__(self, verbose=True):
        self.candidates = soln_list
        self.verbose = verbose
    
    def play(self, game_state):
        self.candidates = game_state.candidates
        if self.verbose:
            print(f'Player: {len(self.candidates)} candidate(s) to choose from')
        return random.choice(self.candidates)

In [34]:
class Shallow2(Agent):
    """Strategy: choose the guess that gives the widest / most evenly distributed tree against the field """
    
    CACHED_RESULTS = {}
    
    def __init__(self, verbose=True, method=1):
        self.candidates = GameState.solutions
        self.move_candidates = GameState.allowed
        self.verbose = verbose
        self.method = method
        
    def _test_guess_vs_candidates(self, guess):
        tree = defaultdict(list)
        for word in self.candidates:
            tree[self._score_guess(guess, word)].append(word)
        return tree
    
    def _score_tree(self, tree):
        nbranches = len(tree)
        largest_branch = max(len(val) for val in tree.values())
        if self.method == 0:
            return (largest_branch, -nbranches)
        elif self.method == 1:
            return (-nbranches, largest_branch)
        else:
            raise ValueError('method needs to be 0 or 1')
    
    def _update_move_candidates(self):
        letters = set("".join(self.candidates))
        self.move_candidates = [word for word in self.move_candidates if set(word) & letters]
    
    def best_guess(self, guess_candidates, verbose=True, text=''):
        best_score = (len(self.candidates), 0) if self.method == 0 else (0, len(self.candidates))
        best_guess = None
        for guess in guess_candidates:
            tree = self._test_guess_vs_candidates(guess)
            score = self._score_tree(tree)
            if score[self.method] == 1:
                # solved!
                if verbose:
                    print(f'{text} Solved with {guess}')
                return guess, score, True
            if score < best_score:
                best_score = score
                best_guess = guess
        if verbose:
            print(f'{text} Not solved: Best guess = {best_guess}, score ={best_score}')
        return best_guess, best_score, False
    
    def play(self, game_state):
        # update candidates
        self.candidates = game_state.candidates
        self._update_move_candidates()
        key = tuple(self.candidates)
        if key in Shallow2.CACHED_RESULTS:
            # this has been done before!
            if self.verbose:
                print("we've seen this before ...")
            return Shallow2.CACHED_RESULTS[key]
        # candidates:
        guess, score, solved = self.best_guess(self.candidates, verbose=self.verbose, text='(soln)')
        if solved:
            Shallow2.CACHED_RESULTS[key] = guess
            return guess
        # additional candidates
        guess2, score2, solved = self.best_guess(self.move_candidates, verbose=self.verbose, text='(extra)')
        if solved:
            Shallow2.CACHED_RESULTS[key] = guess2
            return guess2
        Shallow2.CACHED_RESULTS[key] = guess if score <= score2 else guess2
        return Shallow2.CACHED_RESULTS[key]

In [43]:
results = {key: 0 for key in range(0, 7)}
update_rate = 200
method = 1
for seed in range(len(GameState.solutions)):
    verbose = seed % update_rate == 0
    if verbose:
        print(f'Playing game {seed}')
    thisAgent = Shallow2(verbose=False, method=method)
    game = WordleGameUpdate(thisAgent, verbose=False, summary=verbose)
    player_win, turn = game.play()
    if player_win:
        results[turn] += 1
    else:
        results[0] += 1
print('-------------------------', 'Result summary:', '-------------------------', sep='\n')
print(results)
ntrials = sum(results.values())
print(f'average score: {sum(key*val for key,val in results.items())/sum(val for key,val in results.items() if val != 0)}')
print(f'failure rate = {results[0]/ntrials}')

Playing game 0
trace: ⬜ 🟨 🟨 ⬜ 🟨
palsy: ⬜ 🟩 ⬜ ⬜ ⬜
gamer: ⬜ 🟩 ⬜ 🟩 🟩
barfi: 🟩 🟩 🟨 ⬜ ⬜
baker: 🟩 🟩 🟩 🟩 🟩
Result: Player wins in 5 guesses
Playing game 200
trace: 🟨 🟩 ⬜ ⬜ ⬜
bonus: ⬜ ⬜ ⬜ ⬜ ⬜
drift: ⬜ 🟩 🟩 🟨 🟨
fritz: 🟩 🟩 🟩 🟩 🟩
Result: Player wins in 4 guesses
Playing game 400
trace: ⬜ 🟨 ⬜ ⬜ 🟨
dines: ⬜ ⬜ ⬜ 🟩 ⬜
lumpy: ⬜ ⬜ 🟨 ⬜ ⬜
mover: 🟨 ⬜ ⬜ 🟩 🟩
ember: 🟩 🟩 🟩 🟩 🟩
Result: Player wins in 5 guesses
Playing game 600
trace: ⬜ 🟩 ⬜ ⬜ 🟩
ponds: ⬜ ⬜ ⬜ 🟨 ⬜
drive: 🟩 🟩 🟩 🟩 🟩
Result: Player wins in 3 guesses
Playing game 800
trace: ⬜ ⬜ ⬜ 🟨 ⬜
muils: ⬜ ⬜ 🟩 🟨 ⬜
cling: 🟩 🟩 🟩 🟩 ⬜
clink: 🟩 🟩 🟩 🟩 🟩
Result: Player wins in 4 guesses
Playing game 1000
trace: ⬜ 🟩 ⬜ ⬜ 🟨
defer: 🟨 ⬜ ⬜ 🟩 🟩
order: 🟩 🟩 🟩 🟩 🟩
Result: Player wins in 3 guesses
Playing game 1200
trace: 🟨 ⬜ 🟨 ⬜ ⬜
salut: ⬜ 🟩 ⬜ 🟨 🟩
dough: ⬜ ⬜ 🟩 ⬜ ⬜
jaunt: ⬜ 🟩 🟩 🟩 🟩
vaunt: 🟩 🟩 🟩 🟩 🟩
Result: Player wins in 5 guesses
Playing game 1400
trace: ⬜ 🟩 🟨 ⬜ 🟩
arise: 🟩 🟩 ⬜ 🟩 🟩
arose: 🟩 🟩 🟩 🟩 🟩
Result: Player wins in 3 guesses
Playing game 1600
trace: 🟨 ⬜ 🟨 ⬜ ⬜
salut: ⬜ 🟨 ⬜ 🟨 🟨
aun

In [ ]:
class Better(Agent):
    """ Try to explicitly score each guess up to certain depth, then using Shallow from there ... 
    
    score = 'average number of turns to victory' <- minimize that
        * will require scoring all children and taking weighted average
        * may require pruning to be tractable ...
    """
    
    def __init__(self, verbose=True):
        self.move_candidates = GameState.allowed
        self.candidates = GameState.solutions
        self.verbose = verbose
        
    def play(self, game_state):
        # update candidates based on current game state
        self.candidates = game_state.candidates
        self.move_candidates = self._update_move_candidates()
        
        # play move that results in best "score"
        return self.select_best_guess()
    
    def select_best_guess(self):
        move_candidates = self.update_move_candidates()
        for guess in move_candidates:
            score = self.score_guess(guess)
            if score < best_score:
                best_score = score
                best_guess = guess
                
    def score_guess():
        pass

In [47]:
class GuessNode(Agent):
    
    CACHE = {}
    _bad_score = 99999
    _CORRECT = (2,2,2,2,2)
    
    def __init__(self, guess, candidates, parent=None, **tree_kwargs):
        self.guess = guess
        self.candidates = candidates
        self.len = len(self.candidates)
        self.score = None
        self.children = None
        self.parent = parent
        self.tree_kwargs = tree_kwargs
    
    def _gen_tree(self):
        tree = defaultdict(list)
        for word in self.candidates:
            tree[self._score_guess(self.guess, word)].append(word)
        return tree
    
    def score_from_children(self):
        tree = self._gen_tree()
        self.children = [TreeNode(key, vals, parent=self, **self.tree_kwargs) for key, vals in tree.items()]
        # quality checks:
        if len(self.children) == 1 and self.len > 1:
            # this guess got us nowhere and shouldn't be used
            self.score = GuessNode._bad_score
        elif len(self.children) == self.len:
            # result fully specified by evaluation
            self.score = 2 - (GuessNode._CORRECT in tree) / len(tree)
        else:
            # weighted average of children tree scores
            weights = [len(child.candidates) for child in self.children]
            self.score = 1 + sum(w * child.score_tree() for w, child in zip(weights, self.children)) / sum(weights)
            
    
    def score_guess(self):
        cache_key = (self.guess, tuple(self.candidates))
        if cache_key in GuessNode.CACHE:
            return GuessNode.CACHE[cache_key]
        if len(self.candidates) == 1:
            self.score = 0
        elif len(self.candidates) == 2:
            self.score = 0.5
        else:
            self.score_from_children()
            GuessNode.CACHE[cache_key] = self.score
        return self.score


class TreeNode(Agent):
    
    CACHE = {}
    _bad_score = 99999
    _CORRECT = (2,2,2,2,2)
    
    def __init__(self, name, candidates, parent=None, move_candidates=combined_list, pruning_method=None):
        self.name = name
        self.candidates = candidates
        self.parent = parent
        self.score = len(self.candidates)  # worst possible score
        self.best_guess = None
        self.children = []
        self.len = len(self.candidates)
        self.move_candidates = move_candidates
        self._move_pruning_method = pruning_method
        self._update_move_candidates(pruning_method)
        
    def _score_tree(self, tree):
        nbranches = len(tree)
        largest_branch = max(len(val) for val in tree.values())
        return (-nbranches, largest_branch)
        
    def _update_move_candidates(self, pruning_method=None):
        letters = set("".join(self.candidates))
        # ordered so that solution candidates come first
        move_candidates = sorted([word for word in self.move_candidates if set(word) & letters],
                                 key=lambda x: x in self.candidates, reverse=True)
        if pruning_method == 'shallow':
            scores = [self._score_tree(GuessNode(word, self.candidates)._gen_tree()) for word in move_candidates]
            move_candidates = [word for _, word in sorted(zip(scores, move_candidates))]
            nmax = min(len(self.move_candidates), 60)
            move_candidates = move_candidates[:nmax]
        self.move_candidates = move_candidates
        
    def add_child(self, guess):
        self.children.append(GuessNode(guess, candidates=self.candidates, parent=self, move_candidates=self.move_candidates))
        
    def spawn_children(self):
        self._update_move_candidates(self._move_pruning_method)
        self.children = [GuessNode(guess, self.candidates, move_candidates=self.move_candidates) for guess in self.move_candidates]
        
    def select_best_guess(self):
        # easiest cases
        if self.name == TreeNode._CORRECT:
            self.best_guess = self.candidates[0]
            self.score = 0
        elif len(self.candidates) == 1:
            self.best_guess = self.candidates[0]
            self.score = 1
        elif len(self.candidates) == 2:
            self.best_guess = self.candidates[0]
            self.score = 1.5
        else:  # gonna need to loop
            if not self.children:
                self.spawn_children()
            self.score_from_children()
            return (self.best_guess, self.score)
        
    def score_from_children(self, verbose=False):
        # lower bound (best possible score)
        lower_bound = 1 + (self.len - 1) / self.len  # if guess in self.candidates
        update_lower_bound = True
        best_score = TreeNode._bad_score
        if verbose:
            print(f'score_from_children_called for {len(self.children)} candidates')
        for child in self.children:
            score = child.score_guess()
            if update_lower_bound and child.guess not in self.candidates:
                # well ordered so we can move to the new lower bound
                lower_bound = 2   # will require at least 2 tries
                update_lower_bound = False
            if score < self.score:
                self.score = score
                self.best_guess = child.guess
            if self.score < lower_bound + 1e-6:
                # found an unbeatable solution and we can quit
                if verbose:
                    print(f'exiting with perfect soln: {self.best_guess}, {self.score}, {self.len}, {self.name}')
                return
        if verbose:
            print(f'exiting with soln: {self.best_guess}, {self.score}, {self.len}, {self.name}')
        return
    
    def score_tree(self):
        key = tuple(self.candidates)
        if key in TreeNode.CACHE:
            self.best_guess = TreeNode.CACHE[key][0]
            self.score = TreeNode.CACHE[key][1]
        else:
            self.select_best_guess()
            TreeNode.CACHE[key] = (self.best_guess, self.score)
        return self.score

In [80]:
# beginning stencil for Monte Carlo Tree Search 
# in retrospect I don't think this will be as good for a heuristic as 
# Shallow() once depth is exceeded
class MCTSNode():
    def __init__(self, game_state, parent=None, move=None):
        self.game_state = game_state
        self.parent = parent
        self.move = move
        self.win_counts = 0
        self.num_rollouts = 0
        self.children = []
        self.unvisited_moves = game_state.legal_moves()
        
    def add_random_child(self):
        pass
    
    def record_result(self, result):
        pass
    
    def can_add_child(self):
        return len(self.unvisited_moves) > 0
    
    def is_terminal(self):
        return self.game_state.is_over()
    
    def winning_fraction(self):
        return self.win_counts / self.num_rollouts
    
    def winning_score(self):
        pass

In [ ]:
class MCTSAgent(Agent):
    
    def select_move(self, game_state):
        root = MCTSNode(game_state)
        
        for i in range(self.num_rounds):
            node = root
            while (not node.can_add_child()) and (not node.is_terminal()):
                node = self.select_child(node)
            
            if node.can_add_child():
                node.add_random_child()  # add new node into the tree
            
            result = self.simulate_random_game(node.game_state) 
            
            while node is not None:  # propagate score back up the tree
                node.record_result(result)
                node = node.parent
                
        best_move = None
        best_pct = -1
        best_score = 999
        for child in root.children:
            child_pct = child.winning_fraction(game_state)
            if child_pct > best_pct:
                best_pct = child_pct
                best_pct_move = child.move
            child_score = child.winning_score(game_state)
            if child_score < best_score:
                best_score = child_score
                best_score_move = child.move
        return best_score_move

## playing around

In [10]:
WordleGameUpdate(Basic()).play()

⬜ ⬜ ⬜ ⬜ ⬜
Player: 2315 candidate(s) to choose from
ethos: ⬜ ⬜ 🟨 ⬜ ⬜
Player: 63 candidate(s) to choose from
ethos: ⬜ ⬜ 🟨 ⬜ ⬜
chuck: ⬜ 🟨 ⬜ 🟩 ⬜
Player: 7 candidate(s) to choose from
ethos: ⬜ ⬜ 🟨 ⬜ ⬜
chuck: ⬜ 🟨 ⬜ 🟩 ⬜
pinch: ⬜ 🟩 🟩 🟩 🟩
Player: 2 candidate(s) to choose from
ethos: ⬜ ⬜ 🟨 ⬜ ⬜
chuck: ⬜ 🟨 ⬜ 🟩 ⬜
pinch: ⬜ 🟩 🟩 🟩 🟩
winch: 🟩 🟩 🟩 🟩 🟩
Result: Player wins in 4 guesses


(True, 4)

In [11]:
new_game = GameState.new_game()

In [12]:
new_game.word

'belie'

In [13]:
solved_game = new_game.play_guess('arose').play_guess('cater').play_guess('butte').play_guess('gonzo').play_guess('tater').play_guess('farts')
solved_game

arose: ⬜ ⬜ ⬜ ⬜ 🟩
cater: ⬜ ⬜ ⬜ 🟨 ⬜
butte: 🟩 ⬜ ⬜ ⬜ 🟩
gonzo: ⬜ ⬜ ⬜ ⬜ ⬜
tater: ⬜ ⬜ ⬜ 🟨 ⬜
farts: ⬜ ⬜ ⬜ ⬜ ⬜

In [14]:
solved_game.game_result()

(False, 6)

In [15]:
solved_game.candidates

['belie', 'belle', 'bible']

In [43]:
solved_game

arose: 🟨 ⬜ ⬜ 🟩 ⬜
gnash: 🟩 🟩 🟩 🟩 🟩

In [44]:
solved_game.candidates

['gnash']

In [32]:
new_game.play_guess('arose').play_guess('unlit').play_guess('butte')

arose: ⬜ ⬜ ⬜ ⬜ 🟨
unlit: ⬜ ⬜ ⬜ 🟩 🟨
butte: ⬜ ⬜ 🟨 ⬜ 🟨

In [12]:
print(*fonts.values())

⬜ 🟨 🟩


In [54]:
this_game = GameState(None, None)

In [56]:
next_game = this_game.apply_guess('arose')

In [55]:
this_game.word = 'shoot'

In [60]:
next_game.word = 'shoot'

In [61]:
next_game.evaluate_state()

In [64]:
next_game.moves

[['arose']]

In [62]:
next_game.scores

[(0,)]

# Older stuff
Earlier implementation of WordleGame -- I felt like having access to new copies of GameState would be better for some of the bots / evaluating rollouts.

Still may be stuff here worth scavenging.

Note: Agents (`Basic`, `Shallow`, etc) here are not compatible with the implementation expected by `WordleGameUpdate`

In [4]:
class WordleGame:
    wordlist = combined_list
    victory = (2, 2, 2, 2, 2)
    
    def __init__(self, Hero, Villain, max_turns=6, verbose=True, summary=True):
        self.hero = Hero
        self.villain = Villain
        self.turn = 0
        self.max_turns = max_turns
        self.status = None
        self.verbose = verbose
        self.summary = summary
        
    def take_turn(self):
        assert self.turn < self.max_turns
        self.turn += 1
        guess = self.hero.play()
        score = self.villain.evaluate(guess)
        self.hero.evaluate(guess, score)
        return guess, score
    
    def play(self):
        while self.turn < self.max_turns:
            guess, score = self.take_turn()
            if self.verbose:
                print(f'Guess: {guess}')
                print(f'Score: {score}')
            if score == self.victory:
                if self.verbose or self.summary:
                    print(f'Hero wins with {guess} in {self.turn} attempts')
                self.status = 'win'
                return
        self.status = 'loss'
        if self.verbose or self.summary:
            print(f'Hero fails ... Correct word was {self.villain.word}')

In [54]:
agent = Basic()
agent.evaluate('arose', (0,0,2,0,1))
print(agent.candidates)
agent.evaluate('epoch', (2,2,2,0,0))
print(agent.candidates)

['ebony', 'epoch', 'epoxy', 'gooey']
['epoxy']


In [12]:
game = WordleGame(Basic(), Villain(), verbose=True)
game.play()

Hero: There are 2315 to choose from
Guess: lorry
Score: (0, 0, 0, 0, 0)
Hero: There are 490 to choose from
Guess: manic
Score: (0, 2, 0, 2, 0)
Hero: There are 3 to choose from
Guess: vapid
Score: (0, 2, 0, 2, 0)
Hero: There are 2 to choose from
Guess: habit
Score: (0, 2, 1, 2, 0)
Hero: There are 1 to choose from
Guess: basis
Score: (2, 2, 2, 2, 2)
Hero wins with basis in 5 attempts


In [51]:
agent.candidates

['ebony', 'epoch', 'epoxy', 'gooey']

In [6]:
class Villain:
    candidates = soln_list

    def __init__(self, seed=None):
        self.word = self.candidates[seed % len(self.candidates)] if seed is not None else random.choice(self.candidates)
    
    def evaluate(self, guess):
        """{'miss': 0, 'contains': 1, 'correct': 2}"""
        correct = [2*(g==t) for g,t in zip(guess, self.word)]  # cast as int
        remains = [t for t,c in zip(self.word, correct) if not c]
        for i, (g,c) in enumerate(zip(guess, correct)):
            if not c and g in remains:
                correct[i] = 1
                remains.remove(g)
        return tuple(correct)

In [11]:
class Basic(Agent):
    """Randomly guesses based on current set of possible solutions"""
    
    def __init__(self, verbose=True):
        self.candidates = soln_list
        self.verbose = verbose
    
    def play(self):
        if self.verbose:
            print(f'Hero: There are {len(self.candidates)} to choose from')
        return random.choice(self.candidates)
    
    def evaluate(self, guess, score):
        """{'miss': 0, 'contains': 1, 'correct': 2}"""
        self.candidates = [x for x in self.candidates if self._score_guess(guess, x) == score]        

In [8]:
def unique_letters_in_list(list_of_words):
    return set("".join(list_of_words))

In [9]:
def words_containing_these_letters(letters, word_list=allowed_list, excluded=None):
    if excluded:
        word_list = [word for word in word_list if not set(word) & set(excluded)]
    return [word for word in word_list if set(word) & letters]

In [38]:
class Advanced(Agent):
    """Strategy: explicitly calculate 'score' as expected depth (how many more moves expect to win)"""
    CACHE = {}
    
    def __init__(self, **tree_kwargs):
        self.candidates = soln_list
        self.move_candidates = soln_list
        self.root = TreeNode('root', candidates=soln_list, parent=None,
                             move_candidates=self.move_candidates, **tree_kwargs)
    
    def score_guess(self, guess):
        self.root.add_child(guess)
        self.root.children[-1].score_guess()

In [47]:
class GuessNode(Agent):
    
    CACHE = {}
    _bad_score = 99999
    _CORRECT = (2,2,2,2,2)
    
    def __init__(self, guess, candidates, parent=None, **tree_kwargs):
        self.guess = guess
        self.candidates = candidates
        self.len = len(self.candidates)
        self.score = None
        self.children = None
        self.parent = parent
        self.tree_kwargs = tree_kwargs
    
    def _gen_tree(self):
        tree = defaultdict(list)
        for word in self.candidates:
            tree[self._score_guess(self.guess, word)].append(word)
        return tree
    
    def score_from_children(self):
        tree = self._gen_tree()
        self.children = [TreeNode(key, vals, parent=self, **self.tree_kwargs) for key, vals in tree.items()]
        # quality checks:
        if len(self.children) == 1 and self.len > 1:
            # this guess got us nowhere and shouldn't be used
            self.score = GuessNode._bad_score
        elif len(self.children) == self.len:
            # result fully specified by evaluation
            self.score = 2 - (GuessNode._CORRECT in tree) / len(tree)
        else:
            # weighted average of children tree scores
            weights = [len(child.candidates) for child in self.children]
            self.score = 1 + sum(w * child.score_tree() for w, child in zip(weights, self.children)) / sum(weights)
            
    
    def score_guess(self):
        cache_key = (self.guess, tuple(self.candidates))
        if cache_key in GuessNode.CACHE:
            return GuessNode.CACHE[cache_key]
        if len(self.candidates) == 1:
            self.score = 0
        elif len(self.candidates) == 2:
            self.score = 0.5
        else:
            self.score_from_children()
            GuessNode.CACHE[cache_key] = self.score
        return self.score


class TreeNode(Agent):
    
    CACHE = {}
    _bad_score = 99999
    _CORRECT = (2,2,2,2,2)
    
    def __init__(self, name, candidates, parent=None, move_candidates=combined_list, pruning_method=None):
        self.name = name
        self.candidates = candidates
        self.parent = parent
        self.score = len(self.candidates)  # worst possible score
        self.best_guess = None
        self.children = []
        self.len = len(self.candidates)
        self.move_candidates = move_candidates
        self._move_pruning_method = pruning_method
        self._update_move_candidates(pruning_method)
        
    def _score_tree(self, tree):
        nbranches = len(tree)
        largest_branch = max(len(val) for val in tree.values())
        return (-nbranches, largest_branch)
        
    def _update_move_candidates(self, pruning_method=None):
        letters = set("".join(self.candidates))
        # ordered so that solution candidates come first
        move_candidates = sorted([word for word in self.move_candidates if set(word) & letters],
                                 key=lambda x: x in self.candidates, reverse=True)
        if pruning_method == 'shallow':
            scores = [self._score_tree(GuessNode(word, self.candidates)._gen_tree()) for word in move_candidates]
            move_candidates = [word for _, word in sorted(zip(scores, move_candidates))]
            nmax = min(len(self.move_candidates), 60)
            move_candidates = move_candidates[:nmax]
        self.move_candidates = move_candidates
        
    def add_child(self, guess):
        self.children.append(GuessNode(guess, candidates=self.candidates, parent=self, move_candidates=self.move_candidates))
        
    def spawn_children(self):
        self._update_move_candidates(self._move_pruning_method)
        self.children = [GuessNode(guess, self.candidates, move_candidates=self.move_candidates) for guess in self.move_candidates]
        
    def select_best_guess(self):
        # easiest cases
        if self.name == TreeNode._CORRECT:
            self.best_guess = self.candidates[0]
            self.score = 0
        elif len(self.candidates) == 1:
            self.best_guess = self.candidates[0]
            self.score = 1
        elif len(self.candidates) == 2:
            self.best_guess = self.candidates[0]
            self.score = 1.5
        else:  # gonna need to loop
            if not self.children:
                self.spawn_children()
            self.score_from_children()
            return (self.best_guess, self.score)
        
    def score_from_children(self, verbose=False):
        # lower bound (best possible score)
        lower_bound = 1 + (self.len - 1) / self.len  # if guess in self.candidates
        update_lower_bound = True
        best_score = TreeNode._bad_score
        if verbose:
            print(f'score_from_children_called for {len(self.children)} candidates')
        for child in self.children:
            score = child.score_guess()
            if update_lower_bound and child.guess not in self.candidates:
                # well ordered so we can move to the new lower bound
                lower_bound = 2   # will require at least 2 tries
                update_lower_bound = False
            if score < self.score:
                self.score = score
                self.best_guess = child.guess
            if self.score < lower_bound + 1e-6:
                # found an unbeatable solution and we can quit
                if verbose:
                    print(f'exiting with perfect soln: {self.best_guess}, {self.score}, {self.len}, {self.name}')
                return
        if verbose:
            print(f'exiting with soln: {self.best_guess}, {self.score}, {self.len}, {self.name}')
        return
    
    def score_tree(self):
        key = tuple(self.candidates)
        if key in TreeNode.CACHE:
            self.best_guess = TreeNode.CACHE[key][0]
            self.score = TreeNode.CACHE[key][1]
        else:
            self.select_best_guess()
            TreeNode.CACHE[key] = (self.best_guess, self.score)
        return self.score

In [48]:
z = Advanced(pruning_method='shallow')

In [ ]:
z.score_guess('arose')

In [35]:
len(z.root.children[0].children)

121

In [46]:
for child in z.root.children[0].children:
    print(child.name, child.score, child.best_guess, child.len)

(2, 0, 0, 0, 0) 2.5 train 30
(2, 0, 0, 2, 2) 2.0 blare 3
(2, 0, 0, 0, 2) 2.833333333333333 plane 18
(2, 0, 0, 0, 1) 2.090909090909091 plane 11
(2, 0, 1, 0, 0) 2.333333333333333 slant 9
(2, 1, 1, 0, 0) 2.0 cater 4
(2, 0, 2, 0, 2) 2.428571428571429 diner 7
(2, 1, 2, 0, 0) 2.25 trace 4
(2, 0, 2, 0, 0) 2.416666666666667 dealt 12
(2, 0, 0, 2, 0) 2.2 heart 5
(2, 1, 0, 0, 0) 2.142857142857143 train 7
(2, 1, 2, 0, 2) 1 adore 1
(2, 1, 0, 0, 2) 2.25 heart 4
(2, 1, 0, 0, 1) 2.0 alien 7
(2, 0, 0, 1, 2) 1.5 aisle 2
(2, 2, 1, 0, 0) 2.25 diner 4
(2, 2, 0, 0, 1) 1 arena 1
(2, 2, 0, 0, 2) 1 argue 1
(2, 2, 0, 2, 2) 1 arise 1
(2, 2, 2, 0, 0) 1 aroma 1
(2, 2, 2, 2, 2) 0 arose 1
(2, 2, 0, 0, 0) 1 array 1
(2, 2, 1, 1, 0) 1 arson 1
(2, 2, 0, 2, 0) 1 artsy 1
(2, 0, 1, 1, 0) 1 ascot 1
(2, 0, 0, 1, 1) 2.0 heart 3
(2, 0, 0, 1, 0) 1 assay 1
(1, 0, 1, 0, 0) 3.022727272727273 cleat 44
(1, 0, 0, 0, 2) 3.38 dealt 50
(1, 0, 0, 0, 0) 148 None 148
(1, 0, 0, 0, 1) 78 None 78
(1, 1, 0, 0, 1) 60 None 60
(1, 1, 0, 0, 2) 8 N

In [33]:
for child in z.root.children[0].children:
    print(child.name, child.score, child.best_guess, child.len)

(2, 0, 0, 0, 0) 2.2666666666666666 plait 30
(2, 0, 0, 2, 2) 2.0 blare 3
(2, 0, 0, 0, 2) 2.666666666666667 plait 18
(2, 0, 0, 0, 1) 2.090909090909091 plane 11
(2, 0, 1, 0, 0) 2.2222222222222223 plait 9
(2, 1, 1, 0, 0) 2.0 cater 4
(2, 0, 2, 0, 2) 2.428571428571429 diner 7
(2, 1, 2, 0, 0) 2.25 trace 4
(2, 0, 2, 0, 0) 2.333333333333333 filet 12
(2, 0, 0, 2, 0) 2.2 heart 5
(2, 1, 0, 0, 0) 2.142857142857143 train 7
(2, 1, 2, 0, 2) 1 adore 1
(2, 1, 0, 0, 2) 2.25 heart 4
(2, 1, 0, 0, 1) 2.0 alien 7
(2, 0, 0, 1, 2) 1.5 aisle 2
(2, 2, 1, 0, 0) 2.25 diner 4
(2, 2, 0, 0, 1) 1 arena 1
(2, 2, 0, 0, 2) 1 argue 1
(2, 2, 0, 2, 2) 1 arise 1
(2, 2, 2, 0, 0) 1 aroma 1
(2, 2, 2, 2, 2) 0 arose 1
(2, 2, 0, 0, 0) 1 array 1
(2, 2, 1, 1, 0) 1 arson 1
(2, 2, 0, 2, 0) 1 artsy 1
(2, 0, 1, 1, 0) 1 ascot 1
(2, 0, 0, 1, 1) 2.0 heart 3
(2, 0, 0, 1, 0) 1 assay 1
(1, 0, 1, 0, 0) 2.840909090909091 lance 44
(1, 0, 0, 0, 2) 3.18 dealt 50
(1, 0, 0, 0, 0) 3.358108108108108 plait 148
(1, 0, 0, 0, 1) 78 None 78
(1, 1, 0, 0, 1)

In [32]:
for child in z.root.children[0].children:
    print(child.name, child.score, child.best_guess, child.len)

(2, 0, 0, 0, 0) 2.3 align 30
(2, 0, 0, 2, 2) 3 None 3
(2, 0, 0, 0, 2) 18 None 18
(2, 0, 0, 0, 1) 11 None 11
(2, 0, 1, 0, 0) 9 None 9
(2, 1, 1, 0, 0) 4 None 4
(2, 0, 2, 0, 2) 7 None 7
(2, 1, 2, 0, 0) 4 None 4
(2, 0, 2, 0, 0) 12 None 12
(2, 0, 0, 2, 0) 5 None 5
(2, 1, 0, 0, 0) 7 None 7
(2, 1, 2, 0, 2) 1 None 1
(2, 1, 0, 0, 2) 4 None 4
(2, 1, 0, 0, 1) 7 None 7
(2, 0, 0, 1, 2) 2 None 2
(2, 2, 1, 0, 0) 4 None 4
(2, 2, 0, 0, 1) 1 None 1
(2, 2, 0, 0, 2) 1 None 1
(2, 2, 0, 2, 2) 1 None 1
(2, 2, 2, 0, 0) 1 None 1
(2, 2, 2, 2, 2) 1 None 1
(2, 2, 0, 0, 0) 1 None 1
(2, 2, 1, 1, 0) 1 None 1
(2, 2, 0, 2, 0) 1 None 1
(2, 0, 1, 1, 0) 1 None 1
(2, 0, 0, 1, 1) 3 None 3
(2, 0, 0, 1, 0) 1 None 1
(1, 0, 1, 0, 0) 44 None 44
(1, 0, 0, 0, 2) 50 None 50
(1, 0, 0, 0, 0) 148 None 148
(1, 0, 0, 0, 1) 78 None 78
(1, 1, 0, 0, 1) 60 None 60
(1, 1, 0, 0, 2) 8 None 8
(1, 1, 1, 0, 0) 28 None 28
(1, 0, 0, 1, 0) 67 None 67
(1, 0, 0, 1, 2) 29 None 29
(1, 0, 0, 2, 1) 5 None 5
(0, 0, 0, 0, 1) 120 None 120
(0, 0, 0, 0, 2) 79

In [102]:
class Shallow(Agent):
    """Strategy: choose the guess that gives the widest / most evenly distributed tree against the field """
    
    CACHED_RESULTS = {tuple(soln_list): 'raise'}
    
    def __init__(self, verbose=True, method=0):
        self.candidates = soln_list
        self.move_candidates = allowed_list
        self.first_guess = ('raise', (168, -132), False)  # cached because it's deterministic
        self.verbose = verbose
        self.method = method
        
    def _test_guess_vs_candidates(self, guess):
        tree = defaultdict(list)
        for word in self.candidates:
            tree[self._score_guess(guess, word)].append(word)
        return tree
    
    def _score_tree(self, tree):
        nbranches = len(tree)
        largest_branch = max(len(val) for val in tree.values())
        if method == 0:
            return (largest_branch, -nbranches)
        elif method == 1:
            return (-nbranches, largest_branch)
        else:
            raise ValueError('method needs to be 0 or 1')
    
    def _update_move_candidates(self):
        letters = set("".join(self.candidates))
        self.move_candidates = [word for word in self.move_candidates if set(word) & letters]
    
    def best_guess(self, guess_candidates, verbose=True, text=''):
        if len(self.candidates) == len(soln_list):
            # first guess
            return self.first_guess
        best_score = (len(self.candidates), 0) if self.method == 0 else (0, len(self.candidates))
        best_guess = None
        for guess in guess_candidates:
            tree = self._test_guess_vs_candidates(guess)
            score = self._score_tree(tree)
            if score[0] == 1:
                # solved!
                if verbose:
                    print(f'{text} Solved with {guess}')
                return guess, score, True
            if score < best_score:
                best_score = score
                best_guess = guess
        if verbose:
            print(f'{text} Not solved: Best guess = {best_guess}, score ={best_score}')
        return best_guess, best_score, False
    
    def play(self):
        key = tuple(self.candidates)
        if key in Shallow.CACHED_RESULTS:
            # this has been done before!
            if self.verbose:
                print("we've seen this before ...")
            return Shallow.CACHED_RESULTS[key]
        # candidates:
        guess, score, solved = self.best_guess(self.candidates, verbose=self.verbose, text='(soln)')
        if solved:
            Shallow.CACHED_RESULTS[key] = guess
            return guess
        # additional candidates
        guess2, score2, solved = self.best_guess(self.move_candidates, verbose=self.verbose, text='(extra)')
        if solved:
            Shallow.CACHED_RESULTS[key] = guess2
            return guess2
        Shallow.CACHED_RESULTS[key] = guess if score <= score2 else guess2
        return Shallow.CACHED_RESULTS[key]
    
    def evaluate(self, guess, score):
        self.candidates = [x for x in self.candidates if self._score_guess(guess, x) == score]
        self._update_move_candidates()

In [49]:
print(results)
ntrials = sum(results.values())
print(sum(key*val for key,val in results.items())/sum(val for key,val in results.items() if val != 0))
print(f'failure rate = {results[0]/ntrials}')

{0: 91, 1: 1, 2: 187, 3: 1244, 4: 2010, 5: 1126, 6: 341}
3.9646
failure rate = 0.0182


In [47]:
results = {key: 0 for key in range(0, 7)}
for _ in range(5000):
    Game = WordleGame(Basic(), Villain(), verbose=False, summary=False)
    Game.play()
    if Game.status == "win":
        results[Game.turn] += 1
    else:
        results[0] += 1
print(results)
ntrials = sum(results.values())
print(sum(key*val for key,val in results.items())/sum(val for key,val in results.items() if val != 0))
print(f'failure rate = {results[0]/ntrials}')

Guess: spine
Score: (2, 0, 0, 0, 0)
Guess: smoky
Score: (2, 0, 1, 0, 0)
Guess: salvo
Score: (2, 0, 0, 0, 1)
Guess: south
Score: (2, 2, 2, 2, 2)
Guess: bagel
Score: (0, 0, 0, 0, 0)
Guess: tipsy
Score: (2, 0, 1, 0, 0)
Guess: troop
Score: (2, 2, 2, 2, 2)
Guess: refer
Score: (0, 0, 0, 0, 0)
Guess: minty
Score: (1, 0, 1, 0, 0)
Guess: among
Score: (1, 1, 1, 1, 0)
Guess: nomad
Score: (1, 2, 2, 2, 0)
Guess: woman
Score: (2, 2, 2, 2, 2)
Guess: grief
Score: (0, 0, 0, 1, 0)
Guess: wheat
Score: (0, 0, 1, 1, 0)
Guess: slave
Score: (2, 0, 1, 0, 2)
Guess: sauce
Score: (2, 2, 2, 2, 2)
Guess: study
Score: (2, 0, 0, 0, 2)
Guess: sorry
Score: (2, 2, 2, 2, 2)
Guess: handy
Score: (1, 0, 0, 0, 0)
Guess: botch
Score: (0, 0, 1, 0, 2)
Guess: smith
Score: (0, 0, 2, 1, 2)
Guess: thigh
Score: (2, 2, 2, 2, 2)
Guess: egret
Score: (0, 1, 0, 0, 0)
Guess: gulch
Score: (1, 1, 1, 1, 0)
Guess: clung
Score: (2, 2, 2, 2, 2)
Guess: hardy
Score: (0, 0, 0, 0, 0)
Guess: tepee
Score: (0, 1, 0, 2, 2)
Guess: emcee
Score: (2, 2, 2

In [90]:
len(Shallow.CACHED_RESULTS)

1793

In [63]:
results

{0: 0, 1: 0, 2: 0, 3: 1, 4: 1, 5: 0, 6: 0}

In [103]:
results = {key: 0 for key in range(0, 7)}
update_rate = 100
method = 1
Shallow.CACHED_RESULTS = {}
for seed in range(len(Villain.candidates)):
    verbose = seed % update_rate == 0
    if verbose:
        print(f'Playing game {seed}')
    thisAgent = Shallow(verbose=False, method=method)
    thisVillain = Villain(seed)
    Game = WordleGame(thisAgent, thisVillain, verbose=False, summary=verbose)
    Game.play()
    if Game.status == "win":
        results[Game.turn] += 1
    else:
        results[0] += 1

print('-------------------------', 'Result summary:', '-------------------------', sep='\n')
print(results)
ntrials = sum(results.values())
print(f'average score: {sum(key*val for key,val in results.items())/sum(val for key,val in results.items() if val != 0)}')
print(f'failure rate = {results[0]/ntrials}')

Playing game 0
Hero wins with aback in 4 attempts
Playing game 100
Hero wins with aptly in 3 attempts
Playing game 200
Hero wins with birch in 3 attempts
Playing game 300
Hero wins with bulky in 4 attempts
Playing game 400
Hero wins with clash in 3 attempts
Playing game 500
Hero wins with cubic in 4 attempts
Playing game 600
Hero wins with drone in 3 attempts
Playing game 700
Hero wins with false in 3 attempts
Playing game 800
Hero wins with frail in 3 attempts
Playing game 900
Hero wins with grasp in 3 attempts
Playing game 1000
Hero wins with human in 3 attempts
Playing game 1100
Hero wins with lasso in 3 attempts
Playing game 1200
Hero wins with march in 3 attempts
Playing game 1300
Hero wins with niche in 3 attempts
Playing game 1400
Hero wins with pesto in 3 attempts
Playing game 1500
Hero wins with purse in 4 attempts
Playing game 1600
Hero wins with rival in 2 attempts
Playing game 1700
Hero wins with shady in 3 attempts
Playing game 1800
Hero wins with smack in 4 attempts
Playi

In [91]:
results = {key: 0 for key in range(0, 7)}
update_rate = 100
for seed in range(len(Villain.candidates)):
    verbose = seed % update_rate == 0
    if verbose:
        print(f'Playing game {seed}')
    thisAgent = Shallow(verbose=False)
    thisVillain = Villain(seed)
    Game = WordleGame(thisAgent, thisVillain, verbose=False, summary=verbose)
    Game.play()
    if Game.status == "win":
        results[Game.turn] += 1
    else:
        results[0] += 1

print('-------------------------', 'Result summary:', '-------------------------', sep='\n')
print(results)
ntrials = sum(results.values())
print(f'average score: {sum(key*val for key,val in results.items())/sum(val for key,val in results.items() if val != 0)}')
print(f'failure rate = {results[0]/ntrials}')

Playing game 0
Hero wins with aback in 4 attempts
Playing game 100
Hero wins with aptly in 4 attempts
Playing game 200
Hero wins with birch in 3 attempts
Playing game 300
Hero wins with bulky in 3 attempts
Playing game 400
Hero wins with clash in 3 attempts
Playing game 500
Hero wins with cubic in 3 attempts
Playing game 600
Hero wins with drone in 4 attempts
Playing game 700
Hero wins with false in 3 attempts
Playing game 800
Hero wins with frail in 3 attempts
Playing game 900
Hero wins with grasp in 3 attempts
Playing game 1000
Hero wins with human in 3 attempts
Playing game 1100
Hero wins with lasso in 3 attempts
Playing game 1200
Hero wins with march in 3 attempts
Playing game 1300
Hero wins with niche in 3 attempts
Playing game 1400
Hero wins with pesto in 3 attempts
Playing game 1500
Hero wins with purse in 4 attempts
Playing game 1600
Hero wins with rival in 2 attempts
Playing game 1700
Hero wins with shady in 3 attempts
Playing game 1800
Hero wins with smack in 4 attempts
Playi

In [104]:
len(Shallow.CACHED_RESULTS)

2493

In [88]:
len(Shallow.CACHED_RESULTS)

602

In [26]:
Game.play()

Guess: sinew
Score: (1, 0, 0, 1, 0)
Guess: pesto
Score: (0, 1, 1, 1, 0)
Guess: quest
Score: (0, 0, 2, 2, 2)
Guess: chest
Score: (2, 2, 2, 2, 2)
Hero wins: guess in 4 attempts


In [28]:
Adversary = Villain()
print(Adversary.word)
print(Adversary.evaluate('arose'))

abase
(2, 0, 0, 2, 2)


In [14]:
Adversary = Villain()
Adversary.word

'slate'

In [340]:
def score_guess(guess, truth):
    """{'miss': 0, 'contains': 1, 'correct': 2}"""
    
    correct = [2*(g==t) for g,t in zip(guess, truth)]  # cast as int
    remains = [t for t,c in zip(truth, correct) if not c]
    # too simplistic ...
    #contains = [(g in remains and not c) for g,c in zip(guess, correct)]
    #return tuple(2*corr + cont for corr,cont in zip(correct, contains))
    for i, (g,c) in enumerate(zip(guess, correct)):
        if not c and g in remains:
            correct[i] = 1
            remains.remove(g)
    return tuple(correct)

In [ ]:
def test_guess_vs_candidates(guess, candidates):
    out = defaultdict(list)
    for word in candidates:
        out[tuple(score_guess(guess, word))].append(word)
    return out

In [244]:
def unique_letters_in_list(list_of_words):
    return set("".join(list_of_words))

In [399]:
def words_containing_these_letters(letters, word_list=total_list, excluded=None):
    if excluded:
        word_list = [word for word in word_list if not set(word) & set(excluded)]
    return [word for word in word_list if set(word) & letters]

In [460]:
isclose(1 + 2/3, 5/3+0.000000000001)

True

In [598]:
# try to make a fancy tree
class Node():
    def __init__(self, name, children=None, parent=None, guess=None,
                 solution_candidates=None):
        self.name = name
        self.children = children if children is not None else []
        self.parent = parent if parent is not None else []
        self.score = None
        self.type = 'guess' if isinstance(self.name, str) else 'tree'
        self.guess = guess
        if self.guess is None:
            self.guess = self.name if self.type == 'guess' else None
        self.incoming_score = self.name if self.type == 'tree' else None
        if solution_candidates is None:
            self.solution_candidates = soln_list
        else:
            self.solution_candidates = [x for x in solution_candidates]
        self.next_guesses = []
        self.nsol = len(self.solution_candidates)
        
    def __repr__(self):
        return f'Node({self.name}, {self.guess}, {self.score}, {self.nsol})'
        
    def allowed_guesses(self, excluded=None, word_list=total_list):
        return words_containing_these_letters(unique_letters_in_list(self.solution_candidates),
                                              word_list=word_list, excluded=excluded)
        
    def spawn_children(self, branch_score=True):
        tree = defaultdict(list)
        for word in self.solution_candidates:
            tree[score_guess(self.guess, word)].append(word)
        if branch_score:
            largest_branch = max(len(v) for v in tree.values())
            nbranches = len(tree)
            self.branch_score = (largest_branch, -nbranches)
        for key, val in tree.items():
            self.add_child(Node(key, parent=self, solution_candidates=val))
    
    def expand_tree(self, **kwargs):
        if self.type != 'tree':
            return
        self.next_guesses = self.allowed_guesses(**kwargs)
        for guess in self.next_guesses:
            self.add_child(Node(guess, parent=self, solution_candidates=self.solution_candidates))
    
    def add_child(self, child):
        self.children.append(child)
        
    def choose_best_guess(self, depth=2, **kwargs):
        """Best guess has lowest score"""
        if self.type != 'tree':
            return
        # sorted in order of highest likelihood
        self.next_guesses = sorted(self.allowed_guesses(**kwargs),
                                   key=lambda x: x in self.solution_candidates, reverse=True)
        lower_bound = 1 + (self.nsol - 1) / self.nsol + 1e-6
        best = 10 * nsol
        check_lower = True
        for guess in self.next_guesses:
            if check_lower and guess not in soln_list:  # ordered well
                check_lower = False
                lower_bound = 2.000001
                if best < lower_bound:
                    return best_child
            child = Node(guess, parent=self, solution_candidates=self.solution_candidates)
            child.calc_score(depth=depth)
            if child.score < lower_bound:
                # stop here! can't do better
                return child
            elif child.score < best:
                print(best, child.name)
                best_child = Node(guess, parent=self, solution_candidates=self.solution_candidates)
                best = child.score
        return best_child

    def calc_score(self, depth=2):
        if self.score is not None:
            return self.score
        if len(self.solution_candidates) == 1:
            # 0 if this was the solution, 1 if it'll be the next guess
            self.score = int(self.name != (2, 2, 2, 2, 2))
        elif len(self.solution_candidates) == 2:
            # can't do better than guess 1, then the other
            # 50% you get it next time (score = 1), 50% it takes 2 
            self.score = 1.5
        elif self.type == 'guess':
            # spawn children and calculate from them:
            self.spawn_children()
            self.score = self.calc_score_from_children(depth=depth-1)
        elif self.type == 'tree':
            best_guess = self.choose_best_guess(depth=depth)
            # update self to be 'guess'
            self.name = best_guess.name
            self.type = best_guess.type
            self.guess = best_guess.guess
            self.calc_score(depth=depth)
        return self.score
    
    def upper_bound(self):
        if len(self.solution_candidates) < 3:
            self.calc_score(depth=1)
            return self.score
        else:  # assume the worst
            self.flag = 'upper'
            return len(self.solution_candidates)
    
    def calc_score_from_children(self, depth=-1):
        if depth == 0:
            self.flag = 'depth'
            return 1 + sum(child.upper_bound() for child in self.children) / len(self.children)
            # we're done ... return upper bound with flag saying estimate
        else:
            return 1 + sum(child.calc_score(depth=depth) for child in self.children) / len(self.children)
        
    def calculate_score(self, recalculate=False, calculate_from_children=False):
        if self.score is not None and not recalculate:
            return self.score
        if len(self.solution_candidates) == 1:
            # 0 if this was the solution, 1 if it'll be the next guess
            self.score = int(self.name != (2, 2, 2, 2, 2))
        elif len(self.solution_candidates) == 2:
            # can't do better than guess 1, then the other
            # 50% you get it next time (score = 1), 50% it takes 2 
            self.score = 1.5
        elif calculate_from_children:
            self.score = 1 + sum(child.calculate_score() for child in self.children) / len(self.children)
        else:
            self.score = None
        return self.score

In [599]:
TopNode = Node('arose')
TopNode.spawn_children()

In [603]:
# Next up: need to find a good way to select the "best" guess ...
TestNode = TopNode.children[2]
TestNode

Node((2, 0, 0, 0, 2), None, None, 18)

In [601]:
TestNode

Node((2, 0, 0, 2, 2), None, None, 3)

In [596]:
TestNode = TopNode.children[0]
TestNode

Node((2, 0, 0, 0, 0), None, None, 30)

In [561]:
TopNode.children[0]

Node((2, 0, 0, 0, 0), None, None, 30)

In [523]:
TestNode.solution_candidates

['abase', 'abuse', 'amuse']

In [500]:
TestNode.name

(2, 0, 0, 2, 2)

In [501]:
TestNode.choose_best_guess()

Node(abase, abase, 1.6666666666666665, 3)

In [487]:
TestNode

Node((2, 0, 0, 2, 2), None, None, 3)

In [444]:
z = TestNode.allowed_guesses()
len(z)

12164

In [284]:
tmps = dict()
for guess in TestNode.solution_candidates:
    tmps[guess] = Node(guess, guess=guess, solution_candidates=TestNode.solution_candidates)
    tmps[guess].spawn_children()
    if max(x.nsol for x in tmps[guess].children) == 1: # can't do any better
        print(f'score for {guess} = {tmps[guess].calculate_score(calculate_from_children=True)}')

score for abase = 1.6666666666666665
score for abuse = 1.6666666666666665
score for amuse = 1.6666666666666665


In [277]:
tmps['amuse'].children

[Node((2, 0, 0, 2, 2), None, None, 1),
 Node((2, 0, 2, 2, 2), None, None, 1),
 Node((2, 2, 2, 2, 2), None, None, 1)]

In [269]:
TestNode.guess

In [263]:
TopNode.children[1].solution_candidates

['abase', 'abuse', 'amuse']

In [355]:
TopNode = Node('arose', guess='arose')

In [356]:
TopNode.spawn_children()

In [ ]:
class Tree:
    def __init__(self, root):
        self.root = self.add_node(root, parent=None)
        self.nodes = {self.root.name: (self.root.parent, self.root.children)}
    
    def add_node(self, node, parent=None):
        

In [51]:
guess = 'arose'
nodes = product([0, 1, 2], repeat=5)
top_level_tree = {key: [] for key in nodes}

In [64]:
first_guess = {guess: -1 for guess in allowed_list}
for guess in allowed_list:
    top_level_tree = {key: [] for key in product([0,1,2], repeat=5)}
    for word in soln_list:
        score = tuple(score_guess(guess, word))
        top_level_tree[score].append(word)
    top_tree_counts = {key: len(val) for key, val in top_level_tree.items()}
    first_guess[guess] = max(top_tree_counts.values())

In [69]:
{key: val for key, val in first_guess.items() if val < 185}

{'aeros': 183,
 'aesir': 168,
 'aiery': 171,
 'aloes': 174,
 'anole': 182,
 'ariel': 173,
 'ayrie': 171,
 'raile': 173,
 'reais': 168,
 'realo': 176,
 'serai': 168,
 'soare': 183,
 'stoae': 177}

In [285]:
TopNode = Node('aesir', guess='aesir')

In [286]:
TopNode.spawn_children()

In [291]:
for child in TopNode.children:
    if child.name == (0, 1, 0, 0, 1):
        TestNode = child
        print(*child.solution_candidates, sep='\n')

borne
breed
broke
brute
chore
clerk
credo
creed
creek
creep
creme
crepe
crept
crone
crude
cruel
curve
drone
drove
egret
entry
erect
erode
erupt
every
exert
force
forge
forte
freed
froze
gorge
greed
green
greet
grope
grove
gruel
horde
ombre
overt
preen
probe
prone
prove
prude
prune
puree
purge
query
rhyme
rodeo
rogue
rouge
route
rupee
there
three
threw
trend
trope
trove
truce
where
wreck
wrote


In [304]:
nmax = TestNode.nsol
best_guess = None
for guess in ['brute']:
    ThisNode = Node(guess, guess=guess, solution_candidates=TestNode.solution_candidates)
    ThisNode.spawn_children()
    nsol = max(x.nsol for x in ThisNode.children)
    if nsol < nmax:
        print(f'{guess}: {nsol}')
        nmax = nsol
        best_guess = guess
        bestNode = ThisNode
        print(*bestNode.children, sep='\n')

brute: 11
Node((2, 1, 0, 0, 2), None, None, 1)
Node((2, 2, 0, 0, 1), None, None, 1)
Node((2, 2, 0, 0, 2), None, None, 1)
Node((2, 2, 2, 2, 2), None, None, 1)
Node((0, 1, 0, 0, 2), None, None, 7)
Node((0, 1, 0, 0, 1), None, None, 3)
Node((0, 2, 0, 0, 1), None, None, 9)
Node((0, 2, 0, 0, 2), None, None, 11)
Node((0, 2, 0, 1, 1), None, None, 4)
Node((0, 2, 2, 0, 2), None, None, 3)
Node((0, 2, 2, 0, 1), None, None, 2)
Node((0, 1, 1, 0, 2), None, None, 5)
Node((0, 1, 0, 1, 1), None, None, 5)
Node((0, 2, 2, 1, 1), None, None, 1)
Node((0, 1, 0, 2, 2), None, None, 1)
Node((1, 1, 0, 0, 2), None, None, 1)
Node((1, 2, 0, 0, 2), None, None, 1)
Node((0, 1, 1, 0, 1), None, None, 1)
Node((0, 1, 2, 0, 2), None, None, 1)
Node((0, 1, 2, 2, 2), None, None, 1)
Node((0, 1, 0, 1, 2), None, None, 2)
Node((0, 2, 0, 1, 2), None, None, 2)
Node((0, 2, 2, 1, 2), None, None, 1)
Node((0, 2, 0, 2, 2), None, None, 1)


In [307]:
bestNode.children[11].solution_candidates

['curve', 'puree', 'purge', 'rogue', 'rupee']

In [300]:
bestNode.children

[Node((0, 2, 0, 1, 0), None, None, 6),
 Node((0, 0, 0, 2, 0), None, None, 8),
 Node((0, 1, 0, 1, 0), None, None, 8),
 Node((0, 0, 0, 1, 1), None, None, 4),
 Node((2, 1, 0, 1, 0), None, None, 3),
 Node((2, 0, 0, 1, 0), None, None, 4),
 Node((2, 0, 0, 2, 0), None, None, 4),
 Node((2, 0, 0, 1, 2), None, None, 1),
 Node((2, 0, 1, 1, 0), None, None, 1),
 Node((0, 1, 1, 1, 0), None, None, 3),
 Node((0, 0, 0, 2, 2), None, None, 2),
 Node((1, 0, 0, 1, 2), None, None, 1),
 Node((0, 0, 0, 1, 2), None, None, 2),
 Node((0, 0, 1, 1, 0), None, None, 1),
 Node((1, 2, 0, 1, 0), None, None, 1),
 Node((0, 2, 0, 1, 1), None, None, 2),
 Node((0, 1, 1, 1, 2), None, None, 1),
 Node((0, 0, 0, 1, 0), None, None, 6),
 Node((0, 2, 0, 2, 0), None, None, 1),
 Node((0, 0, 0, 2, 1), None, None, 2),
 Node((0, 1, 0, 1, 1), None, None, 2),
 Node((0, 1, 1, 1, 1), None, None, 1),
 Node((1, 0, 0, 1, 1), None, None, 1),
 Node((1, 0, 0, 1, 0), None, None, 1)]

In [296]:
tmps = dict()
for guess in TestNode.solution_candidates:
    tmps[guess] = Node(guess, guess=guess, solution_candidates=TestNode.solution_candidates)
    tmps[guess].spawn_children()
    print(f'{guess}: {max(x.nsol for x in tmps[guess].children)}')
    if max(x.nsol for x in tmps[guess].children) == 1: # can't do any better
        print(f'score for {guess} = {tmps[guess].calculate_score(calculate_from_children=True)}')

borne: 16
breed: 12
broke: 13
brute: 11
chore: 13
clerk: 32
credo: 10
creed: 13
creek: 16
creep: 11
creme: 15
crepe: 10
crept: 9
crone: 11
crude: 10
cruel: 14
curve: 13
drone: 9
drove: 11
egret: 21
entry: 32
erect: 12
erode: 11
erupt: 15
every: 31
exert: 28
force: 13
forge: 16
forte: 13
freed: 14
froze: 15
gorge: 17
greed: 13
green: 14
greet: 12
grope: 8
grove: 12
gruel: 19
horde: 16
ombre: 20
overt: 16
preen: 12
probe: 11
prone: 10
prove: 12
prude: 11
prune: 11
puree: 11
purge: 14
query: 24
rhyme: 28
rodeo: 18
rogue: 15
rouge: 15
route: 13
rupee: 16
there: 21
three: 16
threw: 23
trend: 12
trope: 9
trove: 11
truce: 10
where: 27
wreck: 17
wrote: 11


In [294]:
tmps['borne'].children

[Node((2, 2, 2, 2, 2), None, None, 1),
 Node((2, 0, 1, 0, 1), None, None, 1),
 Node((2, 1, 1, 0, 2), None, None, 1),
 Node((2, 0, 1, 0, 2), None, None, 1),
 Node((0, 1, 1, 0, 2), None, None, 10),
 Node((0, 0, 1, 0, 1), None, None, 16),
 Node((0, 1, 1, 0, 1), None, None, 2),
 Node((0, 0, 1, 0, 2), None, None, 9),
 Node((0, 1, 1, 2, 2), None, None, 3),
 Node((0, 0, 2, 0, 2), None, None, 4),
 Node((0, 0, 2, 0, 1), None, None, 2),
 Node((0, 0, 1, 1, 1), None, None, 3),
 Node((0, 2, 2, 0, 2), None, None, 5),
 Node((1, 1, 1, 0, 2), None, None, 2),
 Node((0, 0, 1, 2, 2), None, None, 1),
 Node((0, 2, 1, 0, 1), None, None, 1),
 Node((0, 2, 1, 0, 2), None, None, 3),
 Node((0, 0, 1, 2, 1), None, None, 1)]

In [101]:
# pare down empties 
pared_top_tree = {key: val for key, val in top_level_tree.items() if val != []}

In [159]:
# score singletons
to_pare_further = {key: val for key, val in pared_top_tree.items() if len(val) > 2}

In [160]:
len(pared_top_tree)

50

In [161]:
len(to_pare_further)

30

In [162]:
to_pare_further.keys()

dict_keys([(0, 0, 0, 0, 0), (0, 0, 0, 0, 1), (0, 0, 0, 1, 0), (0, 0, 0, 1, 1), (0, 0, 0, 2, 0), (0, 0, 0, 2, 1), (0, 0, 0, 2, 2), (0, 0, 1, 0, 0), (0, 0, 1, 0, 1), (0, 0, 1, 1, 0), (0, 0, 1, 1, 1), (0, 0, 1, 2, 0), (0, 0, 1, 2, 2), (0, 0, 2, 0, 0), (0, 0, 2, 0, 1), (0, 0, 2, 2, 0), (0, 1, 0, 0, 0), (0, 1, 0, 0, 1), (0, 1, 0, 1, 0), (0, 1, 0, 1, 1), (0, 1, 1, 0, 0), (0, 1, 1, 1, 0), (0, 1, 2, 0, 0), (0, 2, 0, 0, 0), (0, 2, 0, 0, 1), (0, 2, 0, 1, 0), (0, 2, 2, 0, 0), (1, 0, 0, 0, 0), (1, 0, 0, 1, 0), (1, 1, 0, 0, 0)])

In [166]:
key = (1, 1, 0, 0, 0)
to_pare_further[key]

['boozy', 'fuzzy', 'jazzy', 'woozy']

In [168]:
new_guess_list = words_containing_these_letters(unique_letters_in_list(to_pare_further[key]))
len(new_guess_list)

9244

In [172]:
next_guess = {guess: -1 for guess in new_guess_list}
candidates = to_pare_further[key]
for guess in new_guess_list:
    next_tree = {key: [] for key in product([0,1,2], repeat=5)}
    for word in candidates:
        score = tuple(score_guess(guess, word))
        next_tree[score].append(word)
    next_tree_counts = {key: len(val) for key, val in next_tree.items()}
    next_guess[guess] = max(next_tree_counts.values())

In [188]:
from collections import defaultdict

In [192]:
def test_guess_vs_candidates(guess, candidates):
    out = defaultdict(list)
    for word in candidates:
        out[tuple(score_guess(guess, word))].append(word)
    return out

In [190]:
candidates

['boozy', 'fuzzy', 'jazzy', 'woozy']

In [201]:
test_guess_vs_candidates(best_choices[6], candidates)

defaultdict(list,
            {(0, 1, 2, 1, 0): ['boozy'],
             (0, 0, 0, 0, 0): ['fuzzy'],
             (1, 0, 0, 0, 0): ['jazzy'],
             (0, 0, 2, 1, 0): ['woozy']})

In [194]:
best_choices = [key for key, val in next_guess.items() if val == 1]
len(best_choices)

329

In [170]:
guess

'aahed'

In [163]:
to_pare_further[(1, 1, 0, 0, 0)]

['boozy', 'fuzzy', 'jazzy', 'woozy']

In [164]:
guess

'zymic'

In [133]:
set(set("abc"))

{'a', 'b', 'c'}

In [105]:
def unique_letters_in_list(list_of_words):
    return set("".join(list_of_words))

In [134]:
def words_containing_these_letters(letters, word_list=allowed_list, excluded=None):
    if excluded:
        word_list = [word for word in word_list if not set(word) & set(excluded)]
    return [word for word in word_list if set(word) & letters]

In [135]:
guess

'zymic'

In [151]:
key = (0, 0, 1, 0, 1)
excluded = 'mi'

In [152]:
pared_top_tree[key]

['champ',
 'charm',
 'chasm',
 'chump',
 'clamp',
 'clump',
 'cramp',
 'cream',
 'creme',
 'crumb',
 'crump',
 'emcee',
 'macaw',
 'macho',
 'macro',
 'march',
 'match',
 'mecca',
 'mocha',
 'mucus',
 'mulch',
 'munch',
 'scamp',
 'scram',
 'scrum',
 'smack',
 'smock']

In [142]:
unique_letters_in_list(pared_top_tree[key])
next_up = words_containing_these_letters(test_set, word_list=soln_list, excluded=excluded)
len(next_up)

1204

In [125]:
test_set = unique_letters_in_list(pared_top_tree[(1,1,0,0,1)])
print(test_set)

{'z', 'c', 'y', 'r', 'a'}


In [70]:
top_level_tree

{(0, 0, 0, 0, 0): ['abase',
  'abate',
  'abbot',
  'abhor',
  'abled',
  'abode',
  'abort',
  'about',
  'above',
  'abuse',
  'adage',
  'adapt',
  'adept',
  'adobe',
  'adopt',
  'adore',
  'adorn',
  'adult',
  'afoot',
  'afoul',
  'after',
  'agape',
  'agate',
  'agent',
  'aglow',
  'agora',
  'agree',
  'ahead',
  'alert',
  'algae',
  'allot',
  'allow',
  'aloft',
  'alone',
  'along',
  'aloof',
  'aloud',
  'alpha',
  'altar',
  'alter',
  'angel',
  'anger',
  'angle',
  'angst',
  'ankle',
  'annex',
  'annul',
  'anode',
  'aorta',
  'apart',
  'apnea',
  'apple',
  'apron',
  'arbor',
  'ardor',
  'arena',
  'argue',
  'arose',
  'arrow',
  'arson',
  'ashen',
  'askew',
  'asset',
  'atoll',
  'atone',
  'augur',
  'avert',
  'awake',
  'award',
  'aware',
  'awash',
  'awful',
  'awoke',
  'badge',
  'bagel',
  'baker',
  'baler',
  'banal',
  'banjo',
  'barge',
  'baron',
  'basal',
  'baste',
  'bathe',
  'baton',
  'beard',
  'beast',
  'began',
  'begat',
  'b

In [62]:
max(first_guess.values())

183

In [4]:
with open('wordle_solution_candidates.txt', 'w') as file:
    file.writelines('\n'.join(soln_list))
with open('wordle_allowed_guesses.txt', 'w') as file:
    file.writelines('\n'.join(allowed_list))

In [12]:
file_name = 'Collins Scrabble Words (2019).txt'

In [13]:
with open(file_name) as FILE:
    words = [w.rstrip() for w in FILE if len(w.rstrip()) == 5]

In [17]:
wordles == [word.lower() for word in words]

True

In [30]:
soln_letter_count = {letter: "".join(soln_list).count(letter) for letter in 'abcdefghijklmnopqrstuvwxyz'}

In [33]:
sorted(soln_letter_count, key=soln_letter_count.get, reverse=True)

['e',
 'a',
 'r',
 'o',
 't',
 'l',
 'i',
 's',
 'n',
 'c',
 'u',
 'y',
 'd',
 'h',
 'p',
 'm',
 'g',
 'b',
 'f',
 'k',
 'w',
 'v',
 'z',
 'x',
 'q',
 'j']

In [24]:
smooshed_words = ''.join(words)
smooshed_words[0:15]

'AAHEDAALIIAARGH'

In [25]:
letter_count = {letter: smooshed_words.count(letter) for letter in 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'}

In [30]:
def score_word(word):
    return sum(letter_count[letter] for letter in set(word))

In [61]:
word1 = 'FUFFY'
word2 = 'FUZZY'
word3 = 'UNLIT'
word4 = 'AROSE'

In [69]:
def total_miss(word):
    return sum(not set(word) & set(w) for w in words)

In [72]:
total_misses = [total_miss(word) for word in words]

In [21]:
def score_word2(word, word_list=soln_list):
    hits = sum(min(word.count(letter), w.count(letter)) for letter in set(word) for w in word_list)
    return hits

In [ ]:
def score_wordle(guess, word):
    if not set(guess) & set(word):
        # no overlap
        
    perfect_matches = [l1 == l2 for l1, l2 in zip(guess, word)]
    

In [22]:
scores2 = [score_word2(word) for word in wordles]

In [27]:
best_words2 = sorted(zip(wordles, [s / len(soln_list) for s in scores2]), key=lambda x: x[1], reverse=True)

In [29]:
best_words2

[('oater', 1.7892008639308856),
 ('orate', 1.7892008639308856),
 ('roate', 1.7892008639308856),
 ('realo', 1.7809935205183585),
 ('alert', 1.778401727861771),
 ('alter', 1.778401727861771),
 ('artel', 1.778401727861771),
 ('later', 1.778401727861771),
 ('ratel', 1.778401727861771),
 ('taler', 1.778401727861771),
 ('irate', 1.7779697624190065),
 ('retia', 1.7779697624190065),
 ('terai', 1.7779697624190065),
 ('ariel', 1.7697624190064796),
 ('raile', 1.7697624190064796),
 ('aeros', 1.7680345572354212),
 ('arose', 1.7680345572354212),
 ('soare', 1.7680345572354212),
 ('arets', 1.7654427645788338),
 ('aster', 1.7654427645788338),
 ('earst', 1.7654427645788338),
 ('rates', 1.7654427645788338),
 ('reast', 1.7654427645788338),
 ('resat', 1.7654427645788338),
 ('stare', 1.7654427645788338),
 ('stear', 1.7654427645788338),
 ('strae', 1.7654427645788338),
 ('tares', 1.7654427645788338),
 ('taser', 1.7654427645788338),
 ('tears', 1.7654427645788338),
 ('teras', 1.7654427645788338),
 ('arles', 1.7

In [75]:
sorted(zip(words, total_misses), key=lambda x: x[1])

[('STOAE', 555),
 ('TOEAS', 555),
 ('ALOES', 560),
 ('AEONS', 565),
 ('AEROS', 577),
 ('AROSE', 577),
 ('SOARE', 577),
 ('AESIR', 626),
 ('ARISE', 626),
 ('RAISE', 626),
 ('REAIS', 626),
 ('SERAI', 626),
 ('PASEO', 651),
 ('PSOAE', 651),
 ('ANISE', 693),
 ('ISNAE', 693),
 ('SAINE', 693),
 ('AISLE', 697),
 ('SAICE', 720),
 ('ARSEY', 722),
 ('AYRES', 722),
 ('EYRAS', 722),
 ('RESAY', 722),
 ('SAYER', 722),
 ('YEARS', 722),
 ('SAYNE', 748),
 ('YEANS', 748),
 ('YATES', 749),
 ('YEAST', 749),
 ('AIDES', 759),
 ('ASIDE', 759),
 ('IDEAS', 759),
 ('ASPIE', 762),
 ('PAISE', 762),
 ('SEPIA', 762),
 ('ZOEAS', 762),
 ('AMIES', 768),
 ('MAISE', 768),
 ('ABIES', 769),
 ('OAVES', 774),
 ('SOAVE', 774),
 ('AEGIS', 778),
 ('AURES', 785),
 ('URASE', 785),
 ('UREAS', 785),
 ('URSAE', 785),
 ('SAUTE', 789),
 ('AUNES', 796),
 ('USNEA', 796),
 ('OASES', 798),
 ('LYASE', 801),
 ('OSIER', 801),
 ('YALES', 801),
 ('SALUE', 806),
 ('YEAHS', 816),
 ('KAIES', 821),
 ('EARNS', 823),
 ('NARES', 823),
 ('NEARS', 823

In [31]:
scores = [score_word(word) for word in words]

In [34]:
best_words = sorted(zip(words, scores), key=lambda x: x[1], reverse=True)

In [57]:
best_words2

[('AEROS', 1.9111162503854455),
 ('AROSE', 1.9111162503854455),
 ('SOARE', 1.9111162503854455),
 ('AESIR', 1.8862935553499847),
 ('ARISE', 1.8862935553499847),
 ('RAISE', 1.8862935553499847),
 ('REAIS', 1.8862935553499847),
 ('SERAI', 1.8862935553499847),
 ('ALOES', 1.8498304039469626),
 ('ARLES', 1.8496762257169288),
 ('EARLS', 1.8496762257169288),
 ('LAERS', 1.8496762257169288),
 ('LARES', 1.8496762257169288),
 ('LASER', 1.8496762257169288),
 ('LEARS', 1.8496762257169288),
 ('RALES', 1.8496762257169288),
 ('REALS', 1.8496762257169288),
 ('SERAL', 1.8496762257169288),
 ('STOAE', 1.843586185630589),
 ('TOEAS', 1.843586185630589),
 ('ARETS', 1.843432007400555),
 ('ASTER', 1.843432007400555),
 ('EARST', 1.843432007400555),
 ('RATES', 1.843432007400555),
 ('REAST', 1.843432007400555),
 ('RESAT', 1.843432007400555),
 ('STARE', 1.843432007400555),
 ('STEAR', 1.843432007400555),
 ('STRAE', 1.843432007400555),
 ('TARES', 1.843432007400555),
 ('TASER', 1.843432007400555),
 ('TEARS', 1.84343200

In [56]:
best_words2[::-1]

[('FUFFY', 0.43031144002466853),
 ('GYPPY', 0.4339346284304656),
 ('HYPHY', 0.440872648781992),
 ('XYLYL', 0.4413351834720937),
 ('FUZZY', 0.4540548874498921),
 ('YUKKY', 0.463691026827012),
 ('IMMIX', 0.463691026827012),
 ('BUBBY', 0.4697810669133518),
 ('JUGUM', 0.4786463151403022),
 ('QAJAQ', 0.49159728646315143),
 ('BUZZY', 0.4948350292938637),
 ('MUMMY', 0.4966851680542707),
 ('CIVIC', 0.49784150477952516),
 ('COCCO', 0.49830403946962687),
 ('YUMMY', 0.49953746530989823),
 ('PUPPY', 0.5),
 ('YUPPY', 0.5027752081406105),
 ('GYNNY', 0.5059358618563059),
 ('HUZZY', 0.5094048720320691),
 ('KUDZU', 0.5120259019426457),
 ('HUDUD', 0.5137218624730188),
 ('FLUFF', 0.5137989515880358),
 ('VIVID', 0.5203515263644773),
 ('WHIZZ', 0.520968239284613),
 ('MUZZY', 0.5217391304347826),
 ('VUGGY', 0.522895467160037),
 ('PHPHT', 0.5245914276904101),
 ('KIBBI', 0.5260561208757324),
 ('ZOPPO', 0.5272124576009868),
 ('ZOCCO', 0.5282917052112241),
 ('DUDDY', 0.5334566759173605),
 ('JINNI', 0.5390841813

In [49]:
best_words2[::-1]

[('XYLYL', 0.41874807277212456),
 ('FUFFY', 0.42067530064754854),
 ('GYPPY', 0.4208294788775825),
 ('HYPHY', 0.4335491828553808),
 ('QAJAQ', 0.4417206290471785),
 ('IMMIX', 0.442799876657416),
 ('COCCO', 0.44950662966389143),
 ('FUZZY', 0.45081714461917977),
 ('YUKKY', 0.45567375886524825),
 ('BUBBY', 0.4614554424915202),
 ('JUGUM', 0.47301880974406413),
 ('CIVIC', 0.47664199814986125),
 ('ZOPPO', 0.4769503546099291),
 ('ZOCCO', 0.47964847363552265),
 ('YUMMY', 0.4883595436324391),
 ('MUMMY', 0.4883595436324391),
 ('YUPPY', 0.4896700585877274),
 ('PUPPY', 0.4896700585877274),
 ('GYNNY', 0.49036386062288007),
 ('BUZZY', 0.49159728646315143),
 ('JEEZE', 0.4922139993832871),
 ('BOFFO', 0.49491211840888066),
 ('HUDUD', 0.4966080789392538),
 ('FLUFF', 0.5041628122109159),
 ('KIBBI', 0.5050878815911193),
 ('VIVID', 0.505781683626272),
 ('HUZZY', 0.5061671292013568),
 ('KUDZU', 0.5063983965464076),
 ('JAFFA', 0.509481961147086),
 ('PHPHT', 0.5107924761023743),
 ('JINNI', 0.5137989515880358),


In [36]:
best_words[::-1]

[('FUFFY', 5700),
 ('XYLYL', 5733),
 ('GYPPY', 5737),
 ('HYPHY', 5853),
 ('IMMIX', 6023),
 ('YUKKY', 6090),
 ('FUZZY', 6134),
 ('BUBBY', 6212),
 ('QAJAQ', 6393),
 ('JUGUM', 6422),
 ('COCCO', 6466),
 ('CIVIC', 6481),
 ('YUMMY', 6561),
 ('MUMMY', 6561),
 ('YUPPY', 6604),
 ('PUPPY', 6604),
 ('BUZZY', 6646),
 ('GYNNY', 6670),
 ('HUDUD', 6724),
 ('HUZZY', 6779),
 ('ZOPPO', 6891),
 ('KIBBI', 6891),
 ('ZOCCO', 6900),
 ('KUDZU', 6903),
 ('VIVID', 6906),
 ('VUGGY', 6923),
 ('WHIZZ', 6992),
 ('MUZZY', 6995),
 ('FLUFF', 6997),
 ('JINNI', 7002),
 ('DUDDY', 7038),
 ('PHPHT', 7074),
 ('BOFFO', 7180),
 ('JIFFY', 7239),
 ('TUKTU', 7311),
 ('FUBBY', 7327),
 ('BUFFY', 7327),
 ('FUGGY', 7344),
 ('FIZZY', 7382),
 ('JEEZE', 7387),
 ('OXBOW', 7392),
 ('JAFFA', 7396),
 ('CHUFF', 7414),
 ('BUTUT', 7433),
 ('HUFFY', 7460),
 ('ZIZIT', 7488),
 ('QUIFF', 7497),
 ('NUNNY', 7537),
 ('CHICH', 7547),
 ('BUCKU', 7671),
 ('WHIFF', 7673),
 ('GOBBO', 7709),
 ('PYGMY', 7713),
 ('PUFFY', 7719),
 ('MIMIC', 7763),
 ('JIGGY',

In [32]:
scores

[16865,
 13120,
 13552,
 17202,
 9645,
 13404,
 11150,
 16310,
 12027,
 9122,
 11612,
 13022,
 20944,
 16042,
 15787,
 17574,
 9691,
 14282,
 16732,
 20944,
 16353,
 15350,
 16307,
 16255,
 18437,
 17650,
 25102,
 24239,
 17973,
 20491,
 24703,
 20103,
 21808,
 24315,
 20945,
 16465,
 15791,
 21170,
 15791,
 19185,
 14031,
 15007,
 18666,
 22875,
 19508,
 17861,
 19411,
 13751,
 13849,
 17510,
 18486,
 22199,
 23018,
 21336,
 13080,
 19742,
 23455,
 20088,
 10562,
 23018,
 18332,
 16356,
 18442,
 15935,
 14683,
 14530,
 20465,
 25503,
 17975,
 13936,
 18893,
 23105,
 14216,
 20895,
 16304,
 16373,
 14729,
 16185,
 20343,
 23321,
 13753,
 20085,
 24297,
 13961,
 18280,
 19566,
 21291,
 25503,
 18388,
 23279,
 20428,
 18024,
 18703,
 19909,
 20486,
 20128,
 16749,
 13757,
 16147,
 17182,
 17803,
 8731,
 15105,
 19263,
 16640,
 18476,
 17081,
 20419,
 13155,
 21375,
 23305,
 22162,
 13371,
 20033,
 17130,
 17473,
 14466,
 21170,
 14508,
 18195,
 23701,
 19991,
 16872,
 19977,
 12601,
 192